In [ ]:
# # Learnt from https://medium.com/analytics-vidhya/custom-named-entity-recognition-ner-model-with-spacy-3-in-four-steps-7e903688d51
# !git clone https://github.com/Vijayvj1/Custom_NER_Spacy3.git

In [ ]:
import os


os.environ['KAGGLE_USERNAME'] = "khanfashee" # username from the json file
os.environ['KAGGLE_KEY'] = "97583e78a52b19db942cca0456cd0a66" # key from the json file
!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6
!kaggle -v
!kaggle datasets download -d khanfashee/tender-training-scripts

Found existing installation: kaggle 1.5.12
Uninstalling kaggle-1.5.12:
  Successfully uninstalled kaggle-1.5.12
     |████████████████████████████████| 1.7 MB 30.7 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
     |████████████████████████████████| 58 kB 5.7 MB/s             
  Preparing metadata (setup.py) ... done
  Created wheel for kaggle: filename=kaggle-1.5.6-py3-none-any.whl size=72857 sha256=e327f0f135651e969df95b37e32a55899c6c99150dedd29fbc1b3f17c8b7b2b0
  Stored in directory: /root/.cache/pip/wheels/aa/e7/e7/eb3c3d514c33294d77ddd5a856bdd58dc9c1fabbed59a02a2b
Successfully built kaggle
Kaggle API 1.5.6
 64% 12.0M/18.9M [00:00<00:00, 125MB/s]
100% 18.9M/18.9M [00:00<00:00, 155MB/s]


In [ ]:
!unzip \*.zip

Archive:  tender-training-scripts.zip
  inflating: content/.config/.last_opt_in_prompt.yaml  
  inflating: content/.config/.last_survey_prompt.yaml  
  inflating: content/.config/.last_update_check.json  
  inflating: content/.config/active_config  
  inflating: content/.config/config_sentinel  
  inflating: content/.config/configurations/config_default  
  inflating: content/.config/gce     
  inflating: content/.config/logs/2021.06.15/13.36.40.402408.log  
  inflating: content/.config/logs/2021.06.15/13.36.59.704686.log  
  inflating: content/.config/logs/2021.06.15/13.37.15.895583.log  
  inflating: content/.config/logs/2021.06.15/13.37.22.745818.log  
  inflating: content/.config/logs/2021.06.15/13.37.39.858399.log  
  inflating: content/.config/logs/2021.06.15/13.37.40.569743.log  
  inflating: content/Custom_NER_Spacy3/.git/HEAD  
  inflating: content/Custom_NER_Spacy3/.git/config  
  inflating: content/Custom_NER_Spacy3/.git/description  
  inflating: content/Custom_NER_Spacy3/.

In [ ]:
!python -m pip install -r /content/content/Custom_NER_Spacy3/requirements.txt

     |████████████████████████████████| 8.3 MB 23.0 MB/s            
     |████████████████████████████████| 5.9 MB 40.2 MB/s            
     |████████████████████████████████| 42 kB 1.4 MB/s             
     |████████████████████████████████| 628 kB 50.3 MB/s            
     |████████████████████████████████| 10.1 MB 52.5 MB/s            
     |████████████████████████████████| 451 kB 55.2 MB/s            
     |████████████████████████████████| 111 kB 50.6 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 180 kB 71.8 MB/s            
     |████████████████████████████████| 76 kB 5.4 MB/s             
     |████████████████████████████████| 4.3 MB 70.9 MB/s            
     |████████████████████████████████| 63 kB 1.8 MB/s             
     |████████████████████████████████| 124 kB 58.8 MB/s            
     |████████████████████████████████| 790 kB 56.8 MB/s            
     |████████████████████████████████| 374 kB 58.1 MB/s        

In [ ]:
!python -m spacy info



============================== Info about spaCy ==============================

spaCy version    3.1.4                         
Location         /usr/local/lib/python3.7/dist-packages/spacy
Platform         Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic
Python version   3.7.12                        
Pipelines                                      



In [ ]:
### Training Data Preprocessing ###
# convert admin.jsonl from doccano to train.json
import sys
import json


# fo = open('/content/admin_75.jsonl', "r")
fo = open('/content/synesisit.jsonl', "r")
train_data = open('/content/train.json', "w")


lines = fo.readlines()


for line in lines:
    line =json.loads(line)
    if "label" in line:
    	line["entities"] = line.pop("label")
    else:
	    line["entities"] = []

    tmp_ents = []
    for e in line["entities"]:
        if e[2] in ['inviter','inviter_location','tender_type','published_date','memo_no','international','co_inviter','call_type','description']:
     	    tmp_ents.append({"start": e[0], "end": e[1], "label": e[2]})

        line["entities"] = tmp_ents

    if (len(line["data"]) > 5):
        # print (json.dumps({"data": line["data"],"entities": line["entities"]},),file=train_data)
         print (json.dumps([line["data"],{"entities": [line["entities"]]}],),file=train_data)


train_data.close()



In [ ]:
def convert(input):
    text=input[0]
    annotations=input[1]
    labels=annotations.get('entities')
    data=labels[0]
    annotation_list=[]
    for item in data:
        start=item.get('start')
        end=item.get('end')
        label=item.get('label')
        temp_list=[start,end,label]
        annotation_list.append(temp_list)
    d=dict(
        entities=annotation_list
    )
    final_list=[text,d]
    return final_list

In [ ]:
import sys
import json


fo = open('/content/train.json', "r")

train_data = open('/content/train_final_document.json', "w")


lines = fo.readlines()

lines_list = []
count=0
for line in lines:
    line =json.loads(line)
    datas = convert(line)
    # dump_value = datas+','

    print(json.dumps(datas,),file=train_data)

    count = count + 1



train_data.close()

count

148

In [ ]:
def trim_entity_spans(data: list) -> list:
    """Removes leading and trailing white spaces from entity spans.

    Args:
        data (list): The data to be cleaned in spaCy JSON format.

    Returns:
        list: The cleaned data.
    """
    invalid_span_tokens = re.compile(r'\s')
    train_data_ = open('/content/train_final_docu.json', "w")
    cleaned_data = []
    for text, annotations in data:
        entities = annotations['entities']
        valid_entities = []
        for start, end, label in entities:
            valid_start = start
            valid_end = end
            while valid_start < len(text) and invalid_span_tokens.match(
                    text[valid_start]):
                valid_start += 1
            while valid_end > 1 and invalid_span_tokens.match(
                    text[valid_end - 1]):
                valid_end -= 1
            valid_entities.append([valid_start, valid_end, label])
        cleaned_data.append([text, {'entities': valid_entities}])
        print(json.dumps([text,{"entities": valid_entities}],),file=train_data_)
    train_data_.close()
    return cleaned_data

In [ ]:
with open('/content/train_final_document.json', 'r') as istr:
    with open('/content/output.json', 'w') as ostr:
        for line in istr:
            line = line.rstrip('\n') + ','
            print(line, file=ostr)

In [ ]:
rr = [["Image No. 1171\nfile_name:/content/content/jpg_files/20210901-100100768-946.txt\n\nGOVERNMENT OF THE PEOPLE'S REPUBLIC OF BANGLADESH\nMINISTRY OF HEALTH AND FAMILY WELFARE\nDIRECTORATE GENERAL OF FAMILY PLANNING\nLOGISTICS & SUPPLY UNIT (5TH FLOOR)\n6, KARWANBAZAR, DHAKA-1215, BANGLADESH.\nwww.dgfp.gov.bd\ne-Tender Notice (OTM, NCT, Goods)\nFile Name: DGFP/L&SS-2/Delivery Table/2021-22//55/13\nDate: 31/08/2021\ne-Tender\nis invited in the national e-GP pottal for the procurement of following goods:\nTender ID No &\nTender Package name\nLast Down-\nClosing\nOpening\nPackage No.\nloading/selling\nDate &\nDate\nDate &Time\nTime\n&Time\n1\n2\n3\n4\n5\nTender ID No: 598911\nProcurement of Delivery\n26/09/2021\n27/09/2021\n27/09/2021\nPackage No\nTable & Iron Cot Set\n16:00\n14:00\n14:00\n-\nMCH/GD43/e31\nThis is an online tender where only e-tender (online tender) will be accepted through the national e-GP\nportal\nand no offline/hard copy will be accepted. The fees for downloading the e-tender documents\nfrom\nthe\nnational e-GP portal have to be deposited online through any schedule bank of Bangladesh\nregistered by the Central Procurement Technical Unit (CPTU).\nFurther information visit national e-GP portal and from e-GP help desk (Help Desk e-mail:\nhelpdesk@eprocure.gov.bd).\n00/31/08/2021\n(Ratna Talukder)\nDirector (Logistics & Supply) &\nLine Director (PSSM-FP)\n(8xx)\nLogistics & Supply Unit\nTelenhone: 880-2-55012349\n", {"entities": [[168, 206, "inviter"], [246, 264, "inviter_location"], [299, 314, "call_type"], [646, 669, "description"], [714, 734, "description"]]}],
["Image No. 1237\nfile_name:/content/content/jpg_files/20210901-100100768-949.txt\n\ngreferry 31940B 91132 2000 IST (8) 411st\n: 676 TR->, I\nTi- 356/2025\nBank Asia Limited, Scotia Branch, Rangs Bhaban, 113-116, Old Airport\nRoad, Tejgaon, Dhaka at present 32, Kazi Nazrul Islam Avenue, Karwan\nBazar, Dhaka-1215.\nVersus\nDecree-Holder.\n(1) Mr. Taufiqur Rahman Mollick, Proprietor M/s National Dairy &\nFood Products, Hamid Tower (1st Floor), Gulshan Circle-2, Gulshan,\nDhaka-1212 others address: Mr. Taufiqur Rahman Mollick, Proprietor\nM/s National Dairy & Food Products, 121/C, Gulshan Avenue (5th\nFloor), Gulshan-2, Dhaka And Mr. Taufiqur Rahman Mollick,\nProprietor M/s National Dairy & Food Products, Bashati Dream, Flat\nNo. A-1, Road No. 20, House No.3, Gulshan-1, Dhaka-1212.\n(2) Mrs. Rokeya Begum, Wife of late Tofail Ahmed, 403, Purbo Goran,\nP.S- Sobujbag, Dhaka.\n(3) Mr. Md. Solaiman Mollick, House# 47, Road # 03, Sector # 05, uttara, Dhaka.\n(4) Mr. Shafiqur Rahman Mollick, Son of Md. Solaiman Mollick, House\n# 47, Road # 03, Sector # 05, Uttara, Dhaka.\nJudgment-Debtors.\n959151 03/09/2015 9423\nMY electric SIGNED Pet 2114\nPrice R <19 smy R R THE\nPatical 3/12/37 Firth first famal\n4593 01/24/19 281 VICCEE\n151997\n5\nCITY 20/08/2025 officer 2.00 Al\n9519 Path 4599 hilling\n2\nI family 4514 71141 APICE 4599 TN, for THE,\n(TITY 2145 e FACT\n264 I 593 \"yours PATIENT 45% Pacer 4/12/19 264\n9 I FITCH farm 45194 41914 91500 9008\n4/1/19 I\n8 1 21605 EGU 48 0,00,000/- 5218 \u20b90%,\nEBU 45 00,000/- (44T OFFIRE $25 00,00,000/-\n) trat 269 52131 50% 95 535 48 0,00,000/- Blot\nBICTOR Eaks 10% 9 754 est\n9 45269 C 69101 43 at 6900-6\n9/5/8 at 69-01615 4519945 sticet 91129 264\nI\na 9514151 $0,00,000/- (441 Blaks EGG 00\nThe 10,00,000/- (49 blat\n00,00,000/- ***) 565 455 2415 to (GIB)\n20045 952 BIRTH Egu 45\n99901 NO (7512) Mark 2065 022 31 RCG\nBIST\nG at BICHS after (4Th cot GAIRO RICKPAY) 43 styces E for\nsofter 20/08/2025 for RICK 1.00 4574 2115$ callet\n9 45\u00b0T THE\nsports 4994 264 052 294 264\n45% 4895 </18 401 5995\nb. 02/15/19 34000 e meal family T22\n364\n& EXCS alto (8) e () 4 water 2025 MCTIBS 45141515 01/4/09\nROOT 7/45/100 spenfirs 61/24/20 PSICS 9151649\nso I Patrica with 252 30B BIN\nPRICS\nss I a cat 45194 21/Ba a 21133\n01/4/19\nSCHEDULE OF THE PROPERTY\nA. All that piece and parcel of land measuring 116 (one hundred and sixteen)\ndecimals situated within District-Dhaka, Sub-Registry Office-Gulshan,\nDhaka, Police Station-Uttara (Sabek Keranigonj thereafter Gulshan), under\nDhaka Collectorate Touzi, Mouza-Uttor Khan, J.L. No. C.S.195, S.A. 17, R.S.\n17, C.S. Khatian No. 330 & 138, S.A. Khatian No. 239, R.S. Khatian No.\n1600, C.S. and S.A. Dag No. 878, 879, 881, 890,893,894, R.S. Dag No.2924,\n2871, 2853, 2843, 2849/3023, 2849/3024, 2849/3025, 2848, 2870, 2864,\n2865, 2866, 2867, 2868, 2869, 2846 butted & bounded by: on the North-\nKatcha Road, On the South Katcha Road, On the East-Residential house of\nother owners, On the West- Own Vacant land along with all constructions and\nstructures constructed or to be constructed thereon with all other rights,\ninterests, title, easements etc. attached or appertaining to the land.\nB. All that piece and parcel of land measuring 70(seventy) decimals situated\nwithin District- Dhaka, Police Station-Uttara, Sub-Registry Office- Gulshan,\nMuuza-Uttara, J.L. No. Sabek 195, Hal S.A. 17, C.S. Khatian No.330 and 138\nC.S. Dag No. 878, 879, 881, 890, 893, and 894, butted & bounded by: on the\nNorth-Katcha Road, On the South Katcha Road ,On the East- Residential\nhouse of other owners, On the West-Own Vacant land along with all constructions\nand Structures constructed or to be constructed thereon with all other rights,\ninterests, title, easements etc. attached or appertaining to the land.\n31/4/95 - ), I\n33\n", {"entities": [[148, 165, "inviter"], [279, 298, "inviter_location"]]}],
["Image No. 1096\nfile_name:/content/content/jpg_files/20210901-100100768-947.txt\n\nOffice of the Fatickchari Pourashava\n7956\n642 2928\n*\nDistrict : Chattogram.\nMemo No. : Fatick/Poura/Prokow:/2021-279\nDate : 29/08/2021\ne-Tender (OTM) Notice No: 01/2021-2022\ne-Tender\nis\ninvited in the National e-GP system portal (http//:www.eprocure.gov.bd) for the procurement of works as\nmentioned in the following table:\nTender\nPackage\nDescription of work\nOnline (e-GP\nOnline (e- GP\nID No.\nNo.\nSystem) tender\nSystem) tender\npublication\nclosing date &\ndate & time\ntime\n599178\nUIIP-\n(A) Construction of Culvert on the Amanbazar road\n01-09-2021\n23-09-2021\nBC/PCO/\n(3.50x3.00) 6m Width. (B) Construction of Culvert\n10:00 AM\n2:15 PM\nW-433\non the Nahir Md. Chy. Road 2 (2.50x3.00) 6m\nWidth. (C) Construction of Culvert on the Eidgah to\nGawsiya Market road 2 (2.50x3.00) 6m Width. (D)\nImprovement of Kabir Hazi road by H.B.B, Guide\nwall & sand filling. Ch. O+ 000 - Ch.0+ 490 m\n3\nStarting from Gohira - Heyak\u00f3 road. (E)\nImprovement of Rangamatia Sheba Khola road by\nH.B.B. Ch. 0-110, Ch. 570 - 875 m. Under\nFatickchari Pourashava.\n599179\nUIIP-\nConstruction of Drain Starting from Fatickchari\n01-09-2021\n23-09-2021\nBC/PCO/\ncentral Eidgah to Chilonia Chora through Soyodpara\n10:00 AM\n2:00 PM\nW-434\nCh. 0+000-0+900m Under Fatickchari Pourashava.\nThese are an online tender, where only e-tender will be accepted in the national e-GP portal and no offline/hard copies\nwill be accepted. To Submit e-tender, registration in the National e-GP system portal (http//:www.eprocure.gov.bd) is\nrequired.\nThe fees for downloading the e-tendering documents from the National e-GP portal have to be deposited online\nthrough any registered banks branches up to 22/09/2021 at 5.00 PM.\nFarther information and guidelines are available in the National e-GP system portal and from e-GP help desk\n(helpdesk@eproqure.gov.bd\nPl\n29/2021\nBikash Chandra Das\nAssistant Engineer\nPc-1077/21 (5\"x3)\nFatickchari Pourashava\nDistrict: Chattogram\nMobile: 01716060751\n", {"entities": [[106, 116, "inviter"], [215, 236, "call_type"], [144, 154, "inviter_location"], [568, 613, "description"], [861, 927, "description"], [1120, 1142, "description"]]}],
["Image No. 1056\nfile_name:/content/content/jpg_files/20210901-100100768-951.txt\n\nOffice of the Chief Planning, Dev & Works\nDevelopment of Bangladesh University of Professionals (Phase-1)\nBangladesh University of Professionals (BUP)\nMirpur Cantt., Dhaka-1216.\nPhone: 8000442, Fax: 8035903\nRef. No:BUP/P/G/Phase-1(1\" Revised)/2021-22/01\nD\u00e2te: 31:08.2021\nInvitation for Tender\n1\nMinistry/Division\nMinistry of Education\n2.\nAgency\nBangladesh University of Professional's (BUP), Mirpur Cantt., Dhaka\n3\nProcuring Entity Name\nVice ch\u00e0ncellor, BUP, Mirpur Cantt., Dhaka\n4\nInvitation for\nSupply and Installation of Light, Sound & Visual System of BUP Auditorium Under\nthe Project \"Development of Bangladesh University of Professionals (Phase-1) (1st\nRevised)\" at East Campus of BUP Mirpur Cantt.\n5\nInvitation Tender No. & Date\nBUP/P/G/Phase-1(1\" Revised)/2021-22/01, Date: 31 August 2021\n6\nProcurement Method\nOpen Tendering Method (OTM)\n7\nBudget and Source of Funds\nGoB\n8\nTender Selling Date\n02 September 2021 to 04 October 2021 during office time 8.00 am to 2.00 pm\n9\nTender Dropping last Date and Time\n05 October 2021 up to 11.00 am\n10\nTender Opening Date, Time & Place of\n05 October 2021 at 11:30 am\nOpening Tender Document.\nOffice of the Chief Planning Dev & Works, Bangladesh University of Professionals,\nMirpur Cantonment, Dhaka-1216\n11\nName & Address of the offic\u00e9(s) Selling\nOffice of the Chief Planning Dev & Works, Bangladesh University of Professionals,\nTender Document\nMirpur Cantonment, Dhaka-1216.\n12\nName & Address of the office(s) for\nOffice of the Chief Planning Dev & Works, Bangladesh University of Professionals,\nReceiving Tender Document\nMirpur Cantonment, Dhaka-1216.\n13\nPlace/Date/Time of Pre-Tender Meeting\nPre-Tender meeting will be hheld at the Office of the Chief Planning Dev & Works,\nBangladesh Univ\u00e9rsity of Professionals, Mirpur Cantonment, Dhaka. Date: 16\nSeptember 2021, Time: 11:00 am\nINFORMATION FOR TENDERER\nEligibility of Tenderers\nThis Invitation for Tender is open to all eligible Tenderers who satisfy the following:\n14\nExperience Criteria\na) General Experience : The Tenderer shall have a minimum of 10 (Ten) years of\noverall experience in Supply and installation of sound, light and visual system/\nrelated Thbsed products of Auditorium/ Banquet Hall/ Indoor or Outdoor Stadium/\nTheatre Hall.\nb) Specific Exp\u00e9rience:\ni. The Tenderer must have successfully completed minimum 02 (Two) numbers of\nSupply and installation of sound, light and visual system/ related ICT based\n15F\nbz\nproducts of Auditorium/ Banquet Hall/ Indoor or Outdoor Stadium/ Theatre Hall\nwithin last 10 (Ten) years, i.e., years counting backward from the date of\nits\npublication of IFT in the newspaper with at least 01 (One) Contract with a value of\nminim\u00fam Tk. 5,00,00,000.00 (Five Crore).\nii. The Tenderer must have successfully completed minimum 01 (One) numbers of\nSupply and installation of Sound and Light system of Auditorium/ Banquet Hall/\nIndoorior Outdoor Stadium/ Theatre Hall within last 10 (Ten) years, i.e., years\nL\ncounting backward from the date of publication of IFT in the newspaper with a\nvalue of minimum Tk. 2,00,00,000.00 (Two Crore).\niii. The Tenderer must have successfully completed minimum 01 (One) numbers of\nSupply and installation of Visual system (LCD/ LED video ,wall/ Commercial\nDisplay/ Giant Screen etc.) of Auditorium/ Banquet Hall/ Indoor or Outdoor\nStadium/ Theatre Hall within last 10 (Ten) years, i.e., years counting backward\nfrom the date. of publication of IFT in the newspaper, with at value of minimum\nTk. 1,50,00,000.00 (One Crore Fifty Lacs).\nFinancial Criteria\nMinimuni average annual turnover in similar works:\nThe satisfactory completion of supply of similar goots of minimum Tk\n17,50,00,000.00 (Seventeen Crore Fifty Lacs) under maximum [10] contracts in the\nlast [05] of years, i.e., years counting backward from the date of publication of IFT in\nthe newspaper.\nOther Requirements\nThe Tenderer must possess:\ni)\nUp-to-date income tax clearance certificate (FY 2020-21).-\nii)\nEnlistment certificate.\niii)\nOther Documents as mentioned in Tender Documents Sheet (TDS)\n15\nPrice of Tender Document (Tk)\nTk. 11,000/- (Eleven Thousand) only\n16\nAmount of Tender Security\nTk. 26,79;Q00/- (Twenty-Six Lacs Seventy-Nine Thousand) Only in favour of Chief\nPlanning, Dev & Works, BUP\n17\nName, Designation, Address & Cont\u00e1ct\nChief Planning, Dev & Works, BUP, Mirpur Cantonment, Dhaka-1216. Telephone\ndetails of Official Inviting Tender\nNo. 88-02-8000442; Fax No. 88-02-8035903. E-mail: planningbup@gmail.com\n18\nFor any clarifications please contact:\nAddl. Director (Civil), Office of the Chief Planning, BUP Tel: 8000598, Mobile:\n01769-021654\n19\nSpecial Instructions: The procuring entity reserves the right to reject all tenders \u00f3r annul the tender proceedings\nChief Planning, Dev & Works\nBUP\n(50'x8)\nMirpur Cantt., Dhaka-1216\n-\n", {"entities": [[186, 230, "inviter"], [231, 251, "inviter_location"], [577, 632, "description"]]}],
["Image No. 1276\nfile_name:/content/content/jpg_files/20210901-100100768-953.txt\n\n3/5/18\n3/5/90/8\nBanglad\nEIGHT\nRIOO\ncompany, FAIRBARI\n6th\nwww.lged.gov.bd\n2114 GAR\nMemo No- 46.02.4480.000.11.038.21-1257\n24\nDate : 31.08.2021\ne-Tender Notice No. : 01/2021-2022\nOpen Tendering Method (OTM)\n&\ne-Tender invited in the National e-GP' System Portal (http://www.eprocure.gov.bd) for the procurement of works mention below under Shailkupa,\nLGED,District-Jhenidah.\nSL\nPackage No\nTender\nName of Work\nTender\nTender last selling date &\nTender Opening date & time\nTo\nID No\nSecurity\ntime\n(TK)\n.\n1\n2\n3\n4\n6\n7\n8\n9\n10\n1\ne-Tender/PEDP4/JHE\n599813\nConstruction of\n3,70,000/-\n29/09/2021\n17.00\n30/09/2021\n13.00\n//SHA/2021-22 /\nAdditional\nPM\nPM\nW1. 01678\nClass Room\nKhalkula GPS\n2\ne-Tender/PEDP4/JHE\n599947\nConstruction of\n3,50,000/-\n29/09/2021\n17.00\n30/09/2021\n13.00\n//SHA/2021-22\nAdditional\nPM\nPM\nW1. 01679\nClass Room\nBardah GPS\n3\ne-Tender/PEDP4 /JHE\n599948\nConstruction of\n3,47,000/-\n29/09/2021\n17.00\n30/09/2021\n13.00\n//SHA/2021-22.\nAdditional\nPM\nPM\nWI. 01680\nClass Room\n.\nDhawrah GPS\n4\ne-Tender/PEDP4 /JHE\n599949\nConstruction of\n3,67,000/-\n29/09/2021\n17.00\n30/09/2021\n13.00\n//SHA/2021-22 .\nAdditional\nPM\nPM\nWI. 01698\nClass Room\nShohid\nMuktijoddah\nAzibor Rahman\nGPS\n5\ne-Tender/PEDP4/JHE\n599950\nConstruction of\n3,71,000/-\n29/09/2021\n17.00\n30/09/2021\n13.00\n//SHA/2021-22\nAdditional\nPM\nPM\nWI. '01699\nClass Room\n1\nBorutia GPS\n6\ne-Tender/PEDP4 /JHE\n599951\nConstruction of\n3,71,000/-\n29/09/2021\n17.00\n30/09/2021\n13.00\n//SHA/2021-22/\nAdditional\nPM\nPM\nWI. 01700\nClass Room\n.\nKhondokbaria\nGPS\n7\ne-Tender/PEDP4/JHE\n599952\nConstruction of\n3,78,000/-\n29/09/2021\n'17.00\n30/09/2021\n13.00\n//SHA/2021-22\nAdditional\nPM\nPM\nW1. 01701\nClass Room\nMohammadpur\nGPS\n8\ne-Tender/PEDP4/J\n603180\nConstruction of\n2,88,000/-\n29/09/2021\n17.00\n30/09/2021\n13.00\n//SHA/2021-22/\nAdditional\nPM\nPM\nW2.03484\nClass Room\nFulhari GPS\nThis is an online Tender, Where only e-Tender will be accepted in the National e-GP portal and no offline/hard copies will be accepted.\nTo submit -Tender, registration in the National e-GP System Portal (http://www.eprocure.gov.bd) is required.\nThe fees for downloading the e-Tender Documents from the National e-GP System portal have to be deposited online through registered Bank's branches up to\n12.30 pm on 30.09.2021\nFurther information and guidelines are available in the National e-GP System Portal and from e-GP help desk (helpdesk@eprocure.gov.bd).\n(Md. Rowshan Habib)\nUpazila Engineer\nSailkupa, Jhenidah\nPhone:04526-56059,\nDG-1700/21 (8\"x4)\ne-mail: ue.shailkupa@lged.gov.bd\n", {"entities": [[443, 451, "inviter_location"], [222, 237, "call_type"], [883, 904, "description"]]}],
["Image No. 1263\nfile_name:/content/content/jpg_files/20210901-100100768-952.txt\n\nFIT\nOffice of the Chief Plarming, Dev'& Works\nU.P\nDevelopment of Bangladesh University of Professionals (Phase-1)\nBangladesh University of Professionals (BUP)\nMirpur Cantt., Dhaka-1216.\nPhone: 8000442, Fax: 8035903\nDate: 31.08.2021\nRef. No: BUP/P/W/2nd Revised/2021-22/02\nInvitation for Tender\ni\n1\nMinistry/Division\nMinistry of Education\n2\nAgency\nBangladesh University of Professional's (BUP), Mirpur Cantt., Dhaka\n3\nProcuring Entity Name\nVice chancellor, BUP, MirpurCantt., Dhaka,\n4\nInvitation for\nConstruction of 14 Storied Academic Building (FASS) up to 14th Floor of\n15 Storied Foundati\u00f2n of BUP under the project \"Development of\nBangladesh University of Professionals (2nd Revised)\" including ancillary\nworks.\nInvitation Tender No. & Date\nBUP/P/W/2nd Revised/2021-22/02, Date: 31 August 2021\n5,\n6,\nProcurement Method\nOpen Tendering Method (OTM)\n7.\nBudget and Source of Funds\nGoB\n8\nTender Selling Date\n02 September 2021 to 30 September 2021 during office time 9.00 am to\n2.00 pm\n9\nTender Dropping last Date and Time\n03 October 2021 up tol 1.00 am\n10\nTender Opening Date, Time & Place\n03 October 2021 at 11.30 am\nof Opening Tender Document.\nOffice of the Chief Planning Dev & Works, Bangladesh University of\n1\nProfessionals, Mirpur Cantonment, Dhaka-1216\nName & Address of the office(s)\nOffice of the Chief Planning Dev & Works, Bangladesh University of\n11\nSelling Tender Document\nProfessionals, Mirpur Cantonment, Dhaka-1216.\nName & Address of the office(s) for\nOffice of the Chief Planning Dev & Works, Bangladesh University of\n12\nReceiving Tender Document\nProfessionals, Mirpur Cantonment, Dhaka-1216.\nPre-Tender\nPre-Tender meeting will be held at the Office of the Chief Planning Dev\n13\nPlace/Date/Time\nof\nMeeting\n& Works, Bangladesh University of Professionals, Mirpur Cantonment,\nDhaka. Date: 20 September 2021, Time: 11:00 am\nINFORMATION FOR TENDERER\nThis Invitation for Tender is open to all eligible Tenderers who satisfy the\nEligibility of Tenderers\nfollowing:\n14\nExperience Criteria\na) General Experience:\nReputed Contractors/Construction Firms who have minimum 15\n(Fifteen)\nyears of general experience in Construction Works.\nb) Contractor enlisted in MES class \"A\" or in other Govt./Semi\nb>\nGovt./autonomous organization in similar capacity.\nc) Specific Experience:\nHaving Experience of construction of minimum 10 (Ten) Storied\n(Including basement) RCC Building: at least 02 (Two) completed or 01\n(One) Completed and 02 (Two) Ongoing; BDT 30.00 Cr. in a single\ncontract\nin Government/Semi-Government/Autonomous Bodies/\nNGO/Banks/Reputed Private Organization of Bangladesh during last 10 (Ten) years.\nFinancial Criteria\nMinimum average annual construction turnover:\nThe Tenderer shall have a minimum average annual construction turnover\nduring last 5 (Five) years of BDT 60.00 Cr.\nOther Requirements\nThe Tenderer must possess:\ni) Up-to-date income tax clearance, Trade license and VAT certificate (FY 2021-22).\nii) Enlistment certificate.\niii) Other Documents as mentioned in Tender Document Sheet (TDS)\n15\nPrice of Tender Document (Tk)\nTk. 50,000/- (Fifty Thousand) only\n16\nAmount of Tender Security\nTk. 1,50,00,000/- (One Crore Fifty Lacs) Only in favour of Chief\nPlanning, Dev & Works, BUP\n17\nName, Designation, Address\n&\nChief Planning, Dev & Works, BUP, Mirpur Cantonment, Dhaka-1216.\nContact details of Official Inviting\nTelephone No. 88-02-8000442; Fax No. 88-02-8035903. e-mail:\nTender\nplanningbup@gmail.com\n18\nFor any clarifications please contact:\nAddl. Director (Civil), Office of the Chief Planning, BUP Tel: 8000598,\nMobile: 01769-021654\n19\nSpecial Instructions: The procuring entity reserves the right to reject all tenders or annul the tender proceedings\nChief Planning, Dev & Works\nBUP\nBIG-SESS/RS (5\u00f3\"x8)\nMirpur Cantt., Dhaka-1216\n", {"entities": [[239, 259, "inviter_location"], [194, 238, "inviter"], [564, 623, "description"]]}],
["Image No. 1012\nfile_name:/content/content/jpg_files/20210901-100100768-955.txt\n\nonthe ChiemPlanning nev & Morks\nyour\nDevelopment of Bangladesh University of Professionals (Phase-1)\nBangladesh University of Professiorials (BUP)\nMirpur Cantt., Dhaka-1216.\nPhone: 8000442, Fax: 8035903\nRef. No: BUP/P/W/2nd Revised/2021-22/01\nDate: 31.08.2021\nInvitation for Tender\n1\nMinistry/Division\nMinistry of Education\n2\nAgency\nBangladesh University of Professional's (BUP), Mirpur Cantt., Dhaka\n3\nProcuring Entity Name\nVice chancellor, BUP, Mirpur Cantt., Dhaka\n4\nInvitation for\nConstruction of Scientific Laboratory Building (FST) up to 14th Floor\nof\n15 Storied Foundation of BUP under the project \"Development of\nBangladesh University of Professionals (2nd Revised)\" including ancillary\nworks.\n5\nInvitation Tender No. & Date\nBUP/P/W/2nd Revised/2021-22/01, Date: 31 August 2021\n6\nProcurement Method\nOpen Tendering Method (QTM)\n7\nBudget and Source of Funds\nGoB\n8-\nTender Selling Date\n02 September 2021 to 03 October 2021 during office time 9.00 am to\n2.00 pm\n9\nTender Dropping last Date and Time\n04 October 2021 up to11.00 am\n10\nTender Opening Date, Time & Place\n04 October 2021 at 11:30 am\nof Opening Tender Document.\nOffice of the Chief Planning Dev & Works, Bangladesh University of\nProfessionals, Mirpur Cantonment, Dhaka-1216\n11\nName & Address of the office(s)\nOffice of, the Chief Planning Dev.& Works, Bangladesh University of\nSelling Tender Document\nProfessionals, Mirpur Cantonment, Dhaka-1216.\n12\nName & Address of the office(s) for\nOffice of the Chief Planning Dev & Works, Bangladesh University of\nReceiving Tender Document\nProfessionals, Mirpur Cantonment, Dhaka-1216\n13\nPlace/Date/Time\nof\nPre-Tender\nPre-Tender meeting will be held at the Office of the Chief Planning Dev\nMeeting\n& Works, Bangladesh University of Professionals, Mirpur Cantonment,\nDhaka. Date: 19 September 2021, Time: 11:00 am\nINFORMATION FOR TENDERER\n14\nEligibility of Tenderers\nThis Invitation for Tender is open to all eligible Tenderers who satisfy\nthe following:\nExperience Criteria\na) General Experience:\nReputed Contractors/Co\u00f3nstruction Firms who have minimum 15\n(Fifteen) years of general experience in Construction Works.\nb) Contractor enlisted in MES class \"A\" or in other Govt./Semi\nGovt./autonomous organization in similar capacity.\nc) Specific Experience:\nHaving Experience of construction of minimum 10 (Ten) Storied\n24\n(Including basement) RCC Building: at- least 02 (Two) completed or\n01\n(One) Completed and 02 (Two) Ongoing; BDT 30.00 Cr. in a single\ncontract in Government/Semi-Government/Autonomous Bodies/\nNGO/Banks/Reputed Private Organization of Bangladesh during last 10\n(Ten) years.\nFinancial Criteria\nMinimum average annual construction turnover:\nThe Tenderer shall have a- minimum average annual construction\nturnover during last 5 (Five) years of BDT 60.00 Cr.\nOther Requirements\nThe Tenderer must possess:\ni) Up-to-date income tax clearance certificate (FY 2020-21).\nii) Enlistment certificate.\niii) Other Documents as mentioned in Tender Documents Sheet (TDS)\n15\nPrice of Tender Document (Tk)\nTk. 50,000/- (Fifty Thousand) only\n16\nAmount of Tender Security\nTk. 1,50,00,000/- (One Crore Fifty Lacs) Only in favour of Chief\nPlanning, Dev & Works, BUP\n17\nName, Designation, Address\n&\nChief Planning, Dev & Works, BUP, Mirpur Cantonment, Dhaka-1216.\nContact details of Official Inviting\nTelephone No.88-02-8000442; .Fax No. 88-02-8035903. e-mail:\nTender\nplanningbup@gmail.com\n18\nFor any clarifications please contact:\nAddl. Director (Civil), Office of the Chief Planning, BUP Tel: 8000598,\nMobile: 01769-021654\n19\nSpecial Instructions: The procuring entity reserves the right to reject all tenders or annul the tender proceedings\nChief Planning, Dev & Works\nBUP\n(S\u00f3'x8)\nMirpur Cantt., Dhaka-1216\n", {"entities": [[181, 226, "inviter"], [227, 247, "inviter_location"], [550, 611, "description"]]}],
["Image No. 1278\nfile_name:/content/content/jpg_files/20210901-100100768-960.txt\n\nGovernment of the Peoples Republic of Bangladesh\nOffice of the Executive Engineer, (RHD)\nRoad Division, Bhola.\nE-mail: eebho@rhd.gov.bd\n200\nF\nMemo No. .35.01.0918.404.34.001:20-1755\nDate: 31/08/2021\nInvitation for e-Tender (OTM)\ne- -Tender is invited in the national e-GP system portal for the following works:\nTender ID\n601217\nName of Tender\nAggregate Base Type Il with Carpeting Seal coat work-\nat 13+064 to 13+964km of Borhanuddin (Udaypur)=\n(Z8905)\nZilla\nHighways under Road Division Bhola during the Fiscal\nyear 2021-22 (Package No. 05re-GP/Bho/Min/2021-22)\nTender Last Selling Date and\n14-Sep-2021 17:00\nTime\n15-Sep-2021 12:00\nA\nTender Closing & Opening Date\nand Time\nThis is an online tender, where only e-tender will be accepted in e-GP portal and no offline\nhard copy will be accepted. To submit e-tender, please register on e-GP system.\nhttp:/www.eprocur\u00ea.gov.bd, For more details please contact e-gp help desk,\nrhelpdesk@eprocure.gov.bdi\nNable\n31.08.21\n(Md. Nazmul Islam)\nA\nID No-602232\nExecutive Engineer (C.C.), RHD\nVDP.- 796/21 (6x3)\nRoad Division, Bhola.\n:\n", {"entities": [[164, 167, "inviter"], [129, 161, "co_inviter"], [184, 189, "inviter_location"], [309, 319, "call_type"]]}],
["Image No. 1028\nfile_name:/content/content/jpg_files/20210901-100100768-961.txt\n\nGovernment of the- people's Republic of Bangladesh\nLocal Government Engineering Department\nOffice of the Executive Engineer\n6+4\nBanglades\nDistrict: Jamalpur.\nTesas\nwww.iged.gov.bd\nMemo No. 46.02.3900.000.07.00121 -2474\nDate:- 31-08-2021\ne-Tender. Notice No-07/2021-2022\ne-Tender is invited the National e-GP System Portal (http://www.eprocure.gov.bd) for the Procurement of works\ndetails in table;\nTender Reference No\nName of work\nTender\nID\nCAFDRIRP/Jamalpur/\nRehabilitation of Ambaria RHD Pochabobhola GC via Ruknia Bazar Road from Ch.\n596797\nUZR/W-10/2021-22\n3900-6400m Under Melandah Upazila, District: Jamalpur (Road ID No: 339612015).\nCAFDRIRP/Jamalpur/\nR\u00e9habilitation of Melandah GC-Tonkey GC via Bhaluka Road from Ch. 1000-6340m\n596798\nUZR/W-11/2021-22\nUnder Melandah Upazila, District: Jamalpur (Road ID No: 339612009).\nCAFDRIRP/Jamaipur/\nRehabilitation of Nangla UZR- Horipur High School Ghat Road from Ch. 1800-3280m\n596799\nVR-A/W-12/2020-21\nUnder Melandah Upazila, District: Jamalpur (Road ID No: 339614024).\nCAFDRIRPJamalpur/\nRehabilitation of Mahmudpur UP- Adervita UP via Jatiarpar RHD (Melandah Portion)\n596800\n\"UNR/W-13/2021-22\nRoad from Ch. 00-1500m & 9700-12250m Under Melandah Upazila, District: Jamalpur.\nCAFDRIRP/Jamalpur/\nRehabilitation of Charpakerdah UP- Karaichara UP Road from Ch. 00-3000m Under\n596801\nUNR/W-15/2021-22\nMaderganj Upazila, District: Jamalpur (Road ID No:339583008).\nCAFDRIRP/Jamalpur/\nRehabilitation of Mahmudpur UP-Adervita UP Road via Bir Saguna Road from Ch.\n596802\nUNR/W-16/2021-22\n9700-12360m & 19700-23200m. under Madarganj Upazila, District: Jamalpur.\nCAFDRIRP/Jamalpur/\nRehabilit\u00e1tion of \"Koyra GC-Aramnagar GC via Satpoa UP Road (Madarganj Part) from\n596803\nUZR/W-17/2021-22\nCh. 00-3500m under Madarganj Upazila, District: Jamalpur (Road ID No: 339582013).\nCAPDRIRP/Jamalpur/\nRehabilitation of Noarpara UP-Janata Bazar Road from Ch.00-5585 under Islampur\n596804\nUNR/W-18/2021-22\n\"Upazila, District: Jamalpur (Road ID No: 339293002).\nCAFDRIRP/Jamalpur/\nRehabilitation of Jabbarganj GC- Surjanagar Bazar Road from Ch. 00-7450m under\n596805\nUZR/W-19/2021-22\nBakshiganj Upazila, District: Jamalpur (Road ID No: 339072015).\nCAFDRIRP/Jamalpur/\nRehabilitation of Bakshiganj R&H (Makibag.More)-Merurchar UP via Kathaltoli Road\n596806\nUNR/W-20/2021-22\nfrom Ch. 00-6650m under Bakshiganj Upazila, District: Jamalpur.\n-\nCAFDRIRP/Jamalpur/\nRehabilitation of Adervita UP (Gazaria More)- Koyra GC Road from Ch. 00-2850m\n59681\nUNR/W-14/2020-22\nUnder Maderganj Upazila, District: Jamalpur (Road ID No: 339583007).\nThis is an online Tender, where only e-Tender will be accepted in the National c-GP Portal and no offline/hard\ncopies. will be accepted.F\nTo submit e-Tender, registration in the National e-GP System portal (http://www.eprocure.gov,bd) is required.\nThe fees for downloading the \u00e9-Tender Documents form the National e-GP System Portal have to be deposited\nonline through any registered Banks' branches up to 17.00 PM on 04-Oct-2021.\nForther information and guidelines are available in the National e-GP System Portal and from e-GP help desk\n(helpdesk @epr\u00f3cure.gov.bd).\nfrom\nEx\u00e9cutive Engineer\nLocal Govt. Engineering Department\nJamalpur.\nTel. No:0981-63698\n\"Fax No: 0881-62569\nVDP-\nEmail: xed. jathatpur@lged.cov.bd\n", {"entities": [[131, 170, "inviter"], [171, 203, "co_inviter"], [228, 236, "inviter_location"], [317, 325, "call_type"], [540, 607, "description"]]}],
["Image No. 1076\nfile_name:/content/content/jpg_files/20210901-100100768-962.txt\n\nGovernment of the People's Republic of Bangladesh\nLocal Government Engineering Department\nOffice of the Executive Engineer\n664 2/19/19\nBanglad\nLGED, Madaripur\n21th 12655\nwww.lged.gov.bd\nMemo No: 46.02.5400.001.07.002.17 (Part-5)-2471\nDate : 31/08/2021\ne-Tender Notice\nTender No. e-Tender Notice 2021-22/05\n,\ne- Tender is invited in the National e-GP System Portal (http://www.eprocure.gov.bd) for the procurement of the following works :\nLocation\nT-ID\nName of the Work\nTender\nRemarks\n/Method\nClosing\n603175\nImprovement of UZR-12, Monu Kazi Shop to UNR-4 Road (Choto Chowdhurir Bill)\n(LTM).\nH/O. Chan Mia Khan road (Ch: 00m-750m), (ID: 354875027) (Salvage Cost: Tk\n17,00,382.00) (DW-01).\nImprovement of UZR-1, house of Nirmol Guho to VRB-146, house of Samad Mathbor\n1603178\nroad via house (Via Rishipara) of -Halan Molla road (Ch: 00m-500m), (ID:\n2nd Call.\n(LTM)\n354875147) (DW-02).\nImprovement of UZR-1, house of Nannu Baperi to Bhojal Shikderkandi road via\n603179\nhouse of Tofazzel Sardar (Ch: 800m-1500m), (ID: 354875121) (Salvage Cost: Tk\n(LTM)\n16,13,263.00) (DW-04).\nThis is an online Tender, where only e-Tender will be accepted in the National e-GP Portal and no offline/hard copies will be accepted.\nTo submit e-Tender, registration in the National e-GP system portal (www.eprocure.gov.bd) is required.\nThe fees for .downloading the e-Tender Documents from the National e-GP system Portal have to be deposited on line through any e-GP\nregistered banks Branches.\nFurther information and guidelines are ,available in the National e-GP System portal and from e-GP help desk\nhefpdesk@eprocure.dov.bd} and also from e-mail: xen.madaripur@joed.gov.bd or call to 0661-62445.\nInterested persons can communicate with the undersigned during office hours.\nJanuary\ncopt\n(Md. Babul Akhter)\nExecutive Engineer\n14\nLGED, Madaripur\nTel: 0661-62445(Off)\ne-mail: ken.madaripur@lged.gov.bd\nVDP-795/21 (7x4)\n", {"entities": [[130, 169, "inviter"], [170, 202, "co_inviter"], [229, 238, "inviter_location"], [332, 347, "call_type"], [587, 638, "description"]]}],
["Image No. 1168\nfile_name:/content/content/jpg_files/20210901-100100768-964.txt\n\ntab 7/642011 diany\nof 3 321510\nFax: 49270201\nBangladesh Agricultural Research Institute\nTel: 94270077\nGazipur-1701\nE-mail: se.building@bari.gov.bd\nWeb: www.bari.gov.bd\n.\nMemo no. 12.21.0000.010.41.468.20.113\nDate: 29.08.2021\ne-TENDER NOTICE\nTender is invited-Through National e-GP System Portal (http//:wwweprocure.gov.bd) for procurement of following\nworks:\nSI\nTender\nTendering\nDescription of works\nOnline (e-GP\nOnline (e-GP\nno\nID No.\nMethod\nsystem) tender\nsystem)\n14\npublication date\ntender closing\n& time\ndate & time\n1.\n601708\nOTM\nLand Development work for ,under\n29-Aug-2021\n14-Sep-2021\nResearch, Development & Extension of\n16:00\n14:25\ncashwnut and coffee (BARI Part) at\nARS, BARI, Raikhali, Rangamati\n2.\n601622\nOTM\nLand Deyelopment work for under\n29-Aug-2021\n14-Sep-2021\nResearch, Development & Extension of\n16:00\n14:00\ncashwnut and coffee (BARI Part) at\nHARS, BARI, Khagrahari\nThis is online tender, only e-Tender will be accepted in National e-GP Portal and no offline/hard copy will be\naccepted. Registration is essential to submit e-Tender.\nFees have to be deposited for downloading the tender documents.from e-GP Portal through any registered bank. For\nfurther information please contact e-GP help desk (helpdesk@eprocure.gov.bd)\nis\nEngr. Md. Ferozur Rahman\nVDP- 794/21 (5x4)\nSuperintending Engineer (In-charge)\n", {"entities": [[125, 167, "inviter"], [182, 190, "inviter_location"], [305, 320, "call_type"]]}],
["Image No. 1182\nfile_name:/content/content/jpg_files/20210901-102100886-974.txt\n\nGovernment of the People's Republic of Bangladesh\nLocal Government Engineering Department\nBanglade\nOffice of the Upazila Engineer\nUpazila:- Mirzagonj District:- Patuakhali.\n\"\nCAR 21/29/8 244113\n2/12/ 2255 Bala\"\nMemo No-46.02.7876.000.14.086.2020-589\nDate:- 31/08/2021\nInvitation for Tender (Works)\nOTM\ne-Tender Notice Number : 01/2021-2022\nTender is invited in the national e-GP system portal (http://www.eprocure.gov.bd)\nfor the procurement of following works in Open Tender Method.\nSI\nPackage No.\nTender\nName of Scheme\nTender last\nTender\nNo.\nID\nSelling (Date\nClosing (Date\nand Time)\nand Time)\n1\n2\n3\n4\n5\n6\n1.\ne-Tender/\n599430\nSub Project for Construction of 3\n15/09/2021\n16/09/2021\nUGDP/MIR/I\nclass room for Majidbaria *High\n16.00\n14.05\nNF-2019-20-\nSchool under Majidbaria Union,\n107876-01\nMirjaganj Upazilla, Patuakhali.\nThis is an online Tender, where only e-Tender will be accepted in the Notional e-GP\nportal\nand\nno\noffline/hard copies will be accepted. To Submit e-Tender, registration\nin the National e-GP system portal (http://www.eprocure.gov.bd) is requird.\nFurther information and Guideline are available in the National e-GP_system portal\nand from e-GP help Desk (helpdesk@eprocure.gov.bd)\n(Sheikh Azim Ur Rashid)\ntrial\nS\nUpazila Engineer\nLGED\nMirzagonj, Patuakhali.\nTel No: 04426-75223\nE-mail: ue.mirjaganj@lged.gov.bd\n", {"entities": [[130, 169, "inviter"], [179, 209, "co_inviter"], [241, 251, "inviter_location"], [382, 397, "call_type"], [723, 805, "description"]]}],
["Image No. 1005\nfile_name:/content/content/jpg_files/20210901-100100768-963.txt\n\nTHE\n9442 copy 5/22/19 CISURN\n0\n18/08-28/20, 6147-3256\nwww.bsmrmu.edu.bd\n14\nINVITATION FOR TENDERS\nDate: 31 August 2021\nNo. BSMRMU/Procurement-356/02/21/1983\nMinistry/Division\nMinistry of Education\nProcuring Entity Name\nBangabandhu Sheikh Mujibur Rahman Maritime University (BSMRMU)\nProcuring Entity District\nDhaka, Bangladesh\nTender Invitation for\nProcurement of Annual Report, Quarterly Magazine & Journal\nInvitation Reference No\n31 August 2021\nDate\nBSMRMU Open Tender No-02/2021-2022\nKEY INFORMATION\nProcurement Method\nOpen Tender Method (National)\nFUNDING INFORMATION\nBudget and Source of Funds\nGovernment of Bangladesh\nPARTICULAR INFORMATION\nTender Last selling Date and Time\n15 September 2021, 12;00 pm\nTender Submission Date and Time\n16 September 2021, 12:00 pm\nTender Opening Date and Time\n16 September 2021, 12:15 pm\nINFORMATION FOR TENDERER\nEligibility of\nThe Tenderer should have a valid trade license, updated VAT and TAX certificate, up to date\nTenderer\nbank solvency and other document as per tender document.\nBrief description of goods\nDetails description is available with tender document\nQty\nLocation\nTender Security\nCompletion time\nLot\nIdentification of Lot\nMoney\nNo.\nTk. 3,000.00\n1.\nAnnual Report\nAs per\nBSMRMU\nTk. 45,000.00\n30 days from the\n2.\nQuarterly Magazine\nsignature of\nTD\nTk. 10,000.00\n3.\nBangladesh Maritime Journal V-6\ncontract\n4.\nJournal of Earth and Ocean Science\nTk. 2,000.00\nTk 500/00 for each lot (non refundable) in the form of Pay Order/\nTender document price\nbank draft (MICR) from any Scheduled bank of Bangladesh in\nfavour of \"BSMRMU General Fund\".\nPROCURING ENTITY DETAILS\nName Designation of official inviting\nRegistrar, BSMR Maritime University, Bangladesh\ntender\nAddress of official inviting tender\nRegistrar Office, BSMRMU, Plot No. 14/06-14/23, Pallabi, Mirpur -12,-\nDhaka-1216\nContract details\nTel No :\nFax No:\nWeb: www.bsmrmu.edu.bd\nOfficial Inviting tender\n01769721010\n02-58070457\n-mall:procurement@bsmrmu.edu.bd\nThe procuring entity reserves the right to accept or reject any tender without assigning any reason.\nProcurement Section\nBSMRMU, Bangladesh\nVDP- 792/21 (7x4)\n", {"entities": [[299, 361, "inviter"], [388, 393, "inviter_location"], [428, 486, "description"]]}],
["Image No. 1380\nfile_name:/content/content/jpg_files/20210901-101419567-965.txt\n\nI\nBAKHRABAD GAS DISTRIBUTION COMPANY LIMITED\n(A Company of Petrobangla)\nHead office, Chapapur, Cumilla\nEngineering Service Department, Cumilla.\n(website : www.bgdcl.org.bd)\n/\nReference No.: 28.11.0000.710.38.005.21/534\nDate: 25/08/2021\n)\ne-Tender Notice (OTM)\ne-Tender is invited in the National e-GP Portal (http://www.eprocure.gov.bd)\nfor the procurement of :\nLast\nTender\nSelling\nClosing\nOpening\nID No.\nPackage No.\nDescription of Works\nDate\nDate and\nDate and\nand time\ntime\ntime\nPainting of DRS/\nCMS of Poultry\nFarm, Cumilla\nCentral Jailkhana,\nCantonment,\nChandina,\nChandina Power\n20-Sep-\n21-Sep-\n595723\n28.11.0000.7\n21-Sep-\n10.38.003.21\nPlant,\n2021\n2021,\n2021\nGauripur HP DRS,\n17:00\n12:00\n12:00\nNandanpur HP/IP\nDRS, EPZ\nOld, BGDCL Office\nComplex\nUnder BGDCL Gas\nDistribution Areas.\nThis is an online Tender, where only e-Tender will be accepted in the National\ne-Gp portal and no offline/hard copies will be accepted.\nTo submit e-Tender, registration in the National e-GP system Portal\n(http://www.eprocure.gov.bd) is required.\nThe fees for downloading the e-Tender Documents from the National e-GP\nSystem portal have to be deposited online through any registered Bank\nbranches.\nFurther information and guidelines are available in the National e-Gp System\nportal and from e-GP help desk (helpdesk@eprocure.gov.bd).\nFor more details please contact to the PE's Support desk (081-68660).\n34\n26.08.2021.\n(Engr. Mortuza Rahman Khan)\nDeputy General Manager\nEngineering Service Department, Cumilla\nPF-384-824/21 (8\" x3)\nBGDCL, Chapapur, Cumilla-3500\n", {"entities": [[82, 124, "inviter"], [165, 182, "inviter_location"], [318, 339, "call_type"], [560, 575, "description"]]}],
["Image No. 1334\nfile_name:/content/content/jpg_files/20210901-101419567-966.txt\n\nGOVERNMENT OF THE PEOPLE'S REPUBLIC OF BANGLADESH\nMinistry of Finance, Finance Division\nSkills for Employment Investment Program (SEIP)\nProbashi Kallyan Bhaban (Level-16)\n71-72 Old Elephant Road, Eskaton Garden, Dhaka-1000\nInvitation for T\u00e9nders\n+1\nMinistry/Division\nMinistry of Finance\n2\nAgency\nFinance Division\n3\nProcuring Entity Name\nSkills for Employment Investment Program\n4\nProcuring Entity Code\nN/A\n5\nProcuring Entity District\nDhaka\n6\nInvitation for\nWorks\n,\n7\nInvitation Ref No\nFD/SEIP/WD -103/572/2021/985\nx\n31 August 202\nORMATION\ntr\n-\n#\nProcurement Method\nOpen Tendering (National)\nFUNDING INFORMATION\n10\nBudget and Source of Funds\nGovernment of Bangladesh (GoB)\n11\nDevelopment Partners (if applicable)\nNot Applicable\nPARTICULAR INFORMATION\n12\nProject/Program Code (if applicable)\n5-0901-5014\n113\nProject/Program Name (if applicable)\nSkills for Employment Investment Program\n214\nTender Package No.\nWD-103\n15\nTender Package Name\nInterior works (Civil, Electrical & Plumbing) for SEIP and NHRDF\nOffice\n16,\nTender Publication Date\n1 September 2021\n17\nTender Last Selling Date\n27 September 2021\n18\nTender Closing Date and Time\n29 September 2021 at 2:30 p.m.\n19\nTender Opening Date and Time\n29 September 2021 at 3:00 p.m.\n20\nName & Address of the office(s)\n- Selling Tender Document\nOffice of the Executive Project Director\nSkills for Employment Investment Program (SEIP) Project Probashi\nKallyan Bhaban (Level 16)\n71-72 Old Elephant Road, Eskaton Garden, Dhaka-1000. Telephone\nNo.: 880255138753-5 (PABX)\nFax No.: 880255138752\nReceiving Tender Document\nAddress: Room # 1201, Building # 11 Finance Division, Ministry of\nFinance Bangladesh Secretariat, Dhaka.\nOpening Tender Document\nAddress: Room #1201. Building # 11 Finance Division, Ministry of\nFinance Bangladesh secretariat, Dhaka,\n21\nPlace/ Date /Time of Pre-Tender Meeting\nConference Room\nSkills for Employment Investment Program (SEIP) Project Probashi\nKallyan Bhaban (Level 16),\n71-72 Old Elephant Road, Eskaton Garden, Dhaka-1000\n14 September 2021 at 11.00 a.m.\n!\nINFORMATION FOR TENDERER\n22\nBrief Eligibility and Qualification of\nThe Invitation for Tender is open to all eligible tenderers who\nTenderer\nsatisfy:\n(a) All eligibility criteria stated in the tender document.\n(b) The minimum number of years of general experience of the\nTenderer in the construction works as Prime Contractor or\nSubcontractor or Management Contractor shall be 5 years.\n(c)\nThe minimum specific experience as a Prime Contractor or\nSubcontractor or Management Contractor in at least 01 (one)\n24\n\"no. of contract or similar works with a minimum value of\nBDT 5.0 Crore within last 05 years. In case of JV, the lead\nPartner shall have this specific experience. The experience of\ndoing, at 15th floor or above, the minimum of 9000 sft floor tile\nworks on a single floor area along with plumbing, electrical\nworks and interior walls in public sector shall be treated as\nspecific expefience;\n(d) The required average annual construction turnover shall be\ngreater than BDT 10 Crore in any best of three years during\nlast five years.\n(e)\nThe minimum amount of liquid assets i.e working capital or\ncredit line(s) of the Tenderers shall be BDT 6 crore.\n(f)\nThe minimum capacity shall be: BDT 6 crore.\n(g)\nA valid trade license, an Income Tax registration certificate, up\nto date Income Tax clearance certificate and a VAT\nregistration certificate.\n23\nBrief Description of Works\nInterior works (Civil, Electrical & plumbing) for NHRDFOffice\n24\nPrice of Tender Document (Tk)\nA non-refundable fee of BDT 3,000 (Taka three thousand only) by\npay order/bank draft in favor of CIP-Imprest Fund\".\n25\nIdentification of Lot\nLocation\nTender Security\nCompletion\nAmount\ntime in\nmonths\nInterior works (Civil, Electrical &\nProbashi Kallyan\nBDT 20 (Twenty)\n4 Months\nPlumbing) for NHRDF Office\nBhaban (Level 18),\n-Lakh in favour of\n71-72 Old Elephant\n\"Executive Project\nRoad, Eskaton\nDirector, Skills for\nGarden,\nEmployment\nDhaka-1000.\nInvestment Program\n(SEIP) Project.\"\nPROCURING ENTITY DETAILS\n26\nName of Official Inviting Tender\nMd. Zahidul Haque\n27\nDesignation of Official Inviting Tender\nExecutive Project Director\n28\nAddress of Official Inviting Tender\nProbashi Kallyan Bhaban (Level 16), 71-72 Old Elephant Road,\nEskaton Garden, Dhaka-1000.\n29\nContact details of Official Inviting\nTelephone No.: 880255138753-5 (PABX)\nTender\nFax No.: 880255138752\ne-mail address: info@selp-fd.gov.bd\n30\nThe procuring entity reserves the right to reject any or all tenders or annual the Tender proceedings\n(Md. Zahidul Haque)\nExecutive Project Director\nSkills for Employment Investment Program\n(SEIP) Project\nDG-1701/21 '(12\"x3)\nPhone: 55138753-5 (PABX), Fax: 55138752\nEmail: zahid63@yahoo.com\n", {"entities": [[130, 167, "inviter"], [276, 297, "inviter_location"], [1017, 1062, "description"]]}],
["Image No. 1280\nfile_name:/content/content/jpg_files/20210901-101419567-967.txt\n\nPRAGOTI INDUSTRIES LIMITED\n6D\n(An Enterprise of Bangladesh Steel & Engineering Corporation)\nFinlay House (2nd Floor), 11, Agrabad Commercial Area\nPost Box No-73, Chittagong, Bangladesh.\nPhone-880-31-716207, FAX: 880-31-710725\nE-Mail: pil bsec@yahoo.com,pilmarketingl@gmail.com\nInvitation for Tender\n1\nMinistry/Division\nMinistry of Industries\n2\nAgency\nBangladesh Steel & Engineering Corporation (BSEC).\n3\nProcuring Entity Name\nPragoti Industries Limited.\n4\nProcuring Entity Code\n-\n5\nProcuring Entity District\nChattogram.\n6\nInvitation for\nAppointment of Supplier for supply with fittings of\nVehicles Accessories & Hood-Structure in different\nmodels of vehicles.\n7\nInvitation Ref No\n36.093.0000.515.77.30.2021-60-\n8\nDate\n31.08.2021\nKEY INFORMATION\n9\nProcurement Method\nOTM, National.\nFUNDING INFORMATION\n10\nBudget and Source of\nOwn Finance.\nFunds\n33\nPARTICULAR INFORMATION\n11\nTender Package No.\n-\n12\nTender Publication Date\n01.09.2021\n13\nTender Last Selling Date\n21.09.2021\n14\nTender Closing Date and\n22.09.2021 at 11.30 AM\nTime\n15\nTender Opening Date\n22.09.2021 at 11.45 AM (Technical offer)\nand Time\n16\nName & Address of the\nAddress: Pragoti Industries Ltd., 11, Agrabad C/A,\noffice(s)\nChattogram.\n17\nSelling Tender\nCash Section, Pragoti Industries Ltd., 151-152, Tejgaon\nDocument (Principal)\nIndustrial Area, Dhaka.\n18\nSelling Tender\nCash Section, Pragoti Industries Ltd, 11, Agrabad, C/A,\nDocument (Others)\nChattogram.\n19\nOpen Tender Document\nMarketing Department, Pragoti Industries Ltd, 11,\nAgrabad, C/A, Chattogram.\nINFORMATION FOR TENDERER\n20\nBrief Eligibility and\nAs per Tender schedule.\nQualification of Tenderer\n21\nBrief Description of\nSupply with fittings Vehicles Accessories & Hood-\nGoods\nStructure in different models of vehicles. (Details are in\nTender schedule)\n22\nPrice of Tender\n3,000.00 (Three Thousand).\nDocument (Tk)\nPROCURING ENTITY DETAILS\n23\nName of Official Inviting\nEng. Md. Akter Hossain\nTender\n24\nDesignation of Official\nManaging Director (Additional Charge)\nInviting Tender-\n25\nAddress of Official\nPragoti Industries Ltd., 11, Agrabad C/A,\nInviting Tender\nChattogram.\n26\nContact details of\n031-724801\n031-716207\n01711-861219\nOfficial Inviting Tender\n27\nThe Procuring Entity reserves the right to accept or reject any/all tenders prior to\nacceptance without assigning any reason what so ever.\n(Humayun Kabir)\nPC-1079/21 (9\"X3)\nDGM, Department of Marketing\nFor Managing Director\n", {"entities": [[80, 106, "inviter"], [242, 252, "inviter_location"], [617, 739, "description"]]}],
["Image No. 1114\nfile_name:/content/content/jpg_files/20210901-101419567-968.txt\n\nGovernment of the People's Republic of Bangladesh\nLocal Government Engineering Department\nus\nOffice of the Upazila-Engineer\n6th\nRajibpur, Kurigram\nsita GATE\nwww.lged.gov.bd\n34\nMemo No: 46.02.4908.000.05.001.21/38\nDate: 31/08/2021\ne-Tender Notice: 01/2021-22 LLTM)\ne-Tender is invited in the National e-GP System Portal (http://www.eproucre.gov.bd) for the procurement of\nfollowing works:\nTender ID\nPackage No.\nDescription of,Work-\nTender\nMethod\n599974\ne-Tender/NBIDNNGPS-1\nConstruction\" of. Boundary Wall & Gate of 1.\nLTM\n/KUR/RAJ/2020-21/W2.00025\nNayachar M.B 2. Nayachar Moddhopara 3.\nNayachar Modon para 4. Nayachar Natunpara & 5.\nPatadhowa Para Primary school.\n599997\nConstruction of Boundary Wall & Gate of 1.\nLTM\ne-Tender/NBIDNNGPS-1\n/KUR/RAJ/202\u00d2-21/W2.00026\nTinthopa Para 2. Diarar Char Munsi Para 3.\nKalkhihara 4. Nayach\u00e4r Kolpara & 5? Nayachar\nMunsi Para Primary,school.\n600000\ne-Tender/NBIDNNGPS-1\nConstruction of Boundary Wall *& Gate of 1.\nLTM\n/KUR/RAJ/2020-21/W2.00027\nNayachar Dakkhin Para 7. Nayachar Poschim\nMollapara & 3. Uttar Char Sajai Mondol para\nPrimary school.\nThis is an online \"tender, where only e-Tender shall be> accepted in the National e-GP portal and no\noffline/hard copi\u00e9s shall be, accepted. To submit-e-Tender, :registration in the National e-GP System Portal\n(http://www.eprocure.gov.bd) -and Enlistment in LGED under district Rangpur, Gaibandha, Kurigram &\nLalmonirhat is required. The fees for Downloading e-Tender Document from the National\"e-G System Portal have\nto be deposited online through any registered Banks branches up to 22 September 2021 at 17.00. Further\ninformation and guidelines are available in the National e-GP, System Portal and e-GP help desk\n(http://www.eproucre.sov.bd)\n(MD. Masudar Rahman)\nUpazila Engineer\nRajibpur, Kurigram\nPF-1196-825/21 (6x4)\n\u00e9-mail: ue.rajibpur@lged.gov.bd\n", {"entities": [[130, 169, "inviter"], [173, 203, "co_inviter"], [208, 226, "inviter_location"], [310, 325, "call_type"]]}],
["Image No. 1360\nfile_name:/content/content/jpg_files/20210901-102100886-973.txt\n\nGovernment of the People's Republic of Bangladesh\nBangladi\nOffice of the Executive Engineer\nDepartment of Public Health Engineering\nRajshahi Division, Rajshahi.\nCAR\n5112 *2658\nMemo No: 46.03.8100.061.14.040.20\nDate: 3T August, 2021,\nInvitation for e-Tender\n1\nMinistry/Division\nLocal Government Division\n2\nImplementing Agency\nDepartment of Public Health Engineering\n3\nProject/Program name\nREV\n4\ne-Tender IDs\n598059\n5\nDescription of Works\nConstruction of One Storied Upazilla office Building\nincluding additional one story stair case arrangement with\nFour storied foundation at Puthia Upazilla of Rajshahi\nDistrict under revenue budget during FY 2021-22.\n6\nName, Designation &\nMd. Rokonujjaman,\nAddress of Official\nExecutive Engineer,\nInviting Tender\nDPHE, Rajshahi Division, Rajshahi.\nTel. No: 0721-774987; E-mail: dpherajshahi@gmail.com\nThis is online Tender, where only e-Tenders will be accepted in the-National e-GP Portal and\nno offline/hard copies will be accepted. Interested person/firms can see details in the website:\nwww.eprocure.gov.bc\nBZ\n31'0821\n(Md. Rokonujjaman),\n52\nExecutive Engineer\nDepartment of Public Health Engineering\n(8' xo)\nRajshahi Division, Rajshahi\n", {"entities": [[172, 211, "inviter"], [139, 171, "co_inviter"], [231, 239, "inviter_location"], [328, 336, "call_type"], [517, 652, "description"]]}],
["Image No. 1267\nfile_name:/content/content/jpg_files/20210901-102100886-975.txt\n\n-\nGovernment of the People's Republic of Bangladesh\nLocal Government Engineering Department\nBanglades!\nOffice of the Upazila Engineer\nJhikargacha, Jashore.\n\" 512 211-9915\nwww.lged.gov.bd\n21121 2055\n62/5\nMemo.No. 46.02.4123.000.21-721\nDated: 29-08-2021\ne-Tender Notice: 01/2021-22\ne-Tender is invited in the National e-GP portal (http:www.eprocure.gov.bd) for the following\nwork\n(procurement of works) under Upazila Engineers Office, Jhikargacha during this year 2021-22.\nSI. No.\nPackage No.\nName of Scheme\ne-TID No.\n1\n2\n3\n4\n01.\nINF-2018-19-404123-01\nSupply of Medical Equipment for Upazila\n601311\nHealth Complex, under UGDP, Jhikargacha'\nUpazila, District- Jashore.\n02.\nINF-2018-19-404123-02\nSupply of Furniture High & Low Bench in\n601312\ndifferent Educational Institution, under UGDP,\n?\nJhikargacha Upazila, District- Jashore.\nThis is an online Tender, where only e-Tender will be accepted in the National e-GP portal\nand no offline/hard copies will be accepted.\nTo submit e-Tender, registration in the National e-GP portal (http:www.eprocure.gov.bd is required.\nThe fees for downloading the e-Tender Documents from the National e-GP portal have to be\ndeposited online through any registered Banks branches up to 19 September/2021, 12.00 and\nTender Closing & Opening date & time 19 September/2021, 16.00.\nFurther information and guidelines are available in the National e-GP system portal and e-GP\nhelp gesk (helpdesk@eprocure.gov.bd) and also from E-mail: ue.jhikargacha@lged.gov.bd\nor Call to 01708161483/01711481885 for any queries interested Tenderer-are requested\ncommunicate with the undersigned during office hours.\nthe\nis\nthe\n13\n(Shamal Kumar Bose)\nUpazila Engineer\nJhikargacha, Jashore.\nPhone No. 04225-713.18.\ne-mail: ue.jh\u00efkargacha@lged.gov.bo\n", {"entities": [[132, 171, "inviter"], [183, 213, "co_inviter"], [214, 234, "inviter_location"], [332, 347, "call_type"], [630, 657, "description"], [772, 791, "description"]]}],
["Image No. 1219\nfile_name:/content/content/jpg_files/20210901-102100886-977.txt\n\nGOVERNMENT OF THE PEOPLE'S REPUBLIC OF BANGLADESH\nMINISTRY OF HEALTH AND FAMILY WELFARE\nDIRECTORATE GENERAL OF FAMILY PLANNING\nLOGISTICS & SUPPLY UNIT (5TH FLOOR)\n6, KARWANBAZAR, DHAKA 1215, BANGLADESH\n19\nwww.dgfp.gov.bd\ne-Tender Notice (OTM), NCT, Goods)\nFile Name: DGFP/L&S-2/Almirah/2021-22/50/12\nDate: 31/08/2021\ne-Tender is invited in the National e-GP portal for the procurement of following goods:\nTender ID No.\nTender Package\nLast Downloading/\nClosing Date\nOpening\n& Package No.\nName\nSelling Date &Time\n& Time\nDate &Time\n1\n2\n3\n4\n5\nTender ID No: 598882\nProcurement of\n20/09/2021,\n21/09/2021\n21/09/2021\nPackage No.\nAlmirah\n16:00\n14:00\n14:00\nPSSM/GD2/e2\nThis is an online Tender, where only e-Tender (Online Tender) will be accepted through the\nNational e-GP portal and no offline/hard copy will be accepted. The fees for downloading the\ne-Tender documents from the National e-GP portal have to be deposited online through any\nschedule bank of Bangladesh registered by the Central Procurement Technical Unit (CPTU).\nFurther information visit National e-GP portal and from e-GP help desk (Help Desk e-mail:*\nhelpdesk@eprocure.gov.bd).\n131/08/2021\n(Ratna Talukder)\nDirector (Logistics & Supply ) &\nLine Director (PSSM-FP)\nLogistics & Supply Unit\nGD-133813 (4\"x3)\nTelephone: 880-2-55012349\n", {"entities": [[130, 167, "inviter"], [168, 206, "co_inviter"], [245, 264, "inviter_location"], [301, 316, "call_type"]]}],
["Image No. 1286\nfile_name:/content/content/jpg_files/20210901-102100886-979.txt\n\nNorth-West - Power Generation Company Ltd.\nISO 9001:2015, ISO 14001:2015 & ISO 45001:2018 Certified\nNORTH-WEST\n(An Enterprise of Bangladesh Power Development Board)\nPOWER\nCONPANY UMITED\nOffice of the Chief Engineer\nKhulna 225 MW Combined Cycle Power Plant, Goalpara, Khalishpur,Khulna.\nTel: 041-763773, Fax: N/A, E-mail: ce.khulnapp@nwpgcl.gov.bd\nRef: 27.28.0156.207.07.508.20.527\nDate: 29.08.2021\ne-Tender Notice\ne-Tender is invited in the national e-GP System Portal (http://www.eprocure.gov.bd) for the following\nprocurements:\nSI.\nPublication\nNo.\nTender ID\nInvitation Reference No.\nTender Description\nDate &\nClosing Date\n& Time\nTime\n1\n601408\n27.28.0156.207.07.411.21.528\nPurchase of Engine\n01/09/2021\n30/09/2021\nCoolant for Madhumati\n10:00 AM\n03:00 PM\n100 MW HBPP,\nNWPGCL, Bagerhat.\nFor detailed information, please visit the following link.\nFor SI. No. 01 https://wwwy.eprocure,gov.bd/resources/common/ViewTender.isp?id=601408\nThis one is online tenders, where only e-Tender will be accepted in the National e-GP portal and no\noffline/hard copies will be accepted. T\u00f3 submit e-Tender, registration in the national e-GP System Portal\n(http://www.eprocure.gov.bd) is required.\nFurther information and guidelines are available in the National e-GP System Portal and from e-GP help\ndesk (helpdesk@eprocure.gov.bd)\nit\n29/08/21\n(Afsanul Tanveer)\nExecutive Engineer (Plant Manager)\nMadhumati 100 MW HFO Based Power Plant\nNWPGCL, Mollahat, Bagerhat.\n20\nMobile: +8801777736407\n5-802 (exu)\n", {"entities": [[80, 122, "inviter"], [347, 364, "inviter_location"], [478, 493, "call_type"]]}],
["Image No. 1314\nfile_name:/content/content/jpg_files/20210901-102100886-980.txt\n\n<<<<04\ntaket 29918\nwww.zpbarisal.org\netter GAIT\n520,\nG/R/2: 23/08/2025.1\ne-Tender Notice (LTM): 01/2021-2022\ne-Tender is invited in the . National e-GP System Portal (htp://www.epro\u00e7ure,gov,bd) for >the\nprocurement of the following scheme listed below:\nTender ID NO: 599786\nThis is an online tender, where only e-Tender will be accepted in the -National o-GP portal and no\noffline/hard copies will be acc\u00e8pted To submit -Tender, registration. in the National e-GP System\nportal (http://www.eprocure.gov.bd) is required.\nthe\nThe Fees for downloading the e-Tender Documents from the national e-GP System portal have to be.\ndeposited online through` any registered Bank's branches up to 17.00 on previous day of receiving\nTender.\n:\nFurther information and guidelines are available in the National \u00e8-GP System portal and from\ne-GP\nhelpdesk ( helpdesk@eprocure.gov.bd).\n.\nipi\nand\nPR 13 your AIRC'S www.xpbarisal.org CTNT'S I\niv\nF\n(Sheikh Tawfique Aziz):\n20\nSenior Assistant Engineer,\nT-SSUC (8x0)\nZilla parishad, Barisal.\n", {"entities": [[153, 168, "call_type"], [1072, 1086, "inviter"], [1088, 1095, "inviter_location"]]}],
["Image No. 1151\nfile_name:/content/content/jpg_files/20210901-102100886-981.txt\n\nOPTO\nar4 e\nBTCL\n-\nvecres a -\nBangladesh Telecomminications Company Limited (BTCL)\nCGM Transmission Regi\u00f3n West, Mohakhali, Dhaka\n5\nInvitation for Local TENDER\n1:\nMinistry/Division\nPosts and Telecommunications, Division.\n2\nAgency\nBangladesh Telecommunications Company Limited.\n3\nProcuring Entity Name\nChief General Manager, Transmission W\u00e9st, BTCL, Mohakhali, Dhaka-1212\nS.\n4\nProcuring Entity District\nDhaka\n5\nInvitation for\nTender\nWorks\n-One Lots\n6\nInvitation Ref. No.\n14.33.0000.721.10.015.19/15\n7\nDate\n29-08-2021\n8\nProcurement Metbod\nNational Competitive-Tenderi Method (NCT).\n9\nBudget and Source of Fund\nContributory Fund\n10\nDevelopment Partner\nNot applicable,\n11\nTender Package No.\n1 (One) Lot\n12\nTender Package Name\nSupply, Instaliation, Testing and Commissioning of MSPP/OTN, Bafterigs and\nRectifiers at BNS Sher-e-Bangla and Laying of Optical Fiber Cables cand\nAccessories from Kalapara BTCL to BNS Sher-e-Bangla, Lalua, Kalapaira.\n13\nTender Publication Date\n01-09-2021\n14\nTender Last Selling Date\n12-09-2021\n15\nTender Closing Date and Time\n13-09-2021 at 12-00 HRS (BST)\n16\nTender Operling Date and Time\n13-09-202) at 12-10 HRS (BST)\n17\nEligibility of Tenderer\nAll Eligible Bidders\nto\n18\nName and Address of the Office\nAddress(s)\nII\nSelling Tender Do\u00e7uments\nOffice of the CGM, Transmission West Region, Mohakhali, Dhaka-1212,\nReceiving Tender Document\nRoom-lli, Ist Floor, Office of the COM, Transmission West R legi an,\nMohakhali, Dhaka-1212.\nOp\u00e9ning Tender Document\nRoom-111, '1st Floor, Office of the CGM, Transmission. West Rieg Jon,\nMohakhali, Dhaka-1212.\n19\nBrief Description of Goods and\nSupply, Installation, Testing and Commissioning of MSPP/OTN, Batterie is and\nWorks\nRectifiers at BNS Sher-e-Barigla and Laying; of Optical Fiber Cably :S and\nAccessories from Kalapara BTCL to BNS Sher-e-Bangla, Lalua, Kalape ra.\n20\nBrief Description of Related\nInstallation, Testing and Commissioning of ov\u00e9rhead OFC, 'MSP P/OTN,\nServices\nBatteries, Rectifiers & Accessories with related services.\n21\n>Tender Document Price\nBDT: 1,000 (Onc Thousand) only (non-refunclable)\n-22\nAmount of Tender Security\nBDT: 1,50, 000 (One Lakh and Fifty Thousand.) only\n23\nTime for Completion of Work\n-45 Forty Five) Days\n24\nPlacc, Date & Time of Pre-Bid\nNo Pre-Bid Meeting will be held\nMeeting\n25\nName of the Official Inviting\nA.S.M. Rezaul Karim\nTender\n26\nDesignation of the Official\nChief General Manager Transmission, Transmission West Region,\nInviting Tender\n27\nAddress of the. Official Inviting\nChief General Manager Transmission West: BTCL, Mohakhali,1\nJhaka-1212\nTender\n28\nContact Details of the Official\nTel: +880-2-41080827\nEmail: cgro.txwest@btcl.gov.bd<\nInviting Tender\nWebsite: http://www.btcl.gov.br\n1\n29\nThe Procuring entity reserves the right to accept or reject all tenders.\n30\nNB: This tehder is also available at websites: http://www.btcl.gov.bd\nFinancial\n20\n(A.S.M. Reza ul.Karim)\nChief General I Manager\nTransmission y Vest R\u00e9gion\nBTCL, Mohakh\nali, Dhaka-1212,\n3-08 (bxo)\nTel: +880-2,\n-41080827\n", {"entities": [[109, 161, "inviter"], [192, 208, "inviter_location"], [801, 887, "description"]]}],
["Image No. 1019\nfile_name:/content/content/jpg_files/20210901-102100886-982.txt\n\nGOVERNMENT OF THE PEOPLE'S REPUBLIG OF BANGLADESH\nPRISONS DIRECTORATE\n30/3, Umesh Datta Road, Bakshi Bazar,Dhaka-1211\nwww.prison.gov.bd\nMel\u00f1o \"No.38.04.0000.030.25.018.21-97'\nDate: 31.08.2021.\ne-Tender Notice\nThis is to notify all concern that the following,tender is invited in the national e-GP portal:\nin\nSL'No\nTender ID No.\nName of work\nPublishing, Closing date & Tim\u00e9\n1\n599646\nMonthly Supply of Mashur Dal (Saru) to \u00e1ll Jail\n23-Aug-2021 20:30,\nfor the period October/2021 to September/2022\n22-Sep-2021 11:00.\n2\n600452\nBlanket\n25-Aug-202r 21:00;\n19-Sep-2021 11:00\nThis is an online Tender where only -Tender will be accepted in the national e-GP portal and no offline/hard copies will be\naccepted. To submit e-tender. registration in the-National e-GP portal http://www.cprocure.gov.bd is required. Further\ninformation and guidelines are available in the national e-GP system portal and from e-GP Thelp desk\n&\n5.8.2021\n-\n20\n(Suraiya Akter)\nBJ-01811100516\nAsst. Inspector General of Prisons (Finance).\nFor Inspector Genera). of Prisons\nTelephone No.57300514.\n", {"entities": [[130, 149, "inviter"], [174, 192, "inviter_location"], [273, 288, "call_type"], [470, 497, "description"], [603, 610, "description"]]}],
["Image No. 1256\nfile_name:/content/content/jpg_files/20210901-102100886-983.txt\n\nNorth-West Power Generation Company Ltd.\nISO 9001:2015, ISO 14001:2015 & OHSAS 45001 2018 Certified\nNORTH-WEST\n(An Enterprise of Bangladesh P.ower Development Board),\nCOMPANT UMITITO\nOffice of the Plant Manager (Superint\u00e9nding Engineet)\nSirajganj 225 MW Combined Cycle Power Plant (Unit-1)\nSoydabad, Sirajganj.\nRef: 27.28.8878.101.07.001.19.22\nDate: 31/08/2021\n\"\ne-Tender Notice\nThe following e-Tenders are invited in the National e-GP System Portal (www.eprocure.gov.bd)\n-SI.\nLast\nNo.\nTender ID\nBrief Description of Tender\nPublication\nSelling\nOpening\nDate\nDate &\nDate\nTime\nSupply, of HCL (Hydrochloric) Acid\n599629\nStorage Tank & Acid Metering Tank\n-19/09/2021,\n26/08/2021\n20/09/2021\n1\n(Including Installation, Testing &\n3:45 PM\nCommissioning):\nThis is an online tender where only e-Tender will be accepted in the National e-GP Portal and no offline/hard\ncopies will be accepted. To submit e-Tender, registration in the National e-GP System Portal\n(www.eprocure.gov.bd) is required. Further information and guidelines, are available in the National e-GP\nSystem Portal and e-GP Help Desk (helpdesk@eprocure.gov.bd).\ngrain\n31/08/2021\n20\nEngr. Brojendra Kumar Sarker\n-Plant Manager (Superintending Engineer)-\n>\nSirajganj 225 MW Combined Cycle Power Plant (Unit-1)\n(BXO)\nNWPGCL, Soydabad, Sirajganj.-\n", {"entities": [[80, 120, "inviter"], [370, 389, "inviter_location"], [443, 458, "call_type"], [654, 688, "description"]]}],
["Image No. 1102\nfile_name:/content/content/jpg_files/20210901-102100886-984.txt\n\nGOVERNMENT OF THE PEOPLE'S REPUBLIC OF BANGLADESH\nOFFICE OF THE EXECUTIVE ENGINEER\nPWD E/M DIVISION-4\nBangladesh Secretariat, Dhaka.\nTel : 9545888 (Office)\nA\ne-Tender Notice\nMEMO NO : 420\nDate: 31.08.2021\ne-Tender is invited in the National e-GP system Portal (http//www.eprocure.gov.bd\nfor the Procurement of following Goods and Related Service:\n\u20ac\nLast 3\nSL.\nTender\nBrief Description of Goods and\nselling\nClosing\nNo.\nID\nRelated Service\nDate &\nDate &\nTime\nTime\nRemaining work of Up-\nGradation of Sir Salimullah\nMedical College, Dhaka (Sub-\nHead : C Electro Mechnical\n14-Sep-\n15-Sep:\n01.\n602052\nComponent)\nSupply\nand\n2021\n2021\nInstallation of 750-800 Kg. 8\n17:00\n15:00\nstop 10 Passenger Lift\nincluding Connecting Cable,\nAVR, ARD and A.CT2 nos Lift).\nThis is online Tender, where only e-Tender will be accepted in the National e-GP\nPortal and No Offline/ Hard coptes will be accepted. To Submit e-Tender,\nRegistration in the National e-GP' system Portal (http://www.eprocure.gov.bd) is\nrequired. Further information and guidelines are available in the National e-GP\nsystem Portal and from e-GP help desk (http://www.eprocure.gov.bd).\nSD/-\n(Md. Mohibul Islam)\nExecutive Engineer\nDG-622/21 (3 x2)\nPWD E/M Division-4, Dhaka.\n", {"entities": [[130, 162, "co_inviter"], [541, 646, "description"], [756, 770, "description"], [163, 166, "inviter"], [285, 293, "call_type"]]}],
["Image No. 1287\nfile_name:/content/content/jpg_files/20210901-102603781-985.txt\n\n-\nBanglad\nem\n612 3092116\nRajshahi City Corporation\nSIN 42C555\nEngineering Department\nNagar Bhaban, Rajshahi.\nMemo No.46.12.0000.006.14.273.2020.1875\nDate: 31.08.2021\ne-Tender Notice\n(IFT-02/2021-22)\ne-Tender\nis invited in the National e-GP System Portal (http://www.eprocure.gov.bd) for the\nprocurement of the following goods & works:\nTender\nTender Dccument last\nS L\nID\nTender Ref.\nName of Works\nselling Time and Date\n1\n46.12.0000.006.1\nSupply of Desktop Computer, Plotter Machine, Photocopy\n1.\n600989\n4.273.2020.1876\nMachine, Laser Printer, Ink Tank Printer & UPS.\nRecovery & Re-installation of 11KV/440V\nElectrical Distribution Line, Poles & Related\n2.\n598937\n46.12.0000.006.1\nWorks at Uposhahor Sheikh Rasel Child\n12:30 PM on'\n4.131.2021.1877\nRehabilitation Center to Sopura Grayeyard,\n15-Sept-2021\nCourt Dhalan to Rentu Khorir Arot, Nimtala\nMour to Court College, Asphalt-Piat-Area and\nVatapara to Court Station Mour Road.\nThis\nis\nan\nonline\nTender,\nwhere\nonly\ne-Tender\nwill\nbe\naccepted\nin\nthe\nNational\ne-GP\nPortal\nand\nno\noffline/hard\ncopies will be accepted_ To submit e-Tender, registration in the National e-GP System Portal\n(http://www.eprocure.gov.bd) is required. The fees for downloading the e-Tender Documents from the\nNational\ne-GP\nSystem Portal have to be deposited online through any registered B\u00e1nks branches.\nFurther information and guidelines are abailable in the National e-GP System Portal and from e-GP\nhelp desk (helpdesk@eprocure.gov.bd).\nsd/-\n(Md. Noor Islam)\nPfoject Director & Superintending Engineer\n,\nRajshahi City Corporation\nTel. 0247812020\nFIC-3890/25 (e' XO)\nE-mail : nislamtushar@gmail.com\n", {"entities": [[179, 187, "inviter_location"], [246, 261, "call_type"], [105, 130, "inviter"], [517, 571, "description"], [598, 868, "description"]]}],
["Image No. 1053\nfile_name:/content/content/jpg_files/20210901-102736398-989.txt\n\nPORT\nCHITTAGONG PORT AUTHORITY\nMechanical Department\nOffice of the Executive Engineer (Mech.) Zone-D.\ne-Tender Notice (OTM)\nDate: 25-08-2021.\nTender No: 03 /2021-22\nSL.\nTender ID No\nContract\nTender Closing\nNo.\n& Ref No.\nPackage No\nName of Works\nDate & Time\n01.\n591436\n18.04.0000.4\nProcurement of Pump (Circulation Pump) with the\n09-Sep-2021\n.\n,\n23.07.122.21\nother 06 items spare parts for 4-High Straddle Carrier\n11:15\n(Model: Kalmar-ESC-440) of Zonal Workshop-D.\n02.\n595099\n18.04.0000.4\nProcurement of Side' Shifting Double Roller Bearing\n22-Sep-2021\n23.07.094.21\nwith the other 02 items spare parts for 4-High Straddle\n11:15\nCarrier (Model: Noell/Terex-644H/E) of Zonal\nWorkshop-D.\n03.\n595100\n18.04.0000.4\nProcurement of Prop Shaft with the other 03 items\n28-Sep-2021\n23.07.119.21\nspare parts for 4-High Straddle Carrier (Model:\n11:15\nKalmar-ESC-440) of Zonal Workshop-D.,\nI\n04.\n595886\n18.04.0000.3\nProcurement of Inverter with the other 01 items spare\n15-Sep-2021\n84.07.132.21\nparts for 2-High Straddle Carrier (Model: ESH240HW)\n11:15\nof,Zonal Workshop-D\nThis is an online tender, where only e-Tenders will be accepted in e-GP portal and no offline / hard copies will\nbe accepted. To submit e-Tender, please register 'on e-GP system portal (http://www.eprocure.gov.bd)\nExecutive Engineer (M)\nCPA/PR-351/2021\nZonal Workshop-D.\nChittagong Port Authority.\n", {"entities": [[85, 110, "inviter"], [182, 197, "call_type"], [361, 492, "description"], [568, 619, "description"], [788, 837, "description"], [863, 902, "description"], [981, 1034, "description"]]}],
["Image No. 1177\nfile_name:/content/content/jpg_files/20210901-102736398-991.txt\n\nGovernment of the Peoples Republic of Bangladesh;\nOffice of the-Executive Engineer (RHD)\nFisman\nRoad-Division, , Bhola.\nE-mail: eebho@rhd.gov.bd j\nMemo No.35.01.0918.404,34.001.20.1752\nDate:- 31/08/2021\nInvitation for e-Tender (OTM)\ne-Tender is invited in the national e-GP system portal for the following wbrks:\nTender ID\n601214\n54\nName of Tender\nAggregate-Base Type II with Carpeting Seal\nco\u00e1t, work at Ch. 11+300 to 12+400km of\nBanglabazar\n-Khayerhat,Azadnagar-\nNurumiarhat-Daulatkhan (Z8917) Zilla\nHighways. under Road Division Bhola\nduring the Fiscal year 2021 -22 (Package No\n- 02/e - GP/Bho/Min/2021-22)\nTender Last Selling\n14-Sep-2021 1-7:00\nDate and Time\nTender Closing &\n15-Sep-2021 12:00\nM\u00b0\nOpening Date and Time\nThis is online \"tender, where only e-tender will be accepted in e-GP\nportal and no offline/hard copy, will, be accepted. To submit e-tender,\nAJ.\nplease register on \u00e8-GP system. (http://www.eprocure.gov.bd)..For\nmore . details, please ,contact help desk,\n(helpdesk@eprocure.gov.bd)\n)\n51108-11\n(Md. Nazmul Islam)\nID No-602232\nExecutive Engineer (C.C.), RHD\n(exe)\nRoad Division,-Bhola.\n", {"entities": [[130, 162, "co_inviter"], [164, 167, "inviter"], [193, 198, "inviter_location"], [298, 306, "tender_type"], [428, 470, "description"]]}],
["Image No. 1320\nfile_name:/content/content/jpg_files/20210901-102736398-990.txt\n\nOFFICE OF THE DIRECTOR (PLANNING & DEVELOPMENT)\n9/8/09 3 95\n-\n-\nKhulna University of Engineering & Technology\n30021 discription 3 2/3/15\nKhulna-9203, Bangladesh.\n1\nTel: PABX : 041-769468-75 Ext.- 199, E-mail : Fax: +880-41-774236\nTHE (00)/2025\n01/12/2: 05/06/2025\ne-Tender Notice\nE-Tender will be invited in the` National e-Gp System Portal\n(http://www.eprocure.gov.bd) for procurement of\nOnline Tender\nOnline\nSI.\nTender ID\nNotice\nTender\nPackage No.\nDescripti\u00f3n\nNo.\nNo.\nPublication\nClosing\ndate\ndate\n601379\nPackage No.-RG-02\nSupply of\n(KUET/P&\n(Revenue), 2021-2022\nElectrical\n31 August\n12\u00b0\n01\nD/39(05)/2\nLimited Tendering\nGoods for\n2021\nSeptemb\n021 Date-\nMethod (LTM) Khulna\nEngineering\ner 2021\n31/08/2021\nDistrict\nSection\nThis is an online Tender, where only e-Tender will be accepted in the National e- Gp\nPortal and no online/hard copies will be accepted. To submit e-Tender, registration\nin the National e-Gp system portal (http://www.eprocure.gov.b\u00e4) is required.\nThe fees for last selling/downloading the e-Tender Documents from the National e-Gp\nsystem portal have to be deposited online through any registered banks branches.\nFurther information and guidelines are' available in the National e-Gp system portal\nand from e-Gp help desk (helpdesk@eprocure.gov.bd).\n52\n31.08.21\n(Prof. Dr. Kazi ABM Mohiuddin)\nDirector, Planning & Development\nKhulna University of Engineering & Technology (KUET)\nFIC-3495/25 (E'xy)\nKhulna-9203.\n", {"entities": [[144, 189, "inviter"], [217, 223, "inviter_location"], [344, 359, "tender_type"], [645, 655, "description"], [702, 708, "description"]]}],
["Image No. 1070\nfile_name:/content/content/jpg_files/20210901-120215437-1052.txt\n\nGovernment of the Peoples Republic of Bangladesh\nOffice of the Executive Engineer (RHD)\nRoad Division, Bhola.\nE-maile eebho@rhd.gov.b\ngot\nTHCT $148 e\nMemo No. .5.01.0918.404.34.001.20-1752\nDate: . 31/08/2021\nInvitation for e-Tender (OTM)\ne-Tender is invited in the national e-GP system portal for the following works:\nTender ID\n601214\nName of Tender -\nAggregate Base Type II with Carpeting Seal coat work\nat Ch, 11+300 to 12+400km of Banglabazar-Khayerhat\nAzadnagar-Nurumiarhat-Daulatkhan (Z8917) Zilla\nHighways under Road Division Bhola during the Fiscal\nyear 2021-22 (Package No- 02/e-GP/Bho/Min/2021-22;\nTender Last Selling Date and\n14-Sep-2021 17:00\nTime\nTender Closing & Opening Date\n15-Sep-2021 12:00\nand Time\nThis is an online tender, where only e-t\u00e9nder will be accepted in e-GP portal and no offline\n/hard copy will be accepted. .To submit ertender, please register on e-GP system\nhttp:/www.eprocure.gov.bd; For more details please contact e-gp help desk,\n(helpdesk@ebrocure.gov.big\n31.08.21\n76\n(Md. Nazmul Islam)\nID No-602232\nExecutive Engineer (C.C.), RHD\nRoad Division, Bhola.\n(6x3)\nCG - 1894/21\n", {"entities": [[130, 162, "co_inviter"], [164, 167, "inviter"], [184, 189, "inviter_location"], [319, 328, "call_type"], [433, 485, "description"]]}],
["Image No. 1198\nfile_name:/content/content/jpg_files/20210901-102736398-992.txt\n\nGovernment of th\u00e8 People's Republic of Bangladesh\nit\nLocal Government Engineering Department\nOffice of the Upazila Engineer.\nGhoraghat, \"Dinajpur.\"\n(www.lged.gov.bd\nMemo No: 46.02.2743.000.99.001.21-840\nDate: *31.08.2021\nInvitation for Tender-Works/Goods\n(OTM):\n-Tender Notice No: e-GP/01/2021-22\nSL\nID Reference/\nNam\u00e8 of work\nTender\nTender\nNo\nPackage,no\nDocument last\nClosing Date\nselling Date &\n& Time\nTime:\nR\nK\n5-6-27\n1.\nTender Proposal ID: 600025,\n(a) Construction of wash block \u00e1t Ranigonj girls hight school and (b)\n23.09.2021\n23.09.2021\nPackage-No:\nConstruction of Baro paiker ghor high schqol Computer Digital class\nTime:12:00\nTime:14:00\nINF-2018-19-552743-01\nroom Under Upazila GhoraghatiDistrict Dinajpur.\n2.\nTender Proposal ID: 600026\nSupplying of High & Low- Brench at, Different School, collage,\n23.09.2021\n23.09.2021\n1\nPackage No.\nmadrasha Under Upazila Ghoraghat District Dinajpur.\nTime:12:00\nTime:14:00\n1NF-2018-19-552743-02\nir\n3.\nTender Proposal ID: 600027\nSupplying of ceiling fan at Different School, collage, madrashar Under\n23.09:2021\n23.09.2021\nPackage No.\nUpazila Ghoraghat District Dinajpur.\nzl\nTime:12:00\nTime:14:00\n<,\nINF-2018=19-552743-03\n4.\nTentier Proposal ID: 600020'\nSupplying of High &- -Low Brench at Different School and Supplying\n15.09.2021\n16.09.2021\nPackage No.\nof Computer set at, Ranigonj womens Digree college, Digital\nTime:17:00\nTime: 14:00\n1NF-2017-18-552743-02\ncomputer lab Under Upazila Ghoraghat District Dinajpur.\nSE\nThis is an online Tender, Where only e-Tender will be accepted- in the National e-GP portal and No. offline/hard copies will\nbe accepted. To submit e-Tender, registration in the national e-GP Systemporta (http/www.eprocure.gov.bc is required. The\nfees for downloading the e-Tender Documents from the:National e-GP System portal have to be deposited online through any\nregistered Banks branches Further Information and guidelines are available in the National e-GP System portal and' from e-\nGP help desk (helpdesk @.eprocure.gov.bd).\nInterested person can be communicating with the office of the undersigned during office hours.\nthe\n(Nogr Nobi Khan).\n54\nUpazila Engineer\nGhoraghat, Dinajpur\n15f7-000/25 (0,0X8)\ne-mail: _e.ghoraghat@lged.gov.bd\n,\n", {"entities": [[133, 172, "inviter"], [173, 204, "co_inviter"], [205, 225, "inviter_location"], [536, 593, "description"], [636, 703, "description"], [826, 858, "description"], [1054, 1078, "description"], [1382, 1394, "description"]]}],
["Image No. 1338\nfile_name:/content/content/jpg_files/20210901-102736398-993.txt\n\n74 Early\nNational Institute of Biotechnology\nBy\n04\nMinistry of Science and Technology\nis\nGovernment of the People's Republic of Bangladesh\n*\n<19 citcur\nGanakbari, Ashulia, Savar, Dhaka-1349.\n9/9/2019 6/5/2025\nPhone: +88-02-7789458, Fax: +88-02-7789636\nof\n(20/9:\nINVITATION FOR TENDER\n1\nMinistry/Division\nMinistry of Science and Technology\n2\nAgency\nNational Institute of Biotechnology\n3\nProcuring Entity Name\nProject Director, 'Establishment of Center for Next Generation Sequencing and Analytics'\nProject, National Institute of Biotechnology, Ganakbari, SaVar, Dhaka-1349\n4\nProcuring Entity Code\nN/A\n5\n,Procuring Entity District\nDhaka\n6\nInvitation for\nHiring of Microbus under the Establishment of Center for Next Generation Sequencing and\nAnalytics' Project\ni\n7\nInvitation Ref No.\n/39.06.2672.015.07.002.21\n20\n8\nDate\n31/08/2021\nKEY INFORMATION\n9\nProcurement Method\nNCT; Open\nFUNDING INFORMATION\n10\nBudget and Source of Funds\nGOB\n11\nDevelopment Partner (if Applicable)\nN/A\nPARTICULAR INFORMATION'\n12\nProject/Programme Code\n&\n224337500\n13\nProject/Programme Name\nit\nEstablishment of Center for Next Gen\u00e9ration Sequencing and Analytics\n14\nTender Package No.\nSingle Lot\n15\nTender Package Name\n01 (single lot): Hiring of Microbus under.the 'Establishment of Center for Next Generation\nSequencing and Analytics' Project\n16\nTender Publication Date\n01/09/2021\n17\nTender Last Selling Date\n12/09/2021\n18\nTender Closing Date and Time\n13/09/2021and 12.00 PM\n19\nTender Opening Date and Time\n13/09/2021and 12.15 PM\n20\nName and Address of the Office(s)\nSelling Tender Document (Principal)\nAccounts Branch, National Institute of Biotechnology, Ganakbari, Savar, Dhaka-1349\nSelling Tender Document (Others)\nN/A\nReceiving Tender Document\nTender documents (hard copy & CD of soft copy) should be dropped in the tender box kept\nat Administration Branch, National Institute of Biotechnology, Ganakbari, Ashulia, Savar,\n.\nDhaka-1349. Tender documents may be hand delivered or posted by mail or courier service\nwithin the specified date and time.\nOpening Tender Document\nMeeting Room, National Institute of Biotechnology, Ganakbari, Savar, Dhaka-1349\n21\nPlace/Date/Time of Pre-tender Meeting\nN/A\n(Optional)\nINFORMATION FOR TENDERERS\n22\nEligibility of Tenderer\nWhile purchasing tender document att\u00e9sted copies of (i) Up to date trade license, (ii) up to\ndate income Tax clearance certificate, (iii) VAT registration certificate, (iv) financial\nsolvency certificate, v) Car ,registration (Blue Book) vi) Fitness Certificate vii) Tax token\nviji) Insurance Certificate and document of providing of vehicle to different procurement\nentities should be shown and to be enclosed with tender document\n23\nBrief Description of R Services\nHiring of seven (7) seated AC Microbus with driver under the project of 'Establishment of\nCenter for Next Generation Sequencing and Analytics'. Microbus shall have to run all over\nthe country as per the project requirements.\n24\nPrice of Tender Document\nTk. 500.00 (Taka Five hundred only) in cash payment (nonrefundable)\n25\nPackage\nIdentification\nLocation\nTender security amount in Taka\nCompletion Time\nNo.\nof Package\nin weeks/months\n-01 (single\nMICBNGS\nGanakbari,\n15,000/- (BDT Twenty five thousand only); From any Schedule\n1.0 month\nlot)\nSavar, Dhaka\nBank of Bangladesh in the form of Crossed Bank Draft/Pay Order\nfavoring. Difector General, National Institute of Biotechnology\nPROCURING ENTITY DETAILS\n26\nName of Official Inviting Tender\nKeshob Chandra Das\n27\nDesignation of Official Inviting Tender\nProject Director\n28\nAddress of Official Inviting Tender\n'Establishment of Center for Next Generation Sequencing and Analytics' Project,\nNational Institute of Biotechnology, Ganakbari, Savar, Dhaka-1349\n29\nContact details of Official Inviting Tender\nthone:+88-02-7789289, Mob: 01712 287, Fax: +88-02-7789636;\nEmail:ngsnib@gmail.com\n30\nThe procuring entity reserves the right to accept or reject all tenders\nIf there is any transport disruption or Govt. holiday on the date for dropping/opening of the tender, the next normal working day will be\nconsidered valid date of dropping/opening of the tender. The procuring entity reserves the right to accept or reject all or part of tenders.\nProcurement items may be selected on priority basis/quantity of item may increase or decrease. This tender shall be governed by \"The Public\nProcurement Act\" and. \"The Public Procurement Rules\".\nBens,\n31/8/2021\nKeshob Chandra Das\n7-3560 (Jox8)\nProject Director\n", {"entities": [[89, 124, "inviter"], [252, 264, "inviter_location"], [717, 750, "description"]]}],
["Image No. 1143\nfile_name:/content/content/jpg_files/20210901-102736398-996.txt\n\nGovernment of the People's Republic of Bangladesh\nDirectorate General of Family Planning\n6214, C2691 62145\nLogistics & Supply Unit\n3/3/93\n6, Karwanbazar, Dhaka 1215.\n1.\nInvitation for e-Tender (OTM, NCT, Goods)\nMemo No. DGFP/L&S-3/Height & Weight/2021-22/17/17\nDate: 31/08/2021-4\ne-Tender is invited in the national e-GP portal for the procurement of following goods :\nSI\nTender\nPackage No.\nBrief Description of\nLast Selling Date &\nLast Closing Date &\nOpening Date &\nNo.\nID No.\nGoods\nTim\u00e9\nTime\nTime\nl'\n2\n3\n4\n5\n6\n7\n01\n598909\nMCH/GD40r/e29\nProcurement of Height\n22 September 2021\n23 September 2021\n23 September 2021\n& Weight Machine\nTime: 01:00 P.M.\nTime: 11:00 P.M.\nTime: 11:00 P.M.\nThis is an online tender where only e-tender (online tender) will be accepted through the national e-GP portal and no-\noffline/hard copy will be accepted. To submit e-tender please register on e-GP portal.\nThe fees for downloading the e-tender documents from the national e-GP portal have to be deposited online through\nany registered.bank.\nFurther information and guidelines are available in the national e-GP portal and from**-e-GP help desk (help Desk-\nhelpdesk@ebrocur\u00ea.gov.bd)/Contact Number (01762-625528-31)\n31/08/2021,\n(Ratna Talukder)\nDirector (Logistics and Supply) &\nLine Director (PSSM-FP)\nDirectorate General of Eamily Planning\n.\nPhone: 55012349\nE-mail: dirlsdgfp@gmail.com\nDG-623/21 (5\"*x4)\n", {"entities": [[130, 168, "inviter"], [221, 239, "inviter_location"], [360, 368, "call_type"], [618, 639, "description"], [694, 710, "description"]]}],
["Image No. 1058\nfile_name:/content/content/jpg_files/20210901-103042276-998.txt\n\nEASTERN CABLES LIMITED\n*\nNorth Patenga, Chattogram, Bangladesh.\nSOCOTEC\n: Invitation for Local Tender Notice (OTM) for following item\n01.\nMinistry\nMinistry of Industries\n02.\nName of Agency\nBangladesh Steel & Engineering Corporation, Dhaka\n03.\nProcuring Entry Name & District\nEast\u00e8m Cables Limited. North Patenga, Chattogram.\nS/L\nName of Procur\u00e9ment\nReference No.\nCost of Tender\nTender\nReceiving\nOpening Date\n& Date.\nDocuments\nSecurity\nDate & Time\n& Time, for\n5\nI\nRicz TIBCE FICAS\n04\nEC/PUR/L-1/1 101 &\ndt, 14.09.2021\ndt. 14.09.2021\nOUTHERE &\n121/249\nTk : 1,000.00\nTk : 10,000.00\nat 12.00 PM.\nat 12.15 PM\nPTO 354812 afout\nDate: 31.08.2021.\n(\u20b9\n$\n05.\nBudget & Source of Funds\nOwn\n06.\nLast Tender Selling Date\nDate : -13.09.2021\n07.\nSelling place of Tender Documents\n1. Cash Section, Head of Accounts, Eastern Cables Limited, North Patenga, Chattogram.\n2. Cash Section, Controller of Accounts, BSEC Bhaban, Dhaka-1215\n08.\nPlace of Receiving & Opening\nPurchase Department, Eastern Cables Limited, North Patenga, Chattogram.\nTender Documents\n09.\nEligibility of Tenderer\nAs per Tender Document.,\n10:\nBrief Description of Goods\nAs per Tender Document Specification.\n11,\n-Location of Delivery.\nStore Section, Eastern Cables Limited, North Patenga, Chattogram.\n12.\nName of Official Inviting Tender\nHead of Purchase Department, Eastern Cables Limited, North Patenga, Chattogram.\n13.\nDesignation of Official Inviting\nEngineer (Mechanical)\nTender\n14,\nAddress of Official Inviting Tender\nEastern Cables Limited, North Patenga, Chattogr\u00e1m.\n20\n(Engr.\nHead of Purchase Department\nT-000(ex8)\nFor Managing Director.\n", {"entities": [[80, 102, "inviter"], [105, 130, "inviter_location"]]}],
["Image No. 1297\nfile_name:/content/content/jpg_files/20210901-103358556-1000.txt\n\n<1291642 911 6915\nR&13 40st\nBangladesh Water Development Board\nOffice of the\noperating 987\nExecutive Engineer\nPanchagarh O&M Division-\nI.\nBWDB, Panchagarh-5000.\n06/018\nT\u00e9l: 0568-61393 (Office)\n(21-711)\n0568-61393-(Res.)\noct-issue\nwww.bwdb.gov.bd\nemail.x\u00e9nbwdb.panchagarh@yahoo.com\n<<<<\n9/12/20 459/40 Resp\n969/51 special GIOLS Gillicat VIIGUE 6T, 933 Rom 9215515\n9/14/19 RIC2191-200f UST 51160165 7/25 9918 (LTM) 93\nBrittle ou(Tea) 60110 Glat 925 TOT 979 Re(91for) 6151 9/23 9005\n3545112 96255 2025-22 grat Tenant Gard\nBelhist 741314 <PSIT 264 I 03/44/25 as CTO, 01415\n611512 - 20/09/20203: 02/15/655 2025-\n2022 gref <2655 GINT 24/3/01 for <144 2000:00 (52 2/G Glat ok 50% GIT <14th\n000.00 25 02 (5\u20b9) ft for 102/ 5/2PT 75/3/08\nfor\n<14th family 7500 51/28/20 R2114 6th (531).) TIME 93 52566\n9/15/19 6/4 436515 all 2114564 55 265 Blickhar\nTHE PSI 21164 ! Brink 3 That 2145 3/5/09 8.00\n21104 260 Role 0.00 211645 OR 3/45/20. 211004 1, FINITO 3/15/20 3 spical 95th\nat I\nBelnis 04/3/2015\nafter to\n01/15/20\n74151695 459\n2\n2025-2022 510\n18/08/2025\n<2655 5795\n358/2025-2022\n24\n59\n(58h1)\n953 Right\n(5 xo)\n", {"entities": [[109, 143, "inviter"]]}],
["Image No. 1091\nfile_name:/content/content/jpg_files/20210901-103358556-1003.txt\n\n9117 523/07 6415\n-\nOffice-of th\u00e9\n-\nG412 4351\nExecutive-Engineer\n933\nChattogram O&M Division-2\nDISTRICT I 1\nBWDB, Chattogram.\n62019 : 005-beoust\nTel.No. 031-650612.\nE-mail : xen.ctg2@gmail.com\nMemo No : EE/CTG-2/T-6/295\nDate : 31-08-2021\ne-Tender Notice-03/2021-2023\nAn e-Tender has been invited in the national e-GP System Portal (http://www.eprocure.gov.bd/ for\nprocurement of the following works.\nPackage No. & Description\nLocation of\nTender/\nTender/\nSI\nTender\nNo.\nID-\nthe Works/\nProposal\nProposal\nSupply\nDo\u00e7ument Last\nClosing &\n1\nSelling D\u00e2te &\nOpening Date\nTime\n& Time\n\u00c7hanua,\n29-September\n30-September\nl'\n600448.\n(Package : CTG2/W-37)\nSeta-side slope prot\u00e9ctive work by\nBanshkhali,\n2021\n2021\nblocks including breach\nChittagong.\n16:00:00\n14:00:00\nclosing & re-sectioning of\nembankment from km. 15.950 to\nkm. 18.500 = 2550.00 m. at Chanua\nin the Coastal Area of Polder no.\n64/1C in Upazilla- Banskhali,,Dist\nChittagong, in connection, with\n\"Rehabilitation ,of BWDB\ninfrastructures Damage by Natural\nDisaster in the Coastal Area of\nPolder No-64/1A, 64/1B & 64/1C\nat Banskhali Upazilla in the\n11\nChittagong District\" under\nChittagong O&M Division-II,\nBWDB, Chittagong during the year\n2021-2022. (Remaining part of\nPackages CTG2/W-9,W-10, W-11,\nW-12, W-13 & W-14)\nThis is an online Tend\u00e9r, where only,e-Tenderywill be accepted in the,National e-GP portal and no\noffline/hard copi\u00e8s will be accepted. To submit \u00e9-Tender, registration in the National e-GP System Portal\nNational e-GP System portal have to be deposited online through any registered Bank's branches up to\n(http://www.eprocure.gov.bi) is required.; The fees for downloading, the e=Tender Documents from the\nFurther information and guidelines are availabld in the National e-GP System-Porta and from e-GP help\n12:00:00 on 30-09-2021.\ndesk (helpdesk@eprocure.gov.br (01762625528-31).\n977-336/2025-2022\n(Nahid-Uz-Zaman Khan)\n\"-\nExecutive Engineer\n", {"entities": [[188, 192, "inviter"], [100, 144, "co_inviter"], [149, 160, "inviter_location"], [318, 333, "call_type"], [731, 752, "description"], [778, 801, "description"], [842, 869, "description"]]}],
["Image No. 1258\nfile_name:/content/content/jpg_files/20210901-103358556-1005.txt\n\nGOVERNMENT OF THE PEOPLE'S REPUBLIC OF BANGLADESH\nMINISTRY OF HEALTH AND FAMILY WELFARE\nDIRECTORATE GEN\u00c9RAL OF FAMILY PLANNING\nLOGISTICS & SUPPLY UNIT (STH FLOOR)\n6 KARWANBAZAR, DHAKA 1215, BANGLADESH.\n100\nwww.dgfp.gov.bd\ne-Tender Notice (OTM, NCT, Goods)\nFile Name: DGFP /L&S-2/ Almirab/2021-22/50/T2\nDate: 31/08/2021\n-Tender is, invited in the .national e-GP portal for the procurement of following goods:\nTender ID No. & Package:\nTender Packag\u00e8\nLast Down-loading\n\u20b9\n(Closing\nOpening\nNo.\nname\n/selling Date & Time\nDate &Time E\nDate-&Tim\u00e9\n1\n2\n3\n4\n5.\n'\nTender ID No : 598882\nProcurement of\n20/09/2021,\n21/09/2021\n21/09/2021\nPackage No PSSM/GD2/e2-\nAlmirah\n16 : 00\n14:00\n14:00\nThis is an online tender where only e-tender (online tender) will be accepted thro\u00f9gh the national e-GP\nportal and no offline/hard copy will be accepted. The fees for downloading the e-tender documents from\nthe national e-GP portal have to be dep\u00f3sited online through any schedule bank of Bangladesh registered\nby the Central Procurement Technical Unit (CPTU).\nFurther information visit. national e-GP portal and from \u00e9-GP* help desk (H\u00e9lp Desk e-mail:\nhelpdesk@eprocure.gov,bd\nr'\n(Ratna Talukder)\n00/27/08/2021\nDirector (Logistics & Supply)\n35\n%\nLine Director (PSSM-FP)\nLog\u00edstics & Supply Unit\nGD-AK(-377.9.21(4X34\nTelephone: 880-2-55012349\n", {"entities": [[131, 168, "inviter"], [169, 207, "co_inviter"], [246, 264, "inviter_location"], [303, 318, "call_type"], [728, 735, "description"], [655, 669, "description"]]}],
["Image No. 1147\nfile_name:/content/content/jpg_files/20210901-103358556-1007.txt\n\nSYLHET CITY CORPORATION\nBangiso\nNagar Bhaban, Sylhet.\n50\n4\nMUS\n:\nthe\nthe\nwww.scc.gov.bd\n00\n28\n3112 maze sife\"\nMemo No: 46.07.0000.018.99.025.19.303\nDate. 31/08/2021\n-\n-Tender Notice-07/2021-22\n1.\nMinistry/ Division\nMinistry, of the Local Government, Rural\nDevelopment & Co-operatives.\n2.\nAgency:\nSylhet City Corporation.\n3.\nBudget and source of Funds:\nADP/ADP (Special)/Revenue fund of SCC.\nSupplying & Installation of Duplex type Medical Oxygen\n13/09/2021\n603212\n(OTM)\nGenerator (PSA- Plant) beside 10 storied M.A.G. Osmani\ns\nMedical College & Hospital building, Sylhet\n4.\nName, Designation & Address Engr. Md. Ali Akbar\nof PE\nExecutive Engineer.\nSylhet City Corporation.\n0821-714036\n5.\nThis is online Tender, where only- e-Tender will be accepted in the National e-GP\nPortal and offline/hard copies will not be accepted.\nIriterested\npersons/firms\ncan\nsee\ndetails in the web site : www.eprocure.gov.bd\n6.\nThe fees for downloading the e-Tender Documents from the National e-GP System\nPortal have to be deposited online through any registered Banks Branches.\n7.\nFurther\ninformation\nand\nguidelines\nare\navailable\nin\nthe\nNational\ne-GP\nSystem\nPortal\nand from e-GP help desk (helpdesk@eprocure.gov.bd\nExecutive Engineer\nSvlhet City Corporation.\n", {"entities": [[81, 104, "inviter"], [127, 133, "inviter_location"], [472, 526, "description"], [551, 560, "description"]]}],
["Image No. 1315\nfile_name:/content/content/jpg_files/20210901-103358556-1010.txt\n\n$12 9/64/4917\n5/1 91649\nMUJIB\nFax: 49270201\n100\nG/D 3\nTel: 94270077\nE-mail: se.building@beri.gov.bd\nBangladesh Agricultural Research Institute\nWeb: www.bari.gov.bd\nGazipur-1701\nMemo no.12.21.0000.010.41.468.20.110\nDate: 29.08.2021\ne-TENDER NOTICE\nTender is invited Through National e-GP System *Portal (http//:wwweprocure.gov.bd) for procurement of\nfollowing works:\n$\nSI\nTender\nTendering\nDescription of works,\nOnline (e-GP\nOnline (e-GP\nno\nID No.\nMethod\nsystem) tender\nsystem) tender\npublication date &\nclosing date &\ntime\ntime\n1.\n601738\nOTM\nConstruction of irrigation Infrastructureat SRSC,\n29-Aug-2021\n15-Sep-2021\n-\nBARI, Faridpur under Strengthening of Spices Crop\n15:35\n14:30\nResearch in Bangladesh\n2.\n599740\nOTM\nLand Development 3 work at SRSC, BARI,\n29-Aug-2021\n15-Sep-2021\nLalmonirhat under Strengthening of Spices Crop\n15:55\n14:40\nResearch in Bangladesh\n3.\n599736\nOTM\nLand Development work at RSRC, BARI, Gazipur\n29-Aug-2021\n15-Sep-2021\nunder Strengthening of. Spices Crop Research in\n15:50.\n14:20\n-\nBangladesh\n4.\n599727\nOTM\nLand Development work at SRSG, BARI, Faridpur\n29-Aug-2021\n15-Sep-2021\nunder Strengthening of Spices Crop Research in\n15:55\n-14:00\n-\nBangladesh,\nThis is online tender, only e-Tender will be accepted in National ,e-GP Portal and no offline/hard copy will be accepted.\nRegistration is essential to submit e-Tender.\nFees have to be deposited for downloading the tender documents from e-GP.Portal through any registered bank. For\nfurther information please contact e-GP help desk (helpdesk@eprocure.gov.bd).\n37\nEngr. Md. Ferozur Rahman\n171G-258/25 (5 \"x8)\nSuperintending Engineer (In-charge)\n", {"entities": [[181, 223, "inviter"], [245, 252, "inviter_location"], [312, 327, "call_type"], [622, 670, "description"], [1113, 1134, "description"]]}],
["Image No. 1305\nfile_name:/content/content/jpg_files/20210901-103358556-1009.txt\n\nCIVIILAVIATONAUTHORITYOFBANGLADESH\nKURMITOLA, DHAKA-12\nInvitation for Tenders\n1 Ministry/Division\nCivil Aviation Authority of Bangladesh\n2 Agency\nN/A\n3\nProcuring Entity Name\nCEMSU, CAAB.\n4\nProcuring Entity Code\nNot used at present\n5\nProcuring Entity District\nDhakair\n6\nInvitation for\nDesign, Develop, T\u00e9st, Commissioning .and Managed\nService of CAAB independent Website.\n7\nInvitation Ref No\n30.01.0000.121.31.005.17 (Part-1)/52, Date : 22/08/2021\n8\nDate\nKEY INFORMATION\n9\nProcurement Method\nOpen Tendering Method (OTM)\nFUNDING INFORMATION\n10 Budget and Source of Funds\nCAAB's own-Fund CODE NO-100\n11\nDevelopment Partners (if applicable)\nN/A\nPARTICULAR INFORMATION\n12 Project / Programme* Code (if N/A.\napplicable).\n13\nProject / Programme Name (if N/A\napplicable)\n14\nTender Package No.\n15\nTender No.\nCEMSU/COM-WS - 03/2021-2022\nTender Package Name\nDesign, Develop, Test, # Commissioning and Managed\n16\nService of CAAB independent Website.\n17\nTender Publication Date\n02/09/2021\n18\nTender Last Selling Date\n03/10/2021\n5.00 P.M.\nDate\nTime\n19\nTender Closing.Date and Time\n04/10/2021\n12.00 P.M.\n20\nTender, Opening Date and Time\n04/10/2021\n12;15 P.M in presence of\nTenderers or their authorized\n-\nrepresentatives (if any)\n21\nName & Address of the office(s)\nAddress\n-Selling Tender Document (Principal)\nExecutive Director, CEMSU; CAAB, Kurmitola, Dhaka.\nSelling Tender Document (Qthers)\nN/A\nReceiving Tender Document\nExecutive Director, CEMSU, CAAB, Kurmitola, Dhaka,\nOpening Tender Document\nExecutive Director, CEMSU, CAAB, Kurmitola, Dhaka.\n22\nPlace / Date / Time of Pre-Tender Office of the Executive Director, CEMSU, CAAB,\nMeeting\nKurmitola, Dhaka. Date: 13/09/2021, Time: 11:00 AM,\nINFORMATION FOR TENDERER\n23 Eligibility of Tenderer : (Documentary\nAny Bangladeshi Tenderer shall, meet the flowing\nevidence must be submitted)\neligibility criteria as a minimum to participate in the tender\ncompletion.\n1.\nThe minimum number of years of general experien\u00e7e\nof the Tenderer in contracting industries in public\nsector\nas\nPrime\nContractor/Sub\nContractor/Management Contractor-shall be 03\n(Three) syaars. sconcern documents must be\nsubmitted).\n2.\nTh\u00e9 milimum specific experience of design,develop\nand impl\u00e9ment-of Website as a-Prime Contractor in\n11\nGovernment/Semi-government/Autonomous\norganization at least 01.(ONE) contract of, similar\nnature and methods/technology-completed over a\nperiod of last 03 (Three)-year with a value of at least\nof Tk. 35--Thirty five) lac shall be required and the\n01 (One) .website in operation of\nI\nGovernment/Semi-government/ Autonomous\norganization. (All concern documents `must be\nsubmitted).\n3.\nThe. minimum amount of liquid assets i.e working\n'capital or credit line(s) of the Tenderer sh\u00e1ll be Tk. 30\n(Thirty) lac. Adequacy of minimum liquid assets,\n>substantiated by any scheduled Bank of Bangladesh\nin the>format as specified without alteration, issued\nnot earlier than twenty-eight.(28) days prior to the\nday of the original deadline ,for *submission of\nTenders for this Contract. (All concern documents\nmust be submitted).\n24\nBrief Description of Works/ Related\nDesign, Develop, Test, Commissioning and Managed\nServices\nService of CAAB independent Website.\n25\nPrice of Tender Document\n1000.00 (One Thousand) Only (Non refundable)\n26\nLot\nIdentification of\nLocation\nTender Security\nCompletion Time in Weeks/\n.No\nLot\nAmount (Tk)\n'Months\n-\n:\nTk. 1.10 lac in favour\n120 (One Twenty) days front-the\nof Ex\u00e9cutive Director,\nsigning of the contract- As Per\nCEMSU, CAAB,\nSection-4)\nDhaka-1229.\nPROCURING ENTITY DETAILS\n$\nus\n27\nName of Official Inviting Tender\nExecutive Director, CEMSU, CAAB, Kurmitola, Dhaka.\n28\nDesignation of Official Inviting Tender\nDirector\n29\nAddress of Official Inviting T\u00e9nder\nCEMSU,\"CAAB, Kurmitola, Dhaka.\n30\nContact details of Official Inviting\nTel.No.02-\nFax No. 8901411\ne-mail :\nTender\n8901791 & 02-\nmohsin ed @caab:\n8901223\ngov.bd\n31 The procuring entity reserves the right to accept or reject all tenders\nSd/-\n(Md. Mohsin)\n05/06/2025 for:\nExecutive Director\nCentral procurement, Engineering & Supply Unit\nCivil Aviation Authority\nG-2094/09/21 (13x3)\nKurmitola, Dhaka-1229\n", {"entities": [[81, 115, "inviter"], [116, 132, "inviter_location"], [350, 423, "description"]]}],
["Image No. 1063\nfile_name:/content/content/jpg_files/20210901-103358556-1013.txt\n\nGovernment of the People's, Republic of Bangl\u00e4desh\nDepartment of Fire Service & Civil Defence\nBangladi\nKazi Alauddin Road, Dhaka.\nwww.fireservice.gov.bd\n1\ne\n16/05/1428 Bangla\nMemo No. 58.03.0000,008.07.01.20201-2011/1391\nDate-\nI\n31/08/2021 a.d.\ne- Tender Notice No: FSCD(Purchase & Store) G-01 /2021-2022\ne-Tender is invited in the National e-Gp-system Portal (http:/www.eprocure.gov.bd)\nfor the procutement of goods as mentioned in the following table:\nSI Tender\nDescription\n1.\nQty\nLast Selling\nClosing &\nID:No\nDate & Time\nOpening Date\nof e-Tender\n& Time of e\nDocument\nTender\nDocument\n1.\n598316\nProcurement of Kads (Officer)\n1030 Pair\n21.09.2021\n22.09.2021\n2.\n598430\nProc\u00farement of Gum Boot\n3000 Pair\nat 17.00\nat 15:00 pm.\n3.\n598431\nProcurement of Beret Cap (Red Colour)\n11432 Nos\npm.\n4.\n598433\nProcurement of Field Cap Stitching\n11432 Nos\n(with Monograms)\n:\n&\n5.\n598434\nProcurement of Deep Sign\n20804 Nos\nShoe Brush Black\n11432 Nos\nShoe Polish Black'\n11432 Nos\nThis\nis\nan\nonline Tender where only'e-Tenders-will be accepted in the national e-GP system\nportal\nNo offline /hard copies will be acceptable. To submit e-tender please register in the\nnational'e-GP portal system (http://www.eprocure.gov.bd). For more details please contract\nhelp desk (helpdesk@eprocure.gov.bd)\n1\n==\nThe fees for downloading the e-tendering documents from the national e=GP Portal\nhave\nto\nbe deposited online through any registered bank's branches up to 22.09.2021\nat 14.00 pm.,\n59\nMd. Habibur Rahman\nJoint Secretary\nD-1068/21 (8x4)\nDirector (Admin & Finance)\n", {"entities": [[132, 174, "inviter"], [204, 209, "inviter_location"], [326, 342, "call_type"], [677, 706, "description"], [749, 772, "description"], [815, 852, "description"], [877, 911, "description"], [953, 977, "description"], [988, 1004, "description"], [1015, 1032, "description"]]}],
["Image No. 1337\nfile_name:/content/content/jpg_files/20210901-103358556-1012.txt\n\nTHE 211/7 6515\n7412 535\nOffice of the Executive Engineer\n95615 Ropt\nChattogram Dredger Operation Division\nTIMECTI, 5515, I\nBWDB, Sagorika Road, Chattogram.\nxendredgerctg@gmail.com\nE-mail: xendredgerctg@gmail.com\n:\nPhone: 02-43151182\n: 03/06/2025 3\n4594 9/11/600 3 74131675 Riset's'\n957/51\n45695\n9/9/165\nGATS\n251005\n532112\n55G18\nFrom\nTIMECAL, 51/5/19/19 6515, 502112 4365 9/4/95\n4594 51136131 Glark 60110 G1st 9213 T/P 13 20 (Affor) 95\n61st 9233 9/9/5 35451625 956 2025-22 7976555 Gats\n13\nTX\n264\n1\n03/44/25\ngo\n67,\n5/4/5\n<<<<\nTo\n82.\nOS\n.000.000.\n0/18/20\n:\n20/09/2020\n33:\n62101645\n972\nast\n1448 REP 7/44 800.00 (910)(5) Blaks 6%\n5/5/5/45\n5/20\nEx.\n31/25/09\nfamily\n6th\n(3014),\n03.\n9256\n<55:\nB/CO\n43655\n91121695\n2114562\n3022\n05\n21164 I Brink G 3 TO ATTER 3 3540118 that 2/15/ 3/5/09\n5.00 zillerl 265 Rapla 0.00 9/2/33 0/15/20 211566 I 61182 B 3126515 9/8\n01/19/2018 4PST REFIRE 3 202 072 3116440 2/20/20 PST 264 7T1\n511634 3 02/15/19\n5/2/20\n2P52 FLE\n2P52 RICH 6014\n5/16/hay\nof\n4500\n01/12/15\n2/2605\n30\n8\nC\n*\ngone\nos\n05/08/2025 3\n18/08/2025 23:\n23/08/2025 30\n3 74/3/01 (T1)\n021\n05/08/2025 3:\n10/08/2025 38\n25/08/2025 33:\n3 74/3/01 (797)\n(CATT8\n37\nfate\nDENIT 57515 fight\n(&xo)\n6515, DUENT I\n", {"entities": [[105, 137, "co_inviter"], [204, 208, "inviter"], [210, 235, "inviter_location"]]}],
["Image No. 1009\nfile_name:/content/content/jpg_files/20210901-103358556-1014.txt\n\nGovernment of the People's Republic of Bangladesh\nOffice of the Chief Signal and Telecommunication Engineer (Telecom)\nMURB\nBangladesh Railway, Railbhaban, Dhaka-1000.\nNo:- 54.01.0000.501.16.015.17-2128\nDate: 31.08.2021.\nProject Name: Construction of (Khulna-Mongla Port Rail Lirte.\nPackage-No-WD 3: Construction of Non-interlocked color light Signaling System for all\nstations (09) including construction of Telecommunication System with.optical fiber.\nRef: IFT No.- 54.01.0000.501.16.015.17-2094, Dated: 19.07.2021.\n1\nCorrigendum:no.-1(R\u00e8vised), Dated: 31.08.2021\nSI, No.\nSubject\nExisting Provision of\nAmended Provision\nOf IFT.\nCorrigendum no.-1, (in.IFT and\nTender Document)\nTender Document)\n17\nTender bast Selling Date :\n01.09.2021 (During Office hour)\n03.10.2021 (During Office hour)\n18\nTender Closing Date- and Time :\n02.09.2021, 12.00 hrs BST\n04.10.2021, 12.00 hrs BST\n19\nTender Opening Date and Time:\n02.09.2021, 12.30. hrs. BST\n04.10.2021, 12.00 hrs BST\n1\nAll others terms and conditions of the above^Tehder will remain unchanged.\n(Rubayat Sharif)\n66\nDivisional Signal & Telecommunication Engineer (Telecom) HQ\nBangladesh Railway, Railbhaban, Dhaka.\n01711-500893\nGD-ANS-610,9.21(5x4)\ndstethq@railway.gov.bd\n", {"entities": [[204, 222, "inviter"], [224, 241, "inviter_location"], [600, 614, "call_type"]]}],
["Image No. 1331\nfile_name:/content/content/jpg_files/20210901-103358556-1016.txt\n\n4551\nBanglad\n3/7/31\n4/2/19\n694 21/2/19\n65691 : Ficette\nare 12655 GAN\nwww.lged.sylhet.gov.bd\nMemo No. 46.02.9100.000.14.081.21-2175\nDate : 31/08/2021\ne-Tender Notice No : 09/2021-22\ne-Tenders are hereby invited in the National e-GP Portal (http://www.eprocure.gov.bd) for the Procurement of the\nWorks stated as below :\nSI.\nPackage N\u00f2.\nName of Works\nTender\nMethod of\nClosing\nNo.\nID No.\nTender\nDate & Time\n1\nSDIRIIP/SYL/\na) Improvement of Lakkanabondh-Chowdhury\n600414\nOSTETM\n03-10-2021\nUNR/W-016\nazar-Nowai-Kargaon-Fakirtul-Purbobagh Bazar\n14:00:00\nRHD Road by BC from Ch. 2200m-5252m. (Road\nID: 691383007). (b) Construction of 3 Nos\nX1.50mx1.50m Drainage Culvert on the Same Road\nat\nCh. 2396m, 2789m & 3709m and 6\nnos1x.625mx0.600m Drainage Culvert on the Same\nRoad. at 2520m, 3920m, 4120m, 4658m, 4823m &\n5250m. (c) 128m Protection work on the same Road\nUnder Upazila : Golapganj, District: Sylhet.\n2\nSDIRIIP/SYL/\na) Improvement of Toyakul UP Office-Kurikhal Road\n600415\nOSTETM\n03-10-2021\nUNR/W-027\nby BC from Ch. 1000-2600m. (Road ID No.\n14:00:00\n691413005). (b) 22m Protection work on the same\nRoad under Upazila: Gowainghat, District: Sylhet.\n3.\nSDIRIIP/SYL/\na) Improvement of Zakiganj UP Office (Hafsa\n600416\nOSTETM\n03-10-2021\nUNR/W-028\nCollage)-Manikpur-Muminpur-Thana Bazar Road by\n14:00:00\nBC from Ch.1130-2500m. (Road ID. 691943009). (b)\nImprovement of Zakiganj UP Office (Hafsa Collage)-\nManikpur-Muminpur-Thana Bazar Road by Soil\nfrom Ch.1130-2500m. (c) Construction of 3 Nos.\n1.0x2.00x2.00m Size Drainage Culvert Ch. 1170m,\n11\n1610m & 2090m and 1.00x.0.625x0.600m Size 3\nNos Drainage Culvert at Ch. 1953m, 2005m and\n2210m on the Same Road under Upazila: Zakiganj,\nDistrict: Sylhet.\n4\nSDIRIIP/SYLI\na) Improvement of Koskonokpur UP Office-R&H-\n600417\nOSTETM\n03-10-2021\nUNR/W-029\nKodurpur Bazar-Kamalganj Bazar Road by BC from\n14:00:00\nCh. 00-2330m. (Road ID. 691943001). (b)\nImprovement of Koskonokpur UP Office-R&H-\nKodurpur Bazar-Kamalganj Bazar Road by Soil from\nCh. 00-2330m. (c) Construction of 4 Nos.\n1.0x2.00x2.00m Size Drainage Culvert Ch. 890m,\n1183m, 1710m & 1825m and 1.00x.3.00x3.00m Size\n1 Nos Drainage Culvert at Ch. 2305m on the Same\nRoad. (d) 188m Protection work on the same Road\nunder Upazila : Zakiganj, District : Sylhet.\n5\nSDIRIIP/SYL/\na) Improvement of Nijpat UP Headquarter-Digarail-\n600418\nOSTETM\n03-10-2021\nUNR/W-030\nSarighat Bazar Road by BC from Ch. 1300-2300m.\n14:00:00\n(Road ID No. 691533008). (b) Improvement of\nNijpat UP Headquarter-Digarail-Sarigha Bazar Road\nby Soil from Ch. 1300-2300m. under Upazila :\nJointiapur, District : Sylhet.\nThis is an online Tender Notice, where only e-Tenders will be accepted in the National e-GP Portal and no\nOffline/Hard copy will be accepted.\nTo submit e-Tender(s), registration in the National e-GP Portal (http://www.eprocure.gov.bd) is required.\nThe fees for downloading the e-Tender Documents from the National e-GP Portal have to be deposited online at\nany branch of registered banks.\nFurther information and guidelines are available in the National e-GP System Portal and e-GP help desk\n(helpdesk@eprocure.gov.bd)\nThe Tender Notice will be available on the website : www.lged.gov.bd & www.lged.sylhet.gov.bd.\n(Md. Enamul Kabir)\nExecutive Engineer\nLGED, Sylhet\nPhone : 0821-716116\nGC-2092/09/21 (10'x4)\ne-mail : xen.sylhet@lged.gov.bd\n", {"entities": [[230, 245, "call_type"], [502, 539, "description"], [575, 618, "description"], [812, 828, "description"]]}],
["Image No. 1202\nfile_name:/content/content/jpg_files/20210901-103358556-1015.txt\n\nGovernment of the People's Republic of Bangladesh\nDirectorate of Primary Education\nSection-2, Mirpur, Dhaka-1216\nwww.dpe.gov.bd\nInvitation for Tender (IFT)\n1 Ministry/Division\nMinistry of Primary-and Mass Education.\n2\nProcuring Entity Name\nProgram Director, Fourth Primary Education Development program (PEDP4)-\n3\nProcuring Entity Code\n2431\n,4\nProcuring Entity Distri\u00e7t\nDhaka\n5\nInvitation for Tender\n1. Procurement of Office Stationeries & Toiletries (131 items)\n2. Procurement of Computer & Photocopier Toners & Accessories (47 items)\n3.\nProcurement of Crockeries (22 items)\n6\nInvitation Ref No\n38.01.0000.07.005.154.2020-1134\n7\nDate\n31 August, 2021\nKEY INFORMATION\n8.\nProcurement Method\nOpen Tendering Method (National) [(24 (Tw\u00e8nty Four) Months Framework\nContract Agreement)]\nFUNDING INFORMATION\nBudget-and Sourceiof F\u00fcnds\nGoB and Under\nitsign\nDevelopment Partners\nADB, World HICA\nterric cl Peters\nPARTICULAR INFORMATION\n11.\nProgram Cod\u00e9\n224237900\n12.\nProgram Name\nFourth Primary Education Development Program (PEDP4).\n11\n13.\nTender Package No\nGD 232.01, GD 233.01 and GD 234.01\n14.\nTender Package Name\nSupply of Contingency;\n1.\nProcurement of Office Stationeries & Toiletries (131 items)\nProcurement of Computer & Photocopier Toners & Accessories (47\n,\n2.\nitems).\n3.\nProcurement of Crockeries (22 items).\n15.\nTender Publication Date\n04 September, 2021\n16.\nTender Last Selling Date\n21 September, 2021\n17. Tender Closing Date and Time\n22 September, 2021 and 1:00 pm\n18.\nTender Opening Date and Time\n22 September, 2021 and 1:30 pm\n19.\nName and Address of the Office(s)\n-\nSelling Tender Document\nProcurement Division, Room no. 811 (7th floor); Directorate of Primary\n(Principal)\nEducation, Section 2,Mirpur, Dhaka-1216.\u00b9\n-\nDownload of Tender Document\nWebsite of Directorate of Primary Education (www.dpe.gov.bd).\n(Others)\n- Receiving Tender Document\nMinistry of Primary and Mass Education, Room no. 610 (6th floor), Building no.\n06, Bangladesh Secretariat, Dhaka-1000.\nOpening Tender Document\nMinistry of Primary and Mass Education, Room no. 610 (6th floor), Building no.\n06, Bangladesh Secretariat, Dhaka-1000.\n-Place / Date / Time of Pre-Tender\nNone\nMeeting\n(Display of Samples by DPE: From 01 September 2021-to 21 September 2021,\nGround Floor, DPE, from 10.00 am to 4 pm on every working day)\nINFORMATION FOR TENDERER\n20. Eligibility of Tenderer\nTend\u00e9rer shall have legal, financial, professional and technical knowledge and\ncapacity to deliver products as per requirements of DPE time to time and of this\ntender and need to be fulfilled the obligations of all tax and VAT\nrequirements;Minim of 3 (three) years of general experience of the Tenderer\nin the supply of Goods and related services as Supplier\n21. Brief Description of Goods\na) Package GD 232.01: Procurement of Office Stationeries & Toiletries (131\n\"items);\nb)\nPackage GD 233.01: Procurement of Computer & Photocopier Toners &\nAccessories (47 items).\nc)\nPackage GD 234.01: Procurement of Crockeries (22 it\u00e9ms). (Tenders are\nbeing invited for single/multiple Packages. Tenderer-may submit for all\nPackages or individual Package. Tend\u00e9rer will be evaluated tender package\nwise. Price quotation needs to cover 100% of the items specified for\nindividual package).\n22.\nBrief Description. of Related\nNone\nServices\n23.\nPrice of Tender Document (non-)\n1.\nPrinciple: Tk. 1000.00 (Taka One Thousand Only) cash for each with an\nrefundable)\nofficial letter addressing Director (Procurement), Directorate of Primary\nEducation; and\n2.\nOthers: In case of Online Purchase a treasury chalam of Tk. 1000.00 (Taka\nOne Thousand Only) for each package should be attached during the\nsubmission (Code No. 1-2431-0001-2366).\n24.\nIdentification of Packages\nLocation Tender S\u00e9curity Amount (in Tk.) Completion Period\nPackage GD 232.01: Procurement of Office\nCentral\nTk. 1,60,000.00 (Taka One Lac.\n24 (Twenty Four)\nStationeries & Toiletries (131 items)\nStore, DPE,\nSixty Thousand Only)\nmonths under a\nPackage GD 233.01: Procurement of Computer\nDhaka\nTk. 7,00,000.00 (Taka Seven Lac\nFram\u00e8work\nOnly)\nContract (Multiple\n& Photocopier Toners & Accessories (47 items)\nCall Off Order)\nPackage GD 234: Procurement of Crockeries\n21,000.00 (Taka Twenty One\n(22 items).\nThousand Only)\nPROCURING ENTITY DETAILS\n25.\nName of Official Inviting Tender\nMohammad Mizanur Rahman\n26.\nDesignation of Official Inviting Tender\nDirector (Procurement), Directorate of Primary Education.\ni\n27.\nAddress of Official Inviting Tend\u00e9r\nDirectorate of Primary Education, Section-2, Mirpui, Dhaka-1216\n28.\nContact details of Official Inviting Tender\nDirector- (Procurement), Procurement Division, Directorate of\nPrimary Education, Section-2, Mirpur, Dhaka-1216\nPhone No. 02-55074915, E-mail : dirprocurementdpe@gmail.com\n29. The procuring entity reserves the right to accept or reject partial/or all tenders\n&\nMohammad Mizanur Rahman\nDirector (Procurement)\nfor\nProgram Director, PEDP4\nG-2089/09/21(14x4)\n", {"entities": [[131, 163, "inviter"], [175, 188, "inviter_location"], [459, 656, "description"], [1187, 1208, "description"]]}],
["Image No. 1318\nfile_name:/content/content/jpg_files/20210901-103358556-1017.txt\n\nGovernment of the People's Republic of Bangladesh\nOffice of the Executive Engineer\n$1,\nPWD E/M Division-3\nPurta Bhaban, Segunbagicha, Dhaka\n&\nPhone:- 9555305 Fax N6-88-02-9555305:\n,\n35\nWebsite:,www.pwd.gov. bd\ne-Tender Notice\nMemo No-E/M Division-3/380\nDate-31,08.2021\nThis is to notify all concern that the following tender is invited in the national e-GP portal:\nS.L No\nTender ID\nLast date of tender selling\nTender Opening Date\nto\n1.\n599573\n20-September,2021\n.\n21-September,2021\ni\n(Niaz Mrd. Tanvir Alam)\nExecutive Engineer\nPWD E/M Division-3,\n.\nGD-A/O-376.9.21(5x4\nDhaka.\nt\n175th\n", {"entities": [[131, 163, "co_inviter"], [168, 172, "inviter"], [201, 220, "inviter_location"]]}],
["Image No. 1162\nfile_name:/content/content/jpg_files/20210901-103358556-1018.txt\n\nGovernment of the,People's Republic of Bangladesh\nOffice of the Executive Engin\u00e9er\nPatuakhali (PWD-Division, Patyakhali.\n28\nPhone : 0441-62562. Fax No : 0441-62631.\nEmail:ee_patua@pwd:gov.bd\nMemo No: 2206\nDate: 29/08/2021\nInvitation for e-Tender\ne-Tender is invited in the -National e-GP System-Portal (http://wiww.eprocure.gov.bd for the\nprocurement of following works given bellow.\nSI. No.\nTender ID No\nInvitation\nName of Works\nTender Closing,\nReference-No.\nDate & Time.\na\nl'\n601905\nEGPAPATUA\nConstruction of 3-storied building\n22-Sep-2021\n/2021-22/FSC/01\nwith 3-storied foundation Farmers\n12:00 P.M*\nr\nService-Centre at Patuakhali sadar\nUpazilla in the district- of\nPatuakhali under the. Project\nFarmers Training at th\u00e9 \"Upazila\nLevel for Transfer of Technol\u00f2gy\n3rd Phase.\nThis is an online tender, where only e,-Tenderers will be accepted in the national e-GP portal and no\noffline./.1 hard copies will be accepted.\ny\nTo submit e-Tender; registrati\u00f3n in the national e-GP system portal http://www.eprdcure.gov.bi) is\nrequired. Futher information and guidelines are available. in the National e-GP System Pertal and\nform e-GP help desk (Email : helpdesk@eprocure.gov.bd)\n!\nof\nMd. Haroon Or. Rashid\n\"Executive Engineer\n21/5/2016 -2225/27 (9\"x8)\nPW,D Division, Patuakhali.\n", {"entities": [[190, 200, "inviter_location"], [327, 335, "call_type"], [131, 163, "co_inviter"], [176, 179, "inviter"], [576, 610, "description"]]}],
["Image No. 1080\nfile_name:/content/content/jpg_files/20210901-103358556-1019.txt\n\nGovernment of the People's Republic of Bangladesh\nBanglad\nLocal Government Engineering D'epartment\nOffice of the Upazila Engineer\nAustagram,-Kishoreg\u00f3nj.\nMUS\nwww.lged.gov.bd\n-\n28\nthes\n-\nCATH\n\"Mem\u00f3-rio: '46.02.4802.000.99.001.21.363\nDate; 31/08/2021\n3\ne-Tender Notice-No-02/2021-22.\ne-Tender . is invited in the National e-GP, system p\u00f3rt\u00e4l`. (http://www.5\neprocdre.gov.bd for the prosurement of following ID.\n.SI.\nTender\n:\nPackage\n-\nName of Work\nPublished\nTender\nTender\n(Procuiement\nNo.\nID\n5\nNo.\n-Date &\nLast\nproposal\nMethod\nTime :\nselling\n.\nOpening\nto\na,\nDate and\nDate\n)\n4\nTime\nand\nte\n4\nTime\n1\n2\n.3\n4\nis 5.\n6\nis 7 5\n8\u00b0 F\n+\n575874\ne-Tender/ -\nRepair and\n31/08/2021\n31/09/2021i\n22/09/2\nOTM\nPEDP4/K\n;\nrenovation'of T\n12.00 PM\n12.00 PM\n021\n<\nSG/AUS/\nAustagram URC\n14.00\n2020-\nUnder Upazila\nPM-\n2021\nAustogram Dist\n7\n7\n3,\n/W10.231\nTx\nKishoreganj.\nThis is an online tender, where only e-Tender will be accepted in the n\u00e1tional e-GP portal;\nand no offline/hard> copites will he accepted. To submit e-Tender, please register on e-GP-\n*\nsystem (http://www.eproeur\u00e9.gov.bd) for 'more details please contact sypport. desk, contact\nnumbers. urtherinformation and guidelines are availabl\u00e9 in the e-GP.porfal and. also from-\ne-mail : le.austragram@lged.gav.bd or c\u00e1ll to.09422-50615\nMd. Gulam Samdani\nUpazila Engineer\n'Austagram, 'Kishoreg\u00f3nj\nEmail: ue.\u00e4ustragram@lged.gov.bd\n", {"entities": [[139, 179, "inviter"], [211, 234, "inviter_location"], [332, 347, "call_type"], [180, 210, "co_inviter"]]}],
["Image No. 1193\nfile_name:/content/content/jpg_files/20210901-103358556-1020.txt\n\nGoyernment of the, People's Republic of Bangladesh\nBangladi\nLocal Government Engineering Department\nOffice of the Executive Engineer\nDistrict: Cumilla\nwww.lged.gov.bd\nCM\nJJT Gafs\"\nMemo No.,46.02.1900.000.07.002.2021-3081\n28\nDate: 31/08/2021\ne-Tender Notice\nTender Notice No: 06/2021-2022\ne-Tender is invited in the National e-GP Portal (http://www.eprocure.gov.bd) for the Procurement of Works as stated below:\nSI\nPackage No.\nName of Works\nTender/\nDocument last\nTender/`\n\"Method\nNo.\nProposal\nselling/\npropostal\nof\nID No.\ndownloading\nClosing Date\nTender\nDate and Time\nand Time\n1\nIRIDP-3/CUM/\n(a) Improvement of Khurua village -Santir\n600545\n20-09-2021\n21-09-2021\nLTM\nDW-28 .\nBazar Road via Khurua Ghat Road (Ch. 00-\n17:00 pm\n12:10 pm\n1065m (Road ID. 419904065) (b) Construction\nof 2N\u00f2s Culvert at Ch. 325m\n& 620m on the same road under Monohorgonj\nUpazila. District: Cumilla.\n2.\nIRIDP-3/CUM/\n(a) Improvement of Rammohon-Katakhola\n600546\n20-09-2021\n21-09-2021\nLTM\nDW-32\nRoad (Ch. -00-1500m (Road ID. 419095182).\n17:00 pm\n12:10'pm\n(b) Construction of (x1.50mx1.5 50m RCC Box\nCulvert at Ch. 400m on the same road (c)\nConstruction of 1 x0.625mx0.600m Culvert at-\nCh. 540m on the same road under Barura\nUpazila, District: Cumilla.\n3\nIRIDP-3/CUM/\nMaintenance of Atharobag-Gabindapur Road\n600547\n20-09-2021\n21-09-2021\nLTM\nSMW-57\n(Ch.00-1977m) (Road ID.419314099) under\n17:00 pm\n12:10 pm\nChouddagram Upazila, District: Cumilla.\n4.\ne-Tender/\nWidening of Payerkhola +High School-\n601485\n20-09-2021\n21-09-2021\nLTM\nLGEDI\nKachanla-Narankora Road from Ch. 00m-\n17:00 pm*\n12:10 pm\nCurn/GOBM/2\n1500m (Road ID. 4193 15442) under\n1-22/W-103\nChouddagram, Upazila, District: Cumilla.\n.\n5.\ne-Tender/\nPeri\u00f3dic Maintenance of Jamtoli- Nagoripara-\n601183\n20-09-2021\n21-09-2021\nLTM\nLGED/\nSeduci Road from Ch.1225m-4210m (Road ID.\n17:00 pm\n12:10 pm\nCum/GOBM/2\n419915041) Under Lalmai Upazila. District:\n1-\nCumilla.\n22/W-65\n6.\ne-Tender/\nPeriodic Maintenance of Luter char River\n601184\n20-09-2021\n21-09-2021\nLIM\nLGED/\nGhat-Viti Kandi River Ghat via, Flood Shalter\n17:00 pm\n12:10 pm\nCum/GOBM/2\nRoad from Ch.2500m-3500m (Road\n1-22/W-66\nID.419884021) Under Meghna Upazila,\nDistrict: Cumilla.\n7.\ne-Tender/\nPeriodic\" Maintenance of Lolbaria RHD-\n601185\n20-09-2021\n21-09-2021\nLTM\nLGED/\nDakshin Bijoypur GC *Road from Ch.00m-\n17:00 pm\n12:10 pm\nCum/\n2106m Road_ID.419912015) Under Sadar\nGOBM/21-\nDakshin-Upazila. District: Cumilia.\n22/W-67\n8.\ne-Tender/\nPeriodic Maintenance of Mathabanga UPC\n601186\n20-09-2021\n21-09-2021\nLTM\nLGED/Cum/G\n(Cenalya bridge RHD)- Monipur bazar via\n17:00 pm\n12:10 pm\nOBM/21-22/\nGharmora UPC' Road from Ch.00m-1655m\nW-68\n(Road ID.419543003) Under Homna Upazila:\nDistrict: Cumilla.\n9.\ne-Tender/\nPeriodic Maintenance of Gorashal-Purba\n601187\n20-09-2021\n21-09-2021\nLIM\nLGED/\nSunaullah Road from Ch.80m-1200m (Road\n17:00,pm\n12:10 pm\nCum/GOBM/2\nID.419814006) Under Muradnagar Upazila,\n1-22/W-69\nDistrict: Cumilla.\ne\n10.\ne-Tender\nPeriodic Maintenance of Mokam UP Office-\n601188\n20-09-2021\n21-09-2021-\nLTM\n/LGED/\nPachkitta Bazar Road from Ch.500m-2300m\n17:00 pm\n12:10 pm\nCum/GOBM/2\n(Road ID.419183008) Under Burichong\na\n1-22/W-70\nUpazila, District: Cumilla.\n11.\ne-Tender/\nPeriodic Maintenance of Laximpur Bazar\n601189\n, 20-09-2021\n21-09-2021\nLTM\nLGED/\n(Dhighirpar)-Kailain.Bazar Rd. Via Joag UPC\n17:00 pm\n12:10 pm\nCum/GOBM/2\nRoad from Ch.00m-1000m (Road ID.4\n1-22/W-71\n19273013) Under Chandina Upazila, District:\nCumilla.\n12\ne-Tender/\nRehabilitation of Krishnapur R&H-Amdour\n601190\n20-09-2021\n21-09-2021\nLTM\nLGED/\nR&11 Road from Ch.00m-1290m (Road ID.4\n17:00 pm\n12:10'pm\nCum/GOBM/2\n19724008) Under Laksham Upazila, District:\n1-22/W-72\nCumilla.\n13.\ne-Tender/\nWidening of Lakkhanpur Bazar - Compani\n601191\n20-09-2021\n21-09-2021\nLIM\nLGED/\nBazar Road via Sorospur UP Road at\n17:00 pm\n12:10 pm\n-\n\"Cum/GOBM/2\nCh.5000m-605Om (Road ID.419903002)\n1-22/W-73\nUnder-Monogorgang Upazila. District: Cumilla.\n14.\ne-Tender/,\nEmargency Maintenance of Bizra RHD--\n601192\n20-09-2021\n21-09-2021\nLIM\n\"LGED/\nLaksham Upazilla HQ Road from Ch.00m-\n17:00 pm\n12:10.pm\nCum/GOBM/2\n8855m (Road ID.419722004) Under Laksham\n1-22/W-74\n.Upazila. District:-Cumilla.\nThis is an online tender, where only e-Tender will be accepted in the National e-GP portal and no offline/hard copies will be accepted.\nTo submit e-Tender, registration in the National e-GP Portal (http://www.eprocure.gov.bd) is required.\nThe fees for downloading the e-Tender Documents from the National e-GP Portal have to be deposited through online at any Branches of\nregistered Bank.\n:\nFurther information and guidelines are available in the National e.GP System Portal and from e-GP help desk (helpdesk@eprocure.gov.bd)\nThe Tender Notice will be available on the,website: www.lged.gov.bd.\nMirza Md. Iftekhar Ali\nExecutive Engineer\nLGED, Cumilla\nPhone: 081-68960'\nE-mail: xen.cumilla@lged.gov.bd\n", {"entities": [[141, 180, "inviter"], [181, 213, "co_inviter"], [224, 231, "inviter_location"], [322, 337, "call_type"], [819, 873, "description"], [1113, 1160, "description"], [1321, 1361, "description"]]}],
["Image No. 1374\nfile_name:/content/content/jpg_files/20210901-103358556-1021.txt\n\nCECH 6214, CHICK 6014\nGovernment of The People's Republic of Bangladesh\nDirectorate General of Family Planning\n06, \"Kawran Bazar, Dhaka-1215\n60\nInternational Tender Invitation for Tenders (Goods)\n1\nMinistry/Division\nMedical Education and Family Welfare Division\n2.\nAgency\nDirectorate General of Family Planning\n3\nProcuring,Entity Name\nDirector (Logistic & Supply) and\nLine Director (Procurement, Storage and Supply Management)\n4\nProcuring Entity Code\nN/A-\n5\nProcuring Entity District\nDhaka\nby\nInvitation for\nProcurement of Hydrocolloid Dressing Materials\n7.1\nInvitation: Ref No\nDGFP/L&S-3/CC/GD-16/2024-22/37\n8\nDate\nDate: 31/08/2021\nKEY INFORMATION\n9\nProcurement Method\nOpen .Tender Method (OTM) (ICT)\nFUNDING INFORMATION\n10\nBudget and Source of Funds\nRPA (GoB)\n11\nDevelopment Partners (if\nInternational D\u00e9velopment Association (IDA)\napplicable)\nPARTICULAR INFORMATION\n12\nProject / Programme Code'\n224108500\n(if applicable)\n13\nProject? Programme Name\n4th Health, Population & Nutrition Sector Programm\u00e9 (4th HPNSP)\n(if applicable)\n14\n-Tender Package No.\nCC/GD-16\n15\nTender Package Name\nProcurement of Hydrocolloid-Dressing Materials\n16\nTender Publication Date\n01/09/2021\n17\nTender Last Selling Date\n17/10/2021\nDate\nTime\n18\nTender Closing Date and Time\n18/10/2021\n.12:00 PM\n19\nTender Opening Date and Time\n18/10/2021\n12:30 PM\n20\nName & Address of- the office(s)\nAddress\nSelling,Tender Document\nOffice of the Director (Logistic & Supply): and Lin\u00e9 Director\n(Principal)\n(Precurement, Storage and Supply Management), Directorate General\nof Family Planning, 06, Kawran Bazar, Dhaka-1215\nContact No.: 01741404804\nSelling Tender Document\nOffice of the Additional Director\n(Others)\nCentral Warehouse, Directorate Gen\u00e9ral of Family Planning,\nNandipara (Uttargaon) Trimohoni, Union-Nasirabad, Thana-Khilgaon,\nDhaka, Bangladesh.\nReceiving Tender Document\nOffice of the Director (Logistic & Supply) and Line Director\n(Procurement, Storage and Supply Manag\u00e9ment), Directorate General-\nof Family Planning, 06,-Kawran-Bazar;-Dhaka-1215\nOpening Tender Document\nOffice' of the Director (L'ogistic & Supply) and Line Director\n(Procurement, Storag\u00e9 and Supply Management), Directorate General\nof Family Planning, 06, Kawran Bazar, Dhaka-1215.\n32\nPlac\u00e9 / .Date / Time of\nDate\nTime\nPre-Tender Meeting (Optional)\n14/09/2021\n11:00 AM\nThe Pre-Tender Meeting shall take\nConference Room of the Director (Logistic & Supply) and Line\nplac\u00e9 at\nDirector (Procurement, Storage and Supply Management), Logistics\n& Supply Unit (5th Floor, Room No. 15), Directorate General of\nFamily Planning, 06, Kawran Bazar, Dhaka-121\nINFORMATION FOR TENDERER\n22:\nBrief Eligibility\nEligibility As stated in Tender Document\n23.\n*Brief Descripti\u00f3n of G\u00f3ods\nHydrocolloid Dressing-M\u00e1tefials & Dressing Dr\u00fams\n24\nBrief Description of Related\nN/A\nServices\n25\nPrice of Tender Document (Tk)\nTaka' 1'8000/- (Eight Thousand) 'only, non-refundable. Mode of\npayment should be Treasury Challan- from Bangladesh Bank/Sonali\nBank Limited, Code No. 1-7481-0000-2366 in favour of: Director\n50\nGeneral, Directorate General of Family Planning, 06, Kawran Bazar,\nDhaka-1215. Original copy of the Treasury Challan must be certified\n(CTR)/Verified through Online Verification System by the concerned\nAccounts Office.\nLot\nIdentification of Lot\n.Location\nTender\nCompletion\nNo\nDescription of\nUnit of\nQuantity\nSecurity\nTime in\nt\nItem\nMeasurement\n-\nAmount (Tk)\nWeeks /\nMonth's\n26\nSingle\nHydrocolloid\nPcs\n100000\nCentral\nTk.\nDelivery of the\nDressing Materials\nWarehouse\n8,00,000.00\nentire quantity\n(10 cm x 10 cm)\n(Rented).\nor equivalent\nof goods shall\nDirectorate'\nUSD\nhave to' be\nGeneral of\nHydrocolloid\n-Pcs\n600000\n9,412.00\ncompleted\nFamily Planning,\nwithin\n16\nDressing Materials\nNandipara\nweeks latest\n(10 cm x 5 cm)\n(Uttargaon),\nfrom the date\nTrimohoni, Union:\nof opening of\nDr\u00e9ssing Drum\nPcs\n500\nNasirabad, Thana:\nthe Letter Fof\nKhilgaon, Dhaka.\n:Credit,\n27\nName of Official Inviting Tender\nRatna Talukder\n28\nDesignation of Official Inviting\nDirector (Logistic & Supply) and\nTender\nLine Director (Procurement, Storage and Supply Management)\n29\nAddress of Official Inviting Tender\nLogistics & Supply Unit, Directorate General of Family Planning, 06,\nKawran Bazar, Dhaka-1215\n30\nContact details of Official Inviting\nPhone:\nFax:\n88-02-55012349\nemail: dirlsdgfp@@mail.com\nTender\n88-02-55012349\n31\nThe Procuring Entity reserves the right to reject all Tenders and/or annul the Tender proceedings without\nassigning any reasons whatsoever.\n(05/01/08/2021\nRatna Talukder\nDirector (Logistic & Supply) and\nLine Director (PS&SM)\nDirectorate General of Family Planning,\nGD-1069/21- x 4)\n06, Kawran Bazar, Dhaka-1215\n", {"entities": [[153, 191, "inviter"], [574, 635, "description"], [197, 216, "inviter_location"], [225, 245, "international"]]}],
["Image No. 1222\nfile_name:/content/content/jpg_files/20210901-104224763-1022.txt\n\nGovernment of the Peoples, Republic of Bangladesh'\nOffice of the Executive Engineer (RHD)\nRoad Division, Bhola\nmonth 925\nE-mail 1:eebho@rhd:gov.bd\n7564 455 B 5-7-2015\nMemo No. 35.01.0918.404.34.001.20-1756\nDate : 31/08/2021\nInvitation for e-Tender (OTM)\ne-Tender is invited in the national e-GP system portal for the\nfollowing works :\nTender ID\n601218\nName of Tender\nRepairing with Seal Coat of existing\nPavement including Palisiding work at\n14\n4th(P), 5th(P) & 9th(P) km of Debirchar-\n(Nazirpur-Lalmohon-Mongolsikder\nTazumuddin* Road (Z-8913) under Bhola\nRoad Division during the year 2021-2022\n(Package. No-- 06/e-GP/Bho/Min/2021-22)\nTender Last Selling\n14-Sep-2021 17:00\nDate and Time\nTender.Closing &\n15-Sep-2021 12:00\nOpening Date, and Time\nThis is an online' tender, where only e-tender will be accepted in e-\nGP portal and no offline/hard copy will be accepted. To submit\ne-tender,\nplease\nregister\non\ne-GP\nsystem.\n(http://www.eprocure.gov.bd). For more details please contact e-gp\nhelp desk (helpdesk@eprocure.gov.bd)\nSd./-\n(Md. Nazmul Islam)\nID No-602232\nExecutive Engineer (C.C.), RHD\nGC-2088/09/2116'x3)\nRoad Division, Bhola.\n", {"entities": [[132, 164, "co_inviter"], [166, 169, "inviter"], [186, 191, "inviter_location"], [335, 343, "call_type"], [448, 519, "description"]]}],
["Image No. 1092\nfile_name:/content/content/jpg_files/20210901-104241152-1023.txt\n\nMadhabdi Pourashava\nthe -\n602\nDistrict : Narsingdi.\n5965\n62314\ncare 2/19/33\nMamo No:Ma:pou:/Engg.Dept/IFT-02/2021-2022/48/50\nDate : 01-09-2021\nGP Tender Notice\n(IFT NO # 02&03/2021-2022)\n-Tender\nis\nipvited in, GP System Portal (http://www.\u00e9procure.ov.bd for the procurement\nof works Tender ID No. 599793 [Package No. MDI/ADP/2021=22/W=01]-& Tender ID No. 599796\n[Package No. MDIADP/2021-22(W-02]\nTender ID, Package No. &\nName of work / Package\nLast Selling & Closing Date\nPublishing Date\nDescription\nTender ID : 599793\n(1) MW-01 : Maintenance of\n15-September-2021; 16:58\nPackage No : MDI/ADP/2021-\nMadh\u00e1bdi Poura Office Bhavan\n16-September-2021; 15:01\n22/W-01 & 01-September-2021\nUnder Madhabdi Pourashava,\nDist : Narsingdi.\n(2) FW-01 Supplyng, fitting and\nfixing of Swivel Chairs,\n:\nExecutive Table and Fixed\nDrawer at Office of Councilors in\nMadhabdi Pourashava, Dist:\nNarsingdi.\nTender ID : 599796\n(1) PT-01: Re-Construction of\n15-September-2021 ; 16:58\nPackage No : MDI/ADP/2021-\ndamaged Manhole Pit slabs and\n16-September-2021 ; 15:30\n22/W-02 & 01-September-2021\nRCC slabs on existing U-drain at\ndifferent places of different\nWards\nunder\nMadhabdi\nPourashava, Dist : Narsingdi.\nThis\nis\nan\nonline\nTender, Where only e-Tender will be accepted in the National e-GP portal and no\noffline /hard copies will be accepted.\nTo submit e-Tender, registration in the National e-GP System Portal (http;//www.eprocure.gov.bd) is\nrequired.\nThe fees for downloading the e-Tender Documents from the National e-GP System portal have to be\ndeposited online through any registered Bank's branches up to 16:58 PM on dated 15-September-2021\nFurther information and guidelines are available in the National e-GP System portal and e-GP help desk\n(helpdesk@eprocure.gov.bd.)\n11\nSd./-\nMd. Moniruzzaman\nExecutive Engineer\nGC-2087/09/21 (6'x3)\nMadhabdi Pourashava, Narsingdi.\n", {"entities": [[90, 100, "inviter"], [122, 131, "inviter_location"], [224, 240, "call_type"], [612, 707, "description"], [816, 897, "description"], [993, 1179, "description"]]}],
["Image No. 1179\nfile_name:/content/content/jpg_files/20210901-104345038-1024.txt\n\nGovernment of The People's Republic of Bangladesh\n62/45, 6214\nDirectorate General of Family Planning\n3/3/-13 20618\"\n06, Kawran Bazar, Dhaka-1215\nMemo No. DGFP/L&S-3/CC/GD-11/2021-22/39/16\nC.Date: 31/08/2022\nInvitation for Tenders (Goods)\n1\nMinistry/Division\nMedical Education and Family Welfare Division\n2 Agency\nDirectorate General of Family Planning\n3\nProcuring Entity Name\nDirector (Logistic & Supply) and Line Director (Procurement,\nStorage and Supply Management)\n4\nProcuring Entity Code\nN/A\n5\nProcuring Entity District\nDhaka\n6\nInvitation for\nProcurement of 10 Items Medicine for CCSDP\n7.\nInvitation Ref. No.\nDGFP/L&S-3/CC/GD-11/2021-22/39/16\n8\nDate\n31/08/2021\nKEY INFORMATION\n9 Procurement Method\nOpen Tendering Method (NCT)\nFUNDING INFORMATION\n32\n10 Budget and Source of Funds,\nRPA(GoB)\n11\nDevelopment Partners (if applicable)\nN/A\nPARTICULAR INFORMATION\n12 Project / Programme Code (if applicable) 224108500\n13\nProject / Programme Name (if applicable)\n4th HPNSP\n14\nTender Package No.\nCC/GD-11\n15\nTender Package Name\nProcurement of 10 Items Medicine for CCSDP\n16\nTender Publication Date\n01/09/2021\n1-7\nTender Last Selling Date\n21/09/2021\n*\nDate\nTime\nthese\n18\nTender Closing Date and Time\n22/09/2021\n12:00 PM\n19\nTender Opening Date and Time\n(22/09/2021\n12:30 PM\n20\nName & Address of the office(s)\nAddress:\n-\nSelling\nTender\nDocument\nOffice, of the Director (Logistic & Supply) and Line Director,\n(Principal)\n(Procurement, Storage and Supply Management),\nDirectorate General of Family Planning,\n06, Kawran Bazar, Dhaka-1215.\nTelephone No.: 02-55012349\nFax No.: 02-55012349\nE-mail address dirlsdgfp@gmail.com\nContact No.: 01741404804\nSelling Tender Document (Others)\nOffice of the Additional Director\nCentral Warehouse, Directorate General of Family Planning\nNandipara (Uttargaon) Trimohoni, Union-Nasirabad, Thana-\nKhilgaon, Dhaka, Bangladesh.\nNO CONDITIONS APPLY FOR SALE, P\u00dcRCHASE OR DISTRIBUTION OF TENDER DOCUMENTS\nReceiving Tender Document\nConference Room, Room No.-15, Logistics & Supply Unit, Directorate\nGeneral of Family Planning, 06, Karwan Bazar, Dhaka-1215\nOpening Tender Document\nConference, Room, Room No.-15, Logistics*& Supply Unit, Directorate\nGeneral of Family Planning, 06, Karwan Bazar, Dhaka-1215\n21\nPlace / Date / Time of Pre-Tender Meeting (Optional)\nDate\nTime\n08/09/2021\n11:00 AM\nThe Pre-Tender Meeting shall take Conference Room, Room No.-15, Logistics & Supply Unit.\nplace at\nDirectorate General- of Family Planning, 06, Karwan Bazar, Dhaka-\n1215\nINFORMATION FOR TENDERER\n22 Brief Eligibility\nEligibility As stated in Tender Document\n23 Brief Description of Goods\nProcurement of 10 Items Medicine for CCSDP\n24 Brief Description of Related Services\nN/A\n25\nPrice of Tender Document (Tk)\nTaka 2000/- (Two Thousand) only, non-refundable. Mode of payment\nshould be Treasury, Challan from Bangladesh Bank/Sonali Bank\nLimited, Code No. 1-7481-0000-2366 in favour of : Director\nGeneral, Dir\u00e9ctorate General of Family Planning, 06, Kawran Bazar,\nDhaka-1215. Original copy of the Treasury Challan-mus be certified\n(CTR)/Verified through Online Verification System by the concerned\nAccounts Office.\nLot\nIdentification\nLocation\nTender Security\nCompletion Time in\nNo.\nof Lot\nAmount (Tk)\nWeeks / Months\n26\nSingle\nProcurement\nCentral Warehouse (Rented),\nTk. 8,60,000.00 (Eight\n10 (Ten) weeks from the\nof 10 Items\nDirectorate General of Family\nLac and Sixty\ndate of Signing\nMedicine for\nPlanning, Nandipara (Uttargaon)\nThousand)\nContract.\nCCSDP\nTrimohoni, Union: Nasirabad,\nThana: Khilgaon, Dhaka,\nBangladesh.\n27\nName of Official \"Inviting Tender\nRatna Talukder\n28\nDesignation of Official Inviting Director (Logistic & Supply) and Line Director (Procurement,\nTender\nStorage and Supply Management)\n29\nAddress of Official Inviting Tender\nLogistics & Supply Unit, Directorate General of Family Planing\n06, Kawran Bazar, Dhaka-1215\n30\nContact details of Official Inviting Tender 88-02-55012349 88-02-55012349 dirlsdgfp@gmail:com\n31\nThe Procuring Entity reserves the right to reject all Tenders and/or annul the Tender proceedings without\nassigning any reasons whatsoever.\n31/08/2021\nRatna Talukder\n52\nDirector (Logistic & Supply) and\nLine Director (PSSM-FP)\nDirectorate General of Family Planning,\n06, Kawran Bazar, Dhaka-1215\n", {"entities": [[143, 181, "inviter"], [201, 220, "inviter_location"], [613, 660, "description"]]}],
["Image No. 1024\nfile_name:/content/content/jpg_files/20210901-104835726-1030.txt\n\nBEPZA\nGovernment of the People's Republic of Bangladesh\nPrime Minister's Office\nBangladesh-Export Processing Zones Authority\n37\nCh\u00e1ttogram Export Processing Zone\nSouth Halishahar, Chattogram.\n(www.bepza.gov.bd)\nNo. 03:06.1535.324.18.976.21-08\nDate: 19 August 2021\nInvitation for Tender (Frame work contract for goods)\nNo.01/2021-2022\nSealed Tenders are hereby invited from the eligible supplier / contractor for the\nfollowing frame work contract as per terms & conditions stated below:\n1\nMinistry/Division\nPrime Minister's Office\n2\nAgency\nB\u00e4ngladesh Export Processing Zones Authority (BEPZA)\n3\nProcuring Entity name\nOffice of the Superintending Engineer, Chattogram EPZ\n4\nProcuring Entity district\nChattogram.\n5\nInvitation for\nProcurement of frame work contract for goods\n6\nInvitation Ref. no and date\n03.06 1535.324.18.976.21, Date: 19 August 2021\nKEY INFORMATION\n7\nProcurement Type\nNCT\n8\nProcurement method\nOpen Tendering Method (OTM)\nFUNDING INFORMATION\n9\nBudget and source of funds\nSelf finance of BEPZA\n10\nDevelopment Partners (if applicable)\nNot applicable\nPARTICULAR INFORMATION\n11\nProject/Programmed Code (if applicable)\nNot applicable\n12\nProject /Programmed Name (if applicable)\nNot applicable\n13\nTender Package No.\nNot applicable\n14\nTender Package Name\nNot applicable\n15\nTender Publi cation Date\nQn.or before 05 September 2021\n16\nTender Last Selling Date\n20 September 2021 during office hour\n17\nTender Submission Date & Time\n21 September 2021 up to 12:00 PM\n18\nTender Opening Date & Time\n21 September 2021 at 2:00 PM\n19\nName & Address of the office(s)\nSelling of Tender Document (principal)\nAccounts section, Chattogram EPZ, South Halishahar, Chattogram,\nSelling of Tender Document (others)\n1 Accounts section, Kamaphuli EPZ, North Pat\u00e9nga, Chattogram.\n2 Divisional Commissioner's office, Court Building, Chattogram.\nReceiving of Tender Document\n1. Office of the Superintending Engineer, Chattogram EPZ, South\nHalishahar, Chattogram.\n2. Divisional Commissioner's office, Court Building, Chattogram.\nOpening of Tender Document\nOffice of the Superintending Engineer, Chattogram EPZ, South Halishahar,\nChattogram.\n20-\nPlace/Date/Time of Pre-Tender meeting\nNot applicable 52ml\n\"INFORMATION FOR Tenderer\n21\nEligibility and Qualification of Tenderer\nThe-tender is open pallejigible tenderers who satisfy the following:\na) The minimum years of gener\u00e1l experience in the supply of goods and\nrelated services as supplier/Contracto shall be 05 (five) years.\nb) The minimum specific rexperience as supplier/Contractor in supply of\nsimilar goods/frame work contract for Civil Engineering goods in a single\ncontract amounting not less than BOT 13.00 Lakh during last (five) years\nin Govt L/Semi-Govt./Autonomous Bodies. The work completion certificates\nare to be issued by an Officer not below the rank of Executive Engineer or\nequivalent Officer of concerned Department/ Organization.\nc) The minimum amount of liquid assets or working capital or credit facilities\nof the Tenderer shall be BDT 3.00 Lakh.\nd) Debarred firms from any Sovt/Semi-Govt/Autonomous bodies are to be\ntreated as ineligible to participate in the Tender.\ne) Up-to-date trade license, Income Tax clearance certificate, VAT\nregistration certificate etc. to be submitted.\nf) Other required conditions and documents of the tenderer \u00e1re shown in\nTendet Data Sheet (TDS) of bidding documents must be fulfilled.\n22\nBrief description of related works\nCement, Sand, Stone chips, Wood, Paints, Tiles & Thai Aluminium\nkey/lock, Sanitary goods etc.\n23\nPrice of Tender Documents\nTK. 1000.00 (Taka One Thousand) only.\n24\nSI.\nIdentification of lot\nLocation\nTender Security in Pay\nCompletion\nno\nOrder (In favour of\nTim\u00e9\nBEPZA)\n(i)\nFrame work. contract for supply of\nChattogram EPZ, South\nTk. 70,000.00\n365 days\nmaterials for repair & maintenance of\nHalishahar, Chattogram.\nCivil Engineefing\nworks\nat\nChattogram EPZ.\nPROCURING ENTITY DETAILS\n25\nName of Official Inviting Tender\nMd. Mizanui Rahman\n26\nDesignation of Official Inviting Tender\nSuperintending Engineer, Chattogram EPZ\nAddress of official inviting Tender\nChattogram EPZ, South Halishahar, Chattogram.\nContract details of Official Inviting Tender\nTel: 02333341441: Email: mizan10640@gmail.com.\n29\nSpecial Instruction\n(a) Eligible supplerscontractor/authorzed representative may purchase tender documents from the offices mentioned in\nis\nSI- 19 on submitting written request in their letter Head pad.\n(b) Any false, f\u00f3rged and miss leading documents furnished by Tenderer shall result in rejection of the Tender.\n(c) Incase of any confusion arises due to printing mistake or other reason, the Public Procurement Rules-2008 and its\namendments will govern.\n(d) Tender should be submitted original along with copy ard it should be mentioned on the cover sheet.\n(e) All papers of submitted tender documents including bidder's profile must be signed by the contractor or its authorized\nrepresentative.\n(f) The tender shall be valid for a period of 120 (One H\u00fcndred Twenty) days from the date of Submission the Tender. The\ntender notice is also available at web site: www bepza. gov bd of BEPZA.\n30\nThe procuring entity reserves the to accept any or reject all the Tenders or annul the bidding Proceedings.\n(Md. Mizanur Rahman)\nSuperintending Engineer\n37\nChattogram EPZ\nTel: 02333341441\nEmail: nizan10640@gmail.com\n(20\"xb)\n", {"entities": [[161, 205, "inviter"], [209, 242, "co_inviter"], [243, 272, "inviter_location"], [345, 398, "description"]]}],
["Image No. 1303\nfile_name:/content/content/jpg_files/20210901-104945941-1031.txt\n\nOffice of the Project Director &\nMechanical-Marine Building\nChief Engineer (Marine)\nTelephone +8802477753801\nProject- Procurement of Service\nEmail\ncem@mpa.gov.bc\nVessels for Mongla Port (PSVMP)\nWeb\n:\nwww.mpa.gov.bd\nMongla Port Authority\nMongla, Bagerhat-9351\nAPORT\nMongla Port `Authority, Mongla, Bagerhat.\n24\nInvitation for Tender (Single Lot)\n1.\nMinistry/Division\nMinistry of Shipping\n2.\nAgency\nMongla Port Authority\n3\nProcuring Entity Name\nChairman, Mongla Port Authority represented by\nProject Director\n4\nProcuring Entity Code\n5205\n5\nProcuring Entity District\nBagerhat\n6\nInvitation For\nGoods (Procurement of One in Number Survey and\nR\u00e9search Vessel Including\"Supply of Necessary\nEquipment, Accessories and Related Services.)\n7\nInvitation Ref no.\n18. .14.0158.330.014.020.21\n8\nDate\n29/08/2021\nKEY INFORMATION\n9\nProcurement Method\nOpen Tendering Method (International)\nOne Stage & Two Envelope Tendering Method\nFUNDING INFORMATION\n10\nBudget and Source of Funds\nGovernment of Bangladesh (GoB)\nPARTICULAR INFORMATION\n11\nProject Name\nProcurement of Service Vessels for Mongla Port.\n12\nTender Package Name\nProcurement of One in Number Survey and Research\nVessel Including Supply of Necessary Equipment,\nAccessories and Related Services.\n13\nTender Package No.\n1 (One) number\n14\nTender Publication Date\n01/09/2021\n15\nTender Last Selling Date\n10/10/2021\n16\nTender Closing Date and\n14/10/2021 12:00 PM\nTime\n17\nTender Opening Date and\n14/10/2021 12:301 PM\nTime\n18\nName and address of the\nOffice of the Chief Engineer (Marine), Mechanical-\nOffice\nMarine Building. 1st Floor, Mongla Port Authority,\n-Selling Tender Documents\nMongla, Bagerhat, Bangladesh\n-Receiving Tender\nDocuments\n19\nPlace/Date/Time of Pre-\nOffice of the Chief Engirieer (Marine), Mechanical-\nTender Meeting\nMarine Building, 1st Floor, Mongla Port Authority,\nMongla, Bagerhat, Bangladesh Date: 20/09/2021\nTime: AM\n20\nPlace of Submission Tender\nOffice of the Chief Engineer (Marine), Mechanical-\nMarine Building. Mongla Port Authority,\nMongla, Bagerhat, Bangladesh\nINFORMATION FOR TENDERER\n21\nEligibility of Tenderer\nThe invitation for tender is open to all eligible\ntenderers-as mentioned below-\ni) The maximum 3 (three) number of arbitrations\nag\u00e4inst the Tenderer over a period of 5 (five)\nyears\nii) The Tenderer .shall have a minimum 20\n(Twenty) years of overall experience in the\nshipbuilding. Years counting backward from the\ndate of publication of IFT in the newspaper.\niii) The minimum specific experience of\n(Two) nos. coastal going Survey Vessels/\nResearch Vessels/ Survey & Research Vessels\n(greater than 24.00 m) under International\nClassification Society (member of IACS) within\nthe'last 5 (Five) years.\niv) Single contract value of self-propelled class\nvessel should be at least BDT 908.00 million or,\nUSD 10.87 million within the last 5.(Five)\nyears.\nv)\nThe offered survey & research vessel shall be\nbuilt on proven design. \"Related end user list\nwith IMO number/ buyer certificate of the\nSurvey Vessel/ Research Vessel/ Survey &\nResearch Vessel already built on the offered\nProven Design must be submitted with tender\nvi) Supporting documents shall have to be\nsubmitted by the tenderer, without which the\ntender shall be rejected. Complete and authentic\ndocuments must be submitted, failing which the\ntender shall be considered as non-responsive.\nvii) The minimum amount of liquid asset or working\ncapital substantiated by audit report or credit\nfacility substantiate (in the format as specified\n(Form PG4-6A) of the Tenderer shall be BDT\n1248.00 million or USD 14.95 million.\nviii) If the Tenderer submits credit facility, the\ndecision of Board of Directors of the bank\nregarding allocation of CC limit h\u00e1s to be\nsubmitted with the offer\nix) The amount of the Tender Security shall be\nBDT 22.70 million only in favour of Project\n24\nDirector, Mongla Port Authority in the form of\nPay-order /Bank Guarantee from any scheduled\nbank of Bangladesh\nx)\nIn case of foreign bidder, Tender Security shall\nbe USD 0.27 million in favour of Project\nDirector, Mongla Port Authority and in form of\nBank Guarantee issued by an internationally\nreputable bank and be endorsed by any\ncorrespondent bank located in Bangladesh, to\nmake it enforceable.\n22\nBrief Description of Goods\nProcurement of One in Number Survey and Research\nor Works\nVessel Including Supply of Necessary Equipment,\nAccessories and Related Services.\n23\nBrief Description of Related\nTesting, Commissioning, Training, Repair,\nServices\nMaintenance and 01 (One) year warranty period.\n24\nTender Document Price\nBDT- 15000.00 (Fifteen Thousand Only) (Non-\nrefundable)\nPROCURING ENTITY DETAILS\n25\nName of Official Inviting\nCommander A F-M Zahidur Rahman, (E), BN.\nTender\n26\nDesignation of Official\nProject Director, Procurement Of Service Vessel For\nInviting Tender\nMongla Port Project, Mongla Port Authority,\nMongla, Bagerhat\n27\nAddress of Official Inviting\nOffice of the Chief Engin\u00e9er (Marine), Mechanical-\nTender\nMarine Building 1st Floor, Mongla Port Authority,\nMongla, Bagerhat, Bangladesh\n28\nContact details of Official\nPhone: +8802477753801 ; Email: cem@mpa. gov.bd\nInviting Tender\n29\nThe procuring entity reserves the right to accept or reject all applications\n30\nThis tender will be guided in accordance with \"The Public Procurement Act-2006\nand Rules-2008\"\nA'F M Zahidur Rahman\nCommander, (E), BN\no-18.14.0158.132.00.040-1686\nProject Director &\nChi\u00e9f Engineer (Marine)\nMongla Port Authority,\nDC-735/21 (19\"x3)\nMongla, Bagerhat.\n", {"entities": [[296, 317, "inviter"], [318, 334, "inviter_location"], [656, 807, "description"]]}],
["Image No. 1214\nfile_name:/content/content/jpg_files/20210901-104945941-1032.txt\n\nOffice of the Project Director & Chief\nEngineer (Marine)\nMechanical-Marine Building\nProject-Procurement of Service Vessels\nTelephone +8802477753801\nEmail\nfor Mongla Port (PSVMP)\n: cem@mpa.gov.bd\nWeb\nMongla Port Authority\n: www.mpa.gov.bd\nMongla, B\u00e0gerhat-9351\n1\nMongla Port Authority, Mongla, Bagerhat.\nInvitation for Tender (Single Lot)\n1 Ministry/Division\nMinistry of Shipping\nAgency\nMongla Port Authority\nProcuring Entity Name\nChairman, Mongla Port Authority represented by\nProject Director\n4 Procuring Entity Code\n5205\n5 Procuring Entity District\nBagerhat\n34\n6 Invitation For,\nGoods (Procurement of One in Number Buoy Laying\nVessel Including Supply of Necessary Equipment,\nAccessories and Related Services.)\n7 Invitation Ref. no.\n8.14.0158.330.014.019.21\n8 Date\n29/08/2021\nKEY INFORMATION\n9 Procurement Method\nOpen Tendering Method (International)\nOne Stage & Two Envelope Tendering Method\nFUNDING INFORMATION\n10 Budget and Source of Funds\nGovernment of Bangladesh (GoB)\nPARTICULAR INFORMATION\n11 Project Name\nProcurement of Service Vessels for Mongla Porti\nTender Package Name\nProcurement of One in Number Buoy Laying Vessel\nIncluding Supply of Necessary Equipment, Accessories\nand Related Services.\n13 Tender Package No.\n1 (One) number\nTender Publication Date\n01/09/2021\n15 Tender Last Selling Date\n10/10/2021\n16 Tender Closing Date and Time\n17/10/2021 12:00 PM\n17 Tender Opening Date and Time\n17/10/2021 12:30 PM\n18 Name and address of the Office\n-Selling Tender Documents\nOffice of the Chief Engineer (Marine), Mechanical-\nReceiving Tender Documents\nMarine Building, 1st Floor, Mongla Port Authority,\nMongla, Bagerhat, Bangladesh.\n19 Place/Date/Time of Pre-Tender\nOffice of the Chief Engineer (Marine), Mechanical-\nMeeting\nMarine Building, lst Floor, Mongla Port Authority,\nMongla, Bagerhat, Bangladesh.\nDate: 23/09/2021 Time: 11:00 AM\n20 Place of Submission Tender\nOffice of the Chief Engineer (Marine), Mechanical-\nMarine Building, 1st Floor, Mongla Port Authority,\nMongla, Bagerhat, Bangladesh\nINFORMATION FOR TENDERER\n21 Eligibility of Tenderer\nThe invitation for tender is open to all eligible tenderers\nas mentioned below-\ni)\nThe maximum 3 (three) number of arbitrations\nagainst the Tenderer over a period of 5 (five) years.\nii) The Tenderer shall have a minimum 20 (Twenty)\nyears of overall experience in the shipbuilding.\nYears counting backward from the date\nof\npublication of IFT in the newspaper.\niii) The minimum specific experience of successfully\ncompletion as a builder of at least 2 (Two) Nos.\nsimilar type (coastal class, length > 40 m) of Buoy\nLaying Vessel within the last 5 (Five) years under\nsupervision-o Unternational-Classification-Sodiety\n(member of IACS).\niv) Single contract value of self-propelled class vessel\nshould be at least BDT 1012.00 million or USD\n12.15 million within the last 5 (Five) years.\nv) The offered buoy laying vessel shall be built on\nproven design. Related end user list with IMO\nnumber of the built buoy laying vessel must bei\nsubmitted with tender.\nvi) Supporting documents shall have to be submitted\nby the tenderer, without which the tender shall bei\nrejected.\nComplete and authentic documents must be\nsubmitted, failing which the tender shall be\nconsidered as non-responsive.\nvii) The minimum amount of liquid asset or working\ncapital substantiated by audit report or credit\nfacility substantiated in the format as specified\n(Form PG4-6A) `of the Tenderer shall be BDT\n1390.00 million or USD 16.65 million.\nviii) If the Tenderer submits credit facility, the decision\nof Board of Directors of the bank regarding\nallocation of CC limit has to be submitted with the\noffer.\nix) The amount of the Tender Security shall be BDT\n34\n25.30 million only in favour of Project Director,\nMohgla Port Authority in the form of Pay-\norder/Bank Guarantee from any scheduled bank of\nBangladesh.\nx)\nIn case of foreign bidder, Tender Security shall be\nUSD 0.31 million in favour of Project Director,\nMongla Port Authority and in form of Bank\nGuarantee issued by an internationally reputable\nbank and be endorsed by any correspondent bank\nlocated in Bangladesh, to make it enforceable.\n22 Brief Description of Goods or\nProcurement of One in Number Buoy Laying Vessel\nWorks\nIncluding Supply of Necessary Equipment, Accessories\nland Related Services.\nI\n23 Brief Description of Related\nTesting, Commissioning, Training, Repair,\nServices\nMaintenance and 01 (One) year warranty period.\n24 Tender Document Price\nBDT- 20000.00 (Twenty Thousand Only) (Non-\nrefundable)\nPROCURING ENTITY DETAILS\n25 Name of Official Inviting\nCommander A F M Zahidur Rahman, (E), BN.\nTender\n26\nDesignation of Official Inviting Project Director, Procurement Of Service Vessel For\nTender\nMongla Port Project, Mongla Port Authority, Mongla,\nBagerhat.\n27 Address of Official Inviting\nOffice of the Chief Engineer (Marine), Mechanical-\nTender\nMarine Building 1st Floor, Mongla Port Authority,\nMongla, Bagerhat, Bangladesh.\n28 Contact details of Official\nPhone: +8802477753801;\nInviting Tender\nEmail: cem@mpa.gov.bd\n29 The procuring entity reserves the right to accept or reject all applications\n30 This tender will be guided in accordance with \"The Public Procurement Act-2006 and\nRules-2008\".\nTHIS\nA F M Zahidur Rahman\nCommander, (E), BN\nNo-18.14.0158.132.00.040-1685\nProject Director &\nChief Engineer (Marine)\nMongla Port Authority,\nAR-509-440/21\nMongla, Bagerhat.\n", {"entities": [[280, 301, "inviter"], [319, 335, "inviter_location"], [646, 790, "description"]]}],
["Image No. 1363\nfile_name:/content/content/jpg_files/20210901-105159348-1034.txt\n\nGovernment of the People's Republic of Bangladesh\nLET IN a\nOffice of the Executive Engineer\n3\nEducation Engineering Department\n82\nKushtia.\nwww.eedmoe.gov.bd\nMemo No: 37.07.5000.08.00.001.2021/414\nDate : 31/08/2021 Eng.\ne-Tender Notice No :- xen/eed/kus/08/Vertical Extension(7016)/2021-2022\nR-Tender (OTM) is invited in the e-GP System Portal (http://yww.eprocure.gov.bd) for the procurement of following works details are\ngiven below.\nI\nTender\n$1.\nT\u00e9nder\nTender\nPackage No\nDescription of Works\nDocument\nNo\nID No.\nLast Selling/\nClosing\n(Date & Time)\n(Date & Time)\n0,\nEED-KUS-2021-\n603078\nVertical Extension of 1st & 2nd Floor of Existing Academic Building\n15/09/2021\n16/09/2021\nty\n22-7016-KSM.\nConstruction under The Project of Academic Building in Selected\n16.00\n13.00\nK\nDMHSS\nNon-Goyt. Secondary School (2000 School) at K.S.M. Dhaka Minapara\nHigher Secondary School in Sadar Upazilla under Kushtia District.\n,\n02\nEED-KUS-2021-\n603079\nVertical Extension of 1st & 2nd Floor including Sanitary, Water Supply\n15/09/2021\n16/09/2021\n22-7016-KPSS\nand Electrification Works at Kondorpodia Secondary School in Sadar\n16.00\n13.00\n,\nUpazilla under Kushtia District.\n03\nEED-KUS-2021-\n603080\nVertical Extension of 1st & 2nd Floor including Sanitary,Water Supply\n15/09/2021\n16/09/2021\n22-7016-SDSS\nand Electrification Works at Sankardia Secondary School in Sadar\n16.00\n13.00\nUpazilla under Kushtia District.\n<\nThis is an online Tender, Where only e-Tender will be accepted in the National e-GP Portal and no offline/hard topies will be accepted.\nTo sulsmit e-Tender, registration in the National e-GP System Portal (http://www.eprocure.gov.bd) in required. The fees for downloading\nthe e-Tender Documents from the National e-GP System Portal have to be deposited online through any registers Banks. branches up\nto abcyve mentioned date & time.\nFurthey, information and guidelines are available. in the National e-GP System Portal and from e-GP help desk\n(http:/ywww.eprocure.gov.bd).\n(Md:\nExecutive Engineer\nKushtia\n\\\nTel : 071-62438\nDC-11855/21 (6x4)\ne-mail :ee_kus@eedmoe.gov.bd\nthat\n", {"entities": [[140, 172, "co_inviter"], [175, 207, "inviter"], [211, 218, "inviter_location"], [372, 381, "call_type"], [669, 736, "description"], [1017, 1087, "description"], [1123, 1148, "description"]]}],
["Image No. 1358\nfile_name:/content/content/jpg_files/20210901-105647271-1035.txt\n\nJanata Bank Limited\nLocal Office\n98\n01, Dilkusha, C/A, Motijheel Dhaka-1000\nTel: 9560056\nWebsite: www.janatabank-bd.com or www.jb.com.bd\nINVITATION FOR TENDER\nSealed tenders are invited under the provisions of the Public Procurement Rules-2008 for the Goods and related Services\nand\nunder the terms and conditions outlined below and detailed in the Tender Document:\n1.\nTender Ref.\n:\nJBL/LO/LTM-04/10PC/2021\n2.\nTender Name\n:\nProcurement of Workstation (PC) (Supply and Installation) for Local office of Janata\nBank Limited on Turn-Key Basis following Limited Tendering Method.\n3. Brief description of Goods/Services: The items shall be internationally renowned brand item. No clone/Assembled items will be accepted:\nSL\nItems\nUnit of Measurement\nQty\nCategory-l: Workstation\nI\nWorkstation (PC)\nPiece\n10\n4.\nTime for completion of delivery\n:\nReady Stock.\n5.\nEligibility\n:\n5.1 Enlisted Tenderers of the Catrgory-1 of Information and Communications Technology Department-System.\nof\nJanata Bank Limited, Head office. Dhaka and interested vendor are eligible to participate in this Tender.\nTenderers\n5.2 The participating company may offer proposais for the items in any or all Categories and must quate all the items in\nthat/those Categories. Evaluation will be done by Category wise but contract will be awarded to onc. Partial offer for any\nCategory will not be accepted.\n5.3 The offers should have validity for at least 90 (Ninety) days from the date of opening the tender.\n6.\nName and address of the\n:\nJanata Bank Limited, Local Office, (2nd floor), 01 Dilkusha C/A. Dhaka-1000\nOffice(s) for Collection of\nbetween 10.00 a.m. and 04.00 p.m. on all working days.\nTender Document\ninterested Tenderers may view the Notice at the website of the Bank. (Website:\nI\nwww.ianatabank-bd.com or www.jb.com.bd)\n7.\nPrice of Tender Document\n:\nTender Document will be provided free of cost\n1\n8.\nAmount of Tender Security\n:\nTender Security shall not be required\n9.\nLast Date and Time for Collection of Tender Document\n:\n01-09-2021 within 04.00 PM.\n10.\nLast Date and Time for Submission of Tenders\n:\n02-09-2021 within 03.00 PM.\n11.\nName and address of the Office(s) Receiving Tenders\nAs at SI No.-6.\n12.\nName and address of the Office Opening Tenders\n:\nAs at SI No. -6.\n13,\nDate and Time for Opening of Tenders\n:\n02-09-2021 within 03.30 PM.\nTenderer's authorized representative(s) are allowed to attend, if any.\n14.\nSpecial\na The procuring entity reserves the right to reject all tenders or annul the Tender proceedings without\nInstructions:\nshowing any reason.\nb. The Bank shall not be under any obligation to accept the lowest quotation.\nc. If any *Force Majeure* situation arises in the specified date mentioned in the Tender Data Sheet and\nInvitation For Tender, the cvent will take place on the next working date.\nArif Akmed\nDeputy General Manager\nLocal office\nJanata Bank Limited\nSanata\n* Cell\n", {"entities": [[81, 100, "inviter"], [136, 151, "inviter_location"], [505, 562, "description"]]}],
["Image No. 1118\nfile_name:/content/content/jpg_files/20210901-105821441-1036.txt\n\nWJ\nJanata Bank Limited\nDivisional Office, Khulna.\n98\n8 KDA, C/A, Khulna.\nPhone:02477-725369, 730673.\nEmail: khulna@janatabank-bd.com\nWebsite: www.jb.com.bd\nPROCUREMENT METHOD: LIMITED TENDERING METHOD.\nTENDER DOCUMENT\nFOR THE PROCUREMENT OF\n16(SIXTEEN) PIECES PC FOR AREA/BRANCHES UNDER JANATA\nBANK LIMITED, DIVISIONAL OFFICE, KHULNA.\nInvitation for Tender No: JBL/DOK/LTM-204/2021\nIssued on: 22-08-2021\nThe deadline for submission of Tenders is 03.00 noon on 05-09-2021.\nTender Document Issued To:\nFemis\n", {"entities": [[84, 103, "inviter"], [123, 129, "inviter_location"], [307, 343, "description"]]}],
["Image No. 1375\nfile_name:/content/content/jpg_files/20210901-120215437-1050.txt\n\nBAKHRABAD GAS DISTRIBUTION COMPANY LIMITED\n(A Company of Petrobangla)\nHead office, Chapapur, Cumilia\nEngineering Service Department, Cumilla.\n(Webste:www.bgdcl.org.bd/\nwalds\nReference No.: 28.11.0000.710.38.005.21/534\nDate: 25/08/2021.\ne-Tender Notice (OTM)\ne-Tender is invited in the National e-GP Portal (http://www.eprocure.g\u00f3v.bd for the procurement of:\nTender\nPackage NO.\nDescription of Works\nLast Selling\nClosing\nOpening\nID NO\nDate and\nDate\nDate and\ntime\nand time\ntime\n.\n595723.\n28.11.0000.\nPainting of DRS/ CMS of\n20-Sep-2021\n21-Sep-2021\n21-Sep-2021\n710.38.003.21\nPoultry Farm, Cumilla\n17:00\n12:00\n12:00\nCentral Jailkhana,\nCantonment, Chandina,\nChandina Power Plant,\nGauripur HP DRS,\nNandanpur HP/ IP DRS, EPZ\nOld, BGDCL Office Complex\nUnder BGDCL Gas\nDistribution Areas.\nThis is an online Tender, where only e-Tender will be accepted in the National e-GP Portal and no\noffline/hard copies will be accepted.\nTo submit e-Tender, registration in the Nationale-GPsystemPortal(http://www.eprocure.gov.bd)isrequired\nThe fees for downloading the e-Tender Documents from the National e-GP System Portal have to be\ndeposited online through any registered Bank Branches.\nFurther information and guidelines are available in the National e-GP System Portal and from e-GP\nhelp desk (helpdesk@eprocure.gov.bd.)\nFor more details please contact to the PE's Support desk (081-68660)\nM\nin\n31\n26.08.2021.\n(Engr. Mortuza Rahman Khan)\nDeputy General Manager\nC-1291 (8x3)\nEngineering Service Department, Cumilla\nBGDCL, Chapapur, Cumilla-3500\n", {"entities": [[81, 123, "inviter"], [164, 181, "inviter_location"], [317, 333, "call_type"]]}],
["Image No. 1037\nfile_name:/content/content/jpg_files/20210901-120215437-1051.txt\n\n9111\nOffice of the\nFIRIT\nExecutive Engineer\n937 ROM\nChattogram O&M Division-II\n<1915641,\nBWDB, Chattogram.\n.\nBREAK ous. -BOOKS\nTel.No. 031-650612\nE-mail: xen.ctg2@gmail.com\nMemo No: EE/CTG-2/T-6/289\nDate; 29-08-2021\ne-Tender Notice-02/2021-22\nRe-tender: ist Call\nAn e-Tender has been invited in the national e-GP System Portal (http://www.\u00e9procure.gov.bd) for procurement of the\nfollowing works.\nSI\nTender\nPackage No. & Description\nLocation of the\nTender/Proposal\nTender/Proposal\nNo.\nID\nWorks/Supply\nDocument Last\nClysing &\nSelling Date &\nOpening Date &\nTitne\nTime\n1\n601982\nPackage No: BR/CTG-2-15\nMatiranga,\n09-September-2021\n09-September-2021\nBank Protective work at the left bank.of Feni river.\nKhagrachari\n2.30 pm\n4.00 pm\nA\n(Border River) from km. 9.750-to-km, 10.900 5\n1150m pear Karalachara BGB camp (East side) in\nUpazilla- Matiranga, Dist- Khagrach\u00e4ri in Connection\nwith Border River Bank Protection & Development\nProject<(Phase-2), Under Chittagong O&M Divisith\nII, BWDB, Chittagong during the year 2020-\n2021 1.(Package N\u00f3: BR/CTG-2-15). (Remaining part)\nThis is an online Tender, where only e-Tender will be accepted in the National e GP portal and. no offline/hard copies will\nbe accepted.,To, submit e-Tender, r\u00e9gistration in the Fational e-GP System Portal {http://www.cprocture.gov.bd) is required.\nThe fees for downloading the e-Tender- Documents from the National e-GP, p\u00f3rtal have\" to be deposited online\nthrough any registered Bank's branches up to 2:30 pm on 09-09-2021\nFurther information and guidelines are available in, the National e-GP System Portal and from e-GP help desk\n(helpdesk@eprocure.gov. bd) (01762625528-3+ (http://www eprocute. gov. 6d)\n20/08/19\n.\n(Nahid-U3-Zaman Khan)\nExecutiv\u00e9 Engineer\nChattogram O&M Division-II\nC-1294 (7x3)\nBWDB, Chattogram.\n", {"entities": [[170, 174, "inviter"], [176, 186, "inviter_location"], [86, 124, "co_inviter"], [324, 333, "call_type"], [726, 747, "description"]]}],
["Image No. 1059\nfile_name:/content/content/jpg_files/20210901-120215437-1053.txt\n\nGovernment of the People's Republic of Bangladesh\nRoads & Highways Department\nOffice of the Superintending Engineer\nRoad Circle, Moulvibazar.\nPhone+Fax: 02-41110074, 0861-62821\n-mail:serhdmoulvibazar@gmail.com\nI\nMemo No.35.01.5800.187.07.7-96.21-1412\nDate: 31/08/2021\n-\n-Tender Notice\nThis is to notify all concern that the following tender have published trough e-GP portal.\nTender ID, Package No.\nName of Work\nLast selling\nClosing and\n& Date of Publishing\ndate\nOpening\ndate & time\nTender ID No. 603139\nPeriodic Maintenance Programme by Providing Rigid\n29.09.2021,\n30.09.2021,\nTender Package No.\nPavement work at Ch. 0+030Km (Night Chowmohona) to\nUp to 17:00\nAt 13:00\n0)/e-GP/PMP Road\nCh. 0+750km (Juri Bazar), Ch. 0+800Km (Juri Bazar) to Ch.\nMajor/SE/RHDMRC/2021-22\n1+400Km (Juri Food Godowa) of Juri-Lathitila Road (R-\nDate of Publishing\n282) under Road Division, Moulvibazar during the year\n01/09/2021 at 12:00 PM\n2021-2022.\n.\nThis is an online tender, where only e-Tenders will be accepted in c-GP Portal and no offline/ hard copies\nwill\nbe accepted. To submit e-Tender, please register on e-GP System Portal\n(http://www.eprogure.gov.bg). e-Tenders are invited in e-GP System Portal (http://www.eprocure.gov.bd/\nby Superintending Engineer, RHD, Road Circle, Moulvibazar.\nFurther information's and guidelines are available in the National e-GP System portal and from e-GP help\ndesk (helpdesk: www.eprocure.gov.bd)\nes\n76\n(A.K. Shamsuddin\nID No.000379\nSay\nSuperintending Exginect, MHD\nRoad Circle, Moulvibazar\n(6x3)\nCG - 1896/21\n", {"entities": [[131, 158, "inviter"], [159, 196, "co_inviter"], [197, 222, "inviter_location"], [585, 634, "description"], [678, 691, "description"]]}],
["Image No. 1104\nfile_name:/content/content/jpg_files/20210901-120215437-1055.txt\n\nInvitation for Tenders\nBangladesh Shipping Corporation\nBSC Bhaban, Saltgola Road, Chattogram-4100, Bangladesh\nwww.bsc.gov.bd\nGOVERNMENT OF THE PEOPLE'S REPUBLIC OF BANGLADESH\n1\nMinistry/Division\nMinistry of Shipping\n2\nAgency\nBangladesh Shipping Corporation, Chattogram\n3\n.Purchaser Name\nBangladesh Shipping Corporation, Chattogram\n3.1\nPurchaser Department\nShip Stores Management (SSM)\n4\nPurchaser District.\nChattogram\n5\nInvitation for\nEnlistment of Marine Lube oil supplier(s) for the worldwide\nsupply of marine Lube oils to all the vessels under BSC\n(Bangladesh Shipping Corporation) fleet for two (02) years.\n6\nInvitation Ref No\n18.16.0000.385.48.002.21\n7\nDate\n01-09-2021\nKEY INFORMATION\n8\nProcurement Method\nOpen Tendering Method OTM\nFUNDING INFORMATION\n.9\n-Budget and Source of Funds\nBSC's own fund\nPARTICULAR INRORMATION\n10\nTender Package No.\n01\n11\nTender Package Name\nLube oils for BSC vessels\n12\nTender Publication Date\n01-09-2021\n12.1\nTender Last Availability\n13-10-2021; 1600 Hrs (LT) BST; Ship Repair Department\n13\nTender Closing Date and Time\n4-10-2021; 1400 Hrs (LT) BST\n14\nTender Opening Date and Time\n14-10-2021; 1615 Hrs (LT) BST\n15\nName & Address of the office(s)\n16\n- Receiving Tender Document\nOffice of General Manager\nShip Repair Department (SRD)\nBangladesh Shipping Corporation, Saltgola Road, Chattogram-\n4100, E-mail: gm-srd@bsc.gov.bd\n17\n- Opening Tender Document\nOffice of General Manager (Ship Repair)\n19\nPre-Tender Meeting (Optional)\nWill not be held\nINFORMATION FOR TENDERER\n20\nEligibility of Tenderer\nThe Tenderer shall have a minimum of five (05) years of overall\nexperience in the supply of goods (Marine Lube Oil) and related\nservices and other eligibility criteria need to be fulfilled as per the\nTender Data Sheet.\n21\nBrief Description of Goods or Works\nWorldwide supply of Marine Lube oils to all the BSC vessels*\n22\nBrief Description of Related Services\nLab analysis, yearly (02 times) lab test sample kit supply and\ntechnical consultancy at free of cost.\nLotNo\nIdentification of Lot\nLocation\nTender SecurityAmount\n23\n1\nMarine Lube oils as per Section 6\nWorldwide\nUSD 1,500.00 or BDT 1,27,500.00\n(Sch\u00e9dule of Requirement)\nSea Port\nPurchaser DETAILS\n24\nName of Official Inviting Tender\nZiaur Rahman Chy\n25\n\"Designation of Official Inviting Tender\nGeneral Manager\nShip Repair Department(SRD)\n26\nAddress of Official Inviting Tender\nOffice of General Manager\nShip Repair Department(SRD)\nBangladesh Shipping Corporation, Saligola Road,\nChattogram-4100\n27\nCohtatet details of Official Inviting Tender\n+88 031 725651\ngm-srd@bse.gov.bd\n28\nThe Purchaser reserves the right to reject any tenders or annul the Tender proceedings at any time.\nImportant Terms and Conditions:\n1. Tenders will be evaluated on the basis of item wise lowest price.\n2.\nAn agreement will be made only among those who are agreed to our proposal of item wise lowest price.\n3.\nNo additional charge will be applicable for supplying Lube Oils (in Jetty, River Mooring, outer anchorage, buoy,\nbarge etc.) at all ports of the world including Bangladesh.\nAll documents related to tender are available in BSC's website-www.bsc.gov.bd\n(8x3)\nCG-1899/21\n", {"entities": [[104, 135, "inviter"], [148, 173, "inviter_location"], [501, 621, "description"]]}],
["Image No. 1084\nfile_name:/content/content/jpg_files/20210901-120439162-1057.txt\n\nGovernmentofthe,People's.Republic of B\u00e1ngladesh\n,\nDirectorate.General of Family Planning\nEogistics & Supply. Unit\nnote\n6269 COKS, C214\nTIG 3/3/9/3\n6, Karwanbazar. Dhaka 1215\nir\nInvitation for e-Tender (OFM, Net, Goods)\nMemo No. DGFP/L&S-3/Height & Weight/2021-22/17/17\nDate 31.08.2021\nthe 1 : i\n.l\nits\n1\ne- Tender is invited in the national e-GP portal, for ,the procurement of following goods:\nf\n\"\nSI.\nTender\nBrief Description\nLast Selling\nLast Closing Opening Date\nPackage No,\nNo.\nID No.\nof Goods\nDate & Time\nDate & Time\n& Time\nla\n1\n2 of\n+\n3.\n4\n,I\n5\n-\n6.\n7\n7\n2.\n,\n4\na\n4 >)\nIf\n2\nProcurement of,\n22 September\n22-Septembe\nse\n\u00f3l\n2021\n23 Sept\u00e9mber\n598909\nMCH/GD4Or/e29\nHeight & Weight\n2021\n.2021\nTime: 11400\nMachine\nTime: 01:00 P.M.\nTime: 11.00 PM,\nP.M.\n3\nThis is an online tender where only,e-tender) (online render) will be accepted through the national e-GP portal\nand no offline/hard copy will be accepted. To submit e-tender please register onre-GP portak.\nThe fees for downloading the e-tender, documents from the national e-GP portal have to be deposited online\nthrough any registered bank.\nFurther information and guidelines are available in- the national e-GP portal and from e-GP help desk\n(help Desk helpdesk@eprocure.gov.bd)/Contact Number (01762-625528-31)\nthis\nL31108/2023\nbr\n8\napt\n3\nto (Ratna Talukder)\n1\nDirector (Logistics : and Supply) &\nF\nLine Director (PSSM-FP)\nBNG-404/21(5\"x4)\nDirectorate G\u00e9nefal of Family Planning\n*1\n1,\nv\nPhone: 55012349\nI is f\nthe\nE\nE-mail: dirlsdgfp@gmail.com\n", {"entities": [[131, 169, "inviter"], [231, 249, "inviter_location"], [273, 281, "call_type"]]}],
["Image No. 1197\nfile_name:/content/content/jpg_files/20210901-120439162-1058.txt\n\nGovernment of the People's Republic of Bangladesh\nLocal Government Engineering Department\nOffice of the Upazila. Engineer\n\"\nLaksham, Cumilla.\nif\nBangladest\nwww.lged.gov.bd\n000\n:\n644\n12651\n$\nMemo No: 46.02.1972.902.14.036.21-742\n1,\nDate : 31.08.2021\n-Tender Notice No: 01/2021-22,\ne-Tender is invited through National e-GP'Portal http://www.epfocure. gov.bd) for the procurement of;\nSl.\nTender ID\nNo.\n& Ref.No,\nPackage No\nName of the Works\nDate & Time of closing\na\n3\ne\nRepair and. renovation of\n\"\n20 Sep, 2021\nor\n603169\nen\nTender/PEDP4/CUM/L:AK/\nLaksham URC : Laksham\n2020-21/W10.143\nCumilla:\nx\nBST 11:00:00\nI\n11s\ntha\nThis is an'online tender, where only.\u00e9-Tender will be acc\u00e9pted in the Natiorial-e-GP Portal and\" no\noffline/hard copies are allowed. 'To submit \u00e9-Tender, ;registration in the National e-GP Portal\nhttp://www.eprocure.gov.bd) is required. Any \u00f6thers conditions for this tender is stated in TDS.\nInterested Tenderers may communicate with the undersigned during office hours.\nIf\nis\ni.e\n11,\nIT. wht 08/6/20\nof\n8\n+\n(Misuk Kumer Datta)\nUpazila -Engineer\nle\n11\nLaksham, Cumilla.\nBNG-403/21 (6\" X 4)\n#F\nPhone; 0803251367\n21.2\niv\nue.laksham@lged.gov.bd\nT.\n", {"entities": [[131, 170, "inviter"], [171, 202, "co_inviter"], [205, 221, "inviter_location"], [361, 369, "call_type"], [549, 574, "description"], [626, 637, "description"]]}],
["Image No. 1206\nfile_name:/content/content/jpg_files/20210901-120439162-1060.txt\n\npla mass\n22/10/60 21/12/17\nFax: 49270201\nTel: 94270077\nBangladesh Agricultural Research Institute\nE-mail:se.building@bari.gov.bd\nGazipur-1701\nWeb: www.bari.gov.bd\n*.\nMemo no. 12.21.0000.010.41.468.20.120\n3\nDate: 29.08.2021\ne-TENDER NOTICE\nTender is invited. Through Nation\u00e4l e-GP System Portal-(http/kwwweprocure.gov.bd): for procurement of following works:\nSl\nTender\nTendering\nDescription of works\nOnline (e-GP\nOnflifie (e-GP\nno\nID No.\nMethod\nsystem) tender\nsystem). ,tender\npublication date &\nclosing date &\ntime\ntime\n1,\n601723\nOTM\nLand' Development? work \u00e0nder Strengthening of\n29-Atig-2021 15:50\n14-Sep-2021\nQilseed & -Pulses Research and Development in\n14:40\nBangladesh (1st r\u00e9vised), 'at Qilseed Research\nRARS Ishurdi.\n\u00e8\nThis is online tender, only e-Tender will be accepted in National e-GP Portal and no \u00f6ffline/hard copy will be accepted.\nRegistration is essential to submit e-Tender.\nFees have to be deposited for, downloading the tender documents. from e=GP Portal through any registered bank. For further.\ninformation please e-GP help desk\nEngr: Md. FerozurRRahman\nC-1293 (4x4)-\nSuperintending Engineer (In-charge)\nto\n", {"entities": [[136, 178, "inviter"], [210, 217, "inviter_location"], [304, 319, "call_type"], [615, 661, "description"]]}],
["Image No. 1120\nfile_name:/content/content/jpg_files/20210901-120439162-1061.txt\n\nRAFGET\nplate\nplate 3/2/19\nplate\n2/5 9/15/04 93 5/21/11/31\nC+D\nRist 942 6/9/2018 $148 (3%\nColoticall,\nE mail-pdmymnbadc20T8@ gmail.com\nis\nMemo No.-12.251.007.551.02.00.002.17-18/144\nDate : 31.08.2021\nE-T ender Notice\nE-Tenders are invited in e-GP portal (http://www.eprocure.gov.bd) by the Project Director, Mymensingh\nDivision-and Tangail- & Kishoreganj District of Dh\u00e2ka Division Minor- Irrigation Development Project, Dholadia,\nMymensingh-2200.for the following works.\n$1.\nDescription of Works\nTender\n.T ender\nTender\nTender ID No\nMethod\nPublishing Date\nClosing Date\nNo.\n& Time\n& Time\n2\n3\n4\n5\n6\n1\nO1.\n600225,600226\nRe-ex\u00e7avation of Khal/Nala\nOTM, NCT\n31- A ugust-2021\n16 Sep-2021\n21.00\n16.00\n600228-600248\n02.\n600249-600252\nConstruction of W /ater passing structure.\nOTM, NCT\n31. August-2021\n16 Sep-2021\n21.00\n16.00\n.\n601938\n601949-601953\n03.\n600253-600262\nExtension of existing-buried pipe line (500\nOT M, NCT\n31 August-2021\n16 Sep-2021\n,\nmeter each) for 2-Cusec old DTW scheme\n21.00\n16.00\n601961-601970\n16 Sep-2021\n04.\n600263-600284\nConstruction of buried pipe line (1000 meter\nOTM, NCT\n31 August-202\neach) for 2-cusec LLP. Scheme\n21.00\n16.00\n05.\n600285-600295\nConsdruction of Pump House\nOTM, NCT R\n31\u00b0 August-2021\n16 Sep-2021\n21.00\n\"\n-16.00\n601971-601975\n,\n600296-600297\nInstallation of Solar System and pump for\nOT M, NCT\n31 August-2021\n16 Sep-2021\n06.\n21.00\n2 16.00\ndugwell,\n600298-600299\nInstallation of Drip Trrigation and water\nOT M, NCT\n31 August-2021\n16 Sep-202]\n07.\nDistribution System in Dugwell scheme\n21.00\n16.00\n08.\n601954-601955\nConstruction of dugwell\nOT M, NCT\n31 August-2021\n16 Sep:2021\n21.00\n16.00\n09.\n601956-601960\nConstruction of buried pipe line (1000 meter OTM,NCT\n31 August-2021\n16 Sep-2021\neach) for 5-cusec LLP Scheme\ny\n21.00\n16.00\nN\u00b0\nThis is an online Tender, where,only E-Tenders will be accepted in n.e-GP portal and no offline/hard copi\u00e9s will be\naccepted. To submit e-Tender, Please register on e-GP\n31\n(Engr. M\u00f9hammad Badr\u00f9 Alam)\nProject Director,\nMymensingh Division and T angail & Kishoreganj District of Dhaka\nDivision Minor Irrigation\nC-1296 (7x4)\nDholadia, Mymensingh-2200.\n", {"entities": [[280, 296, "call_type"], [388, 398, "inviter_location"], [697, 723, "description"], [806, 847, "description"], [939, 977, "description"], [1245, 1271, "description"], [1356, 1393, "description"], [1476, 1517, "description"], [1559, 1596, "description"], [1627, 1650, "description"], [195, 199, "inviter"]]}],
["Image No. 1329\nfile_name:/content/content/jpg_files/20210901-120439162-1062.txt\n\nGovernment of the people's Republic of Bangladesh\noffice of the Director & Professor\nNational Institute of Cardiovascular Diseases\nSher-e- Bangla Nagar, Dhaka-1207\nNo. NICVD/Administration/ MSE (Rev.) /2021-2022/3171\nDate- 31/08/2021\nInvitation for Tender\n1\nMinistry/Division\nMinistry of Health and family welfare\n2\nAgency\nNational Institute of Cardiovascular Diseases & Hospital\n3\nProcuring entity Name\nNational Institute of Cardiovascular Diseases & Hospital\n4\nInvitation for\nMSR Group No-Group-llinen Items, Group-2Chemical Re-agent & Group-3Fumiture\n5\nInvitation Ref. No\nNICVD/Administration/ MSR (Rev.)/2021-2022/3171\n6\nDate\nDate: 31 /08/2021.\nKEY INFORMATION\n7\nprocurment Method\nOp\u00e9n tendering Method-OTM\nFUNDING INFORMATION\n8\nBudget and Source of Funds\nRevenue Budget (GOB)\nPARTICULAR INFORMATION\n9\nTender Publication Date\nAfter Invitation of tender in Newspaper.\n10\nTender last Selling Date & Time\n28/09/2021 (Office hour)\n11\nTender Submission Date & time\n29/09/2021 10.00 AM to 12:30PM\n12\nTender Opening Date & time\n29/09/2021. 1.30 PM\nNAME AND ADDRESS OF THE OFFICE\n13\nSelling tender document\nTender Document will be available only on following web site:\nNICVD Web site- www.nlcvd.gov.bd\n14\nReceiving Tender Document\n1) Office of the Director & Professor, National Institute of Cardiovascular Diseases,\nSher-e- Bangla Nagar, Dhaka-1207.\n2) Deputy Police Comissinor (Tejgaon), 22/5, Khilzi road, Mohammadpur, Dhaka-1207.\n15\nOpening Tender Document\nOffice of the Director & Professor, National Institute of Cardiovascular Diseases, Sher-\ne- Bangla Nagar, Dhaka-1207.\nINFORMATION FOR TENDERER\n16\nEligibility of Tenderer\nTenderer from Group-1 Linen Items- Suppliers, Group-2 Chemical Re-agent-\nManufacturers/Authorised Dealer of Manufacturers/Supplies Authorised by the\nAuthorised Dealer of the Manufacturers & Group-3 Furniture- Manufacturers.\n17\nBrief Description of Goods.\nGroup No- 1 Linen Items, Group-2 Chemical Re-agent & Group-3 Furniture.\n.\ncode No. 325210S\n18. Price of Tender Document (Tk.):\nGroup\nName of the Group\nPrice of tender documents\nTender security Amount\nNo\n(In Challan)\n(Pay-Order)\n01\nLinen Items\nTk. 3,000/- (Three thousand) only\nTk. 6,00,000/- (six lac) only.\n02\nChemical Re-agent\nTk.\n3,000/- (Three thousand) only\nTk.\n5,00,000/- (five lac) only.\n03\nFurniture\nTk.\n3,000/- (Three thousand) only\nTk. 2,00,000/- (two lac) only.\nPROCURING ENTITY DETAILS\n19\nCode No. for deposition of tender challan\nThe tender documents purchese price of above mentioned tender documents (Non-\ndocuments.\nrefundable) is to be deposited in the Bangladesh bank by Treasury challan in code No.-\n1-2711-0000-2366 to director and professor, NICVD, Dhaka-1207. & Tender Security\n(Refundable) is to be deposited to Director and professor, NICVD, Dhaka-1207,\n20\nName of Official Inviting Tender\nProf. Dr. Mir Jamal Uddin.\n21\nDesignation of Official Inviting Tender\nDirector & Professor\n22\nAddress of Official Inviting Tender\nNational Institute of Cardiovascular Diseases, Sher-e- Bangla Nagar, Dhaka-1207.\n23\nContract ,details- of Official Inviting\nProcurement Section, National Institute of Cardiovascular Diseases. Phone-0248111290,\nTender\nFax-8142986, Web Site- www.nicvg.gov.bd & E-mail- nicvd@hospi.dghs.gov.bd\n24\nAuthority is not bound to accept the lowest tender/rate. Authority may cancel the whole tender/part of the tender without assigning any.reason.\n25\nTerms and Conditions of the tender and other information will be available with the schedule.\nGAB\n(Prof. Dr. Mir Jamal \u00daddin)\nDirector & Professor\nCG-1895/21\n(10x4)\n", {"entities": [[166, 211, "inviter"], [212, 239, "inviter_location"], [1974, 2033, "description"], [544, 634, "description"], [2274, 2291, "description"]]}],
["Image No. 1247\nfile_name:/content/content/jpg_files/20210901-120439162-1064.txt\n\nmash\nplate\n$142\nFax: 49270201\n3 321/515 21/22/T\nTel: 94270077\nE-mail: se.building@bari.gov.bd\nBangladesh Agricultural Research Institute\nWeb: www.bari.gov.bd\nGazipur-1701\nMemo no. 12.21.0000.010.41.468.20.110\nDate: 29.08.2021\ne-Tender Notice\nTender is invited Through National e-GP System Portal (http//:www\u00e9procure.gov.bd) for procurement of following works:\nSI\nTender\nTendering\nDescription of works\nOnline (e-GP\nOnline (e-GP\nno'\nID No.\nMethod\nsystem) tender\nsystem) tender\npublication date &\nclosing date &\ntime\ntime\n1.'\n601738\nOTM\nConstruction of Irrigation infrastructureat SRSC,\n29-Aug-2021\n15-Sep-2021\nBARI, Faridpur under Strengthening of Spices Crop*\n15:35\n14:30\n.\nResearch in Bangladesh\n2.\n599740\nOTM\nLand Development work at SRSC, BARI,\n29-Aug-2021\n15-Sep-2021\nLalmonirhat under Strengthening of Spices Crop\n15:55\n14:40\nResearch in Bangladesh\nFJ\n3.\n599736*\nFOTM\nLand D\u00e9velopment work`at RSRC, BARI, Gazipur\n29-Aug-2021\n15-Sep-2021\n7\nunder Strehgthening of Spices Crop. Research in\n15:50\n14:20\nBangladesh\n4.\n599727\nOTM\nLand Development work at SRSC, BARI, Faridpur\n29-Aug-2021\n15-Sep-2021\nunder Strengthening of Spices Crop Research in\n15:55\n14:00\nBangladesh\nThis is online tender, only e-Tender will be accepted in National e-GP Portal and no offline/hard copy will be accepted.\nRegistration is essential to submit -Tender.\nFees have to be deposited for downloading the tender documents from e-GP Portal through any registered bank. For further\ninformation please contact e-GP help desk (helpdesk@eprocure.gov.bd)\nEngr. Md. Ferozur Rahman\n76\nSuperintending Engineer (In-charge)\n(6x4)\nCG-1898/21\n", {"entities": [[175, 217, "inviter"], [615, 663, "description"], [791, 820, "description"], [953, 974, "description"], [239, 246, "inviter_location"], [307, 322, "call_type"]]}],
["Image No. 1178\nfile_name:/content/content/jpg_files/20210901-120439162-1063.txt\n\nGovernment of the People's Republic of Bangladesh\nLocal Government Engineering Department\nBanglad\nOffice of the Upazila Engineer\nAustagram, Kishoregonj.\nyou 644\nwww.lged.gov.bd\nyou\n.\nMemo No.: 46.02.4802.000.99,001.21.362\nDate : 31/08/2021.\ne-Tender Notice No-01/2021-22\ne Tender is invited in the national e-GP system portal (http://www.eprocure.gov.bd) for the procurement\nof following ID.\nSI\nTender\nPackage No.\nName of Work\nPublished\nTemder\nTender\nProcure-\nNo.\nID\nDate and\nLast sell-\nproposal\nment\nTime\ning Date\nOpening\nMedhod\nand Time\nDate and\nTime\n1\n2\n3\n-4\n5.\n6\n-\n7.\n8\n1\n600136\ne-Tender/AUS/\nInstallation of Central\n31/08/2021\n21/09/2021\n22/09/2021\nOTM\nJICA/2020-\nOxygen System Supply\n12.00 PM\n14.00 PM\n14.00 PM\n2021/01\nof austagram upazila\nHealth Complex Under\nAustagram\nUpazila,\nDist: Kishoreganj.\n2\n600137\ne-Tender/AUS/\nConstruction of Wash\n31/08/2021\n21/09/2021\n22/09/2021\nOTM\nJICA/2020-\nBlock and Passenger\n12.00 PM\n14.00 PM\n14.00 PM\n2021/02\nwaiting room at Kastol\nL4\nZero Point Under Aus-\ntagram Upazila, Dist:\n,\nKishoreganj.\n-\n3\n588873\ne-Tender/DEV/\nConstruction of Ansar\n31/08/2021\n21/09/2021\n22/09/2021\nLTM\nANSAR/2020-\nVaban for The Security\n12.00 PM\n14.00 PM\n14.00 PM\n2021\nof Upazil\u00e0 Nirbahi Officer\nUnder Austagram Upa-\nzila, Dist: Kishoreganj.\nThis-i is an online tender, where only e-Tender will be accepted. in the national e-GP portal and\nno\nof-\nfline/hard copies will be accepted. To submit e-Tender, please register on e-GP system (http://www.\neprocure.gov.hd) for more details please contact support desk contact numbers. Further, information\nand guidelines are available in the e-GP portal and also from e-mail : ue.austagram@lged.gov.bd or\ncall to 09422-50615.\nfor\n03/06/27\nMd. G\u00f3lam Samdani\n1\n76\nUpazila Engineer\nAustagram, Kishoregonj.\nE-mail: ue.austagram@lged.gov.bd\n(7x4)\nCG-1892/21\n3\n", {"entities": [[210, 232, "inviter_location"], [750, 770, "description"], [978, 997, "description"], [1033, 1046, "description"], [131, 170, "inviter"], [179, 209, "co_inviter"], [322, 337, "call_type"], [678, 701, "description"], [909, 929, "description"], [1143, 1164, "description"], [1214, 1219, "description"]]}],
["Image No. 1085\nfile_name:/content/content/jpg_files/20210901-120439162-1065.txt\n\n\"\nit\n\u20b9\n1\nE\nocam\n3\nRisalar 3 764491 918kh\nBANGLADESH COUNCIL OF SCIENTIFIC & INDUSTRIAL RESEARCH (BCSIR)\nDr. Qudrat-i-Khuda Road, Dhanmondi, Dhaka-1205, Bangladesh\nPABX : 861060, FAX: 88-028613022, 3-mail:bcsir@bangla.net\nMemo No.39.02.0000.030.40.218.2021/36\nDate: 31/08/2021\ne-Tender Notice (OTM)\n-Tender is invited in the national c-GP system Portal (http://www.eprocure.gov.bd) for the procurement of following\nTender ID.\nSI.\nTender\nLast Date and Time\nLast Date and, Time\nLast Date and Time\nName of Tender\nNo.\nID No\nfor Tender selling\nfor Tender Closing\nfor Tender opening\n01.\n601515\nRenovation & Modification (Civil &\n15-Sep-2021\n16-Sep-2021\n16-Sep-202)\nElectrical) work of 2 rooms of Pulp &\n(16:00)\n(15:00)\n(15:00)\npaper research division under Dhaka\n-\nlaboratory in the campus of BCSIR,\nDhanmondi, Dhaka.\n.\nThis is an online tender, where only e-tehder will be accepted in the National e-GP Portal and no offline/hard copies will,\nbe accepted.' To submit e-tender, registration in the National e-GP system Portal (http://www.eprocure.gov.bd) is required.\nMon\n76\nMiles.21\n(Md. Monir Hossain Patwary)\nSuperintending Engineer\n'BCSIR, Dhaka.\n- (6x4)\nCG-1897/21\n", {"entities": [[122, 184, "inviter"], [210, 231, "inviter_location"], [357, 378, "call_type"], [668, 702, "description"], [801, 824, "description"], [739, 776, "description"]]}]]


In [ ]:
import re
trim_entity_spans(rr)

[["Image No. 1171\nfile_name:/content/content/jpg_files/20210901-100100768-946.txt\n\nGOVERNMENT OF THE PEOPLE'S REPUBLIC OF BANGLADESH\nMINISTRY OF HEALTH AND FAMILY WELFARE\nDIRECTORATE GENERAL OF FAMILY PLANNING\nLOGISTICS & SUPPLY UNIT (5TH FLOOR)\n6, KARWANBAZAR, DHAKA-1215, BANGLADESH.\nwww.dgfp.gov.bd\ne-Tender Notice (OTM, NCT, Goods)\nFile Name: DGFP/L&SS-2/Delivery Table/2021-22//55/13\nDate: 31/08/2021\ne-Tender\nis invited in the national e-GP pottal for the procurement of following goods:\nTender ID No &\nTender Package name\nLast Down-\nClosing\nOpening\nPackage No.\nloading/selling\nDate &\nDate\nDate &Time\nTime\n&Time\n1\n2\n3\n4\n5\nTender ID No: 598911\nProcurement of Delivery\n26/09/2021\n27/09/2021\n27/09/2021\nPackage No\nTable & Iron Cot Set\n16:00\n14:00\n14:00\n-\nMCH/GD43/e31\nThis is an online tender where only e-tender (online tender) will be accepted through the national e-GP\nportal\nand no offline/hard copy will be accepted. The fees for downloading the e

In [ ]:
# attach third bracket in output.json
with open('/content/train_final_docu.json', 'r') as istr:
    with open('/content/output_final.json', 'w') as ostr:
        for line in istr:
            line = line.rstrip('\n') + ','
            print(line, file=ostr)

In [ ]:
!python /content/content/Custom_NER_Spacy3/scripts/convert.py en /content/output.json /content/train.spacy

In [ ]:
!python /content/content/Custom_NER_Spacy3/scripts/convert.py en /content/eval_10.json /content/eval.spacy

Traceback (most recent call last):
  File "/content/content/Custom_NER_Spacy3/scripts/convert.py", line 31, in <module>
    typer.run(convert)
  File "/usr/local/lib/python3.7/dist-packages/typer/main.py", line 864, in run
    app()
  File "/usr/local/lib/python3.7/dist-packages/typer/main.py", line 214, in __call__
    return get_command(self)(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 829, in __call__
    return self.main(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 782, in main
    rv = self.invoke(ctx)
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 1066, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 610, in invoke
    return callback(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/typer/main.py", line 500, in wrapper
    return callback(**use_params)  # type: ignore
  File "/content/

In [ ]:
!python -m spacy init config --lang en --pipeline ner /content/content/base_config.cfg --force


⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
/content/content/base_config.cfg
You can now add your data and train your pipeline:
python -m spacy train base_config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
!spacy debug data -V /content/content/base_config.cfg --paths.train /content/train.spacy --paths.dev /content/train.spacy



============================ Data file validation ============================
✔ Pipeline can be initialized with data
✔ Corpus is loadable

=============================== Training stats ===============================
Language: en
Training pipeline: tok2vec, ner
148 training docs
148 evaluation docs
⚠ 148 training examples also in evaluation data
⚠ Low number of examples to train a new pipeline (148)
It's recommended to use at least 2000 examples (minimum 100)

============================== Vocab & Vectors ==============================
ℹ 78051 total word(s) in the data (6286 unique)
10 most common words: ' ' (13183), '-' (2933), '.' (2602), ',' (2551), 'of'
(1956), ')' (1746), '(' (1645), 'Tender' (1209), 'the' (1150), '/' (1094)
ℹ No word vectors present in the package

========================== Named Entity Recognition ==========================
ℹ 7 label(s)
0 missing value(s) (tokens with '-' label)
Labels in train data: 'description', 'co_inviter', 'international',
'tender_ty

In [ ]:
!python -m spacy init fill-config /content/content/base_config.cfg /content/config.cfg

⚠ Nothing to auto-fill: base config is already complete
✔ Saved config
/content/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
!python -m spacy train /content/content/base_config.cfg --output /content/ --paths.train /content/train.spacy --paths.dev /content/train.spacy --training.eval_frequency 10 --training.max_steps 1000 --gpu-id -1


ℹ Saving to output directory: /content
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2021-09-30 09:57:50,961] [INFO] Set up nlp object from config
[2021-09-30 09:57:50,980] [INFO] Pipeline: ['tok2vec', 'ner']
[2021-09-30 09:57:50,990] [INFO] Created vocabulary
[2021-09-30 09:57:50,991] [INFO] Finished initializing nlp object
[2021-09-30 09:57:55,268] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00    370.26    0.00    0.00    0.00    0.00
  0      10         11.45   3619.78    0.00    0.00    0.00    0.00
  0      20        342.64    342.06    0.00    0.00    0.00    0.00
  0      30        249.59    154.28    0.

In [ ]:
import spacy
nlp1 = spacy.load("/content/model-best") #load the best model
doc = nlp1('Image No. 537\nfile_name:/content/350/20210902-104959963-1098.txt\n\nGovernment of the people, S Republe of Bangladesh\nOffice of the Superintendent,\n250 bedded General Hospital, Dinajpur.\nEmail-dinajpur@@hospi.dghs.gov.bd\nMemo po:G H.D/2021/1278\nDate-01/09/2021.\nINVITATION TENDER\nThe\nundersigned\nis\ninviting scaled Tenders from Contracter\'s for Diet Supply, Stationary and Others Supply for\n250 Bedded General Hospital, in FY 2021-2022\nThe details of the tender are given below\n1. Ministry/Division: Ministry of Health and Family welfare\n2. Agency: office of the Superintendent, 250 bedded General Hospital, Dinajpur.\n3. procuring Entity Name: Superintendent, 250 bedded General Hospital, Dinajpur.\n4. procuring Entity Code: - 1270210111160\n5., procuring Entity District: Dinajpur.\n6. Invitation for: Goods:\n7. Invitation Ref No: GHD/-2021-2022/1278.\n8. Date:01.09:2021\n84\nKEY INFORMATION\n9:\nProcurement Method: NCT.Open Tendering Mèthod (OTM)\nFUNDING INFORMAYTION,\n10. Budget and Source of Funds: Revenue Budget of GOB\n11. Development Partner: N/A\nPARTICULAR INFORMANON\n12.. Project/Programme Code:\n13. Project/Programme.Name:\n14. Tender Package No: 2 ka & Kha)\n15. Tender Package Name: Ka= Diét, Kha= Other Stationery and,Cleaning & Washing Matetials.\n16. Tender publication late:06.09.2021.\n17. Tender Last seilling date :27.09.2021, at 2.30pm.\n18. Tender Closing date and Time: 12:00 Noon on 28.09.2021.\n19. Tender Opening date and time :12.30.pm on 28.09.2021.\n20. Name &- Address of the Office (S)\nSelling tender Document (principal): Superintendent, 250 bedded General Hospital, Dinajpur:\nSelling tender Document (Others): Office of the Superintendent, 250 bédded Genéral Hospital, Dinajpur.\nReceiving Tender Document\n: Office of the Superintendent, 250 bedded General Hospital, Dinajpur.\nOpening Tender Document\n: Office of the Superintendent, 250 bedded General Hospital,Dinajpur.\n21, Place/Date Time of pre- Tender Meeting (Optional) Office of the Superintendent, 250 bedded General\nHospital, Dinajpur. Date -\nINFORMATION FOR TENDERER-\n22: Eligibility of Tenderer : As per TDS\n23. Brief Description of GOOds Group ka= diet Suppply and Group Kha= Other Stationery and Cleaning\nWashing Matetials in FY2021-2022.\n24 Brief Descriptión of Related Services :NA\n25. Tender Document price Group Ka 750/- for Diet & Group Kha 500/-for Kha= Other Stationery and Cleaning\n& Washing Matetials.shedul (Non tefundable) by. (Code nó 1-2711-0000-2366) deposit- in Sonali Bank\nCorporate Branch, Dinajpur with CTR& Online Verification Copy.\nLOT NO IDENTIFICATION OF LOT LOCATION\nSECURITY AMOUNT\n26.\nGioup Ka\n250 Bedded General Hospital.Dinajpur.\n-2,50,000/-\n27.\nGroup Kha\n250 Bedded General Hospitral, Dinajpur')
print(doc.ents)

spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter

nlp1 = spacy.load("/content/model-best") #load the best model
doc = nlp1('Ter • of works_ al 12:03 PT30K 2020-2123 .com 33 GOVERNMENT OF THE PEOPLE S REPUBLIC OF BANGLADESH Office of the Divisional Engineer-1 #|00 BANGLADESH RAILWAY Memo No. 222-5/1(2019-20) I Dated: 20/06/2021 e-Tender Notice (Open Tender Method) e-Tender in invited in the national e-GP portal for following procurement interested tenderes are to visit http://www.eprocure.gov.bd website for details. SI. Tender ID, Ref. Name of works Last selling Tender NO. No. & Package No. date and closing Date Time & Time. 589470 On LKM-CDR branch Repairing road 08-Jul-2021 08-Jul-2021 DEN-1(CTG)/ surface and Barriar Pole with overhauling 11:00 12:00 2020-21/22 of level crossing Gate No. LC-1/T at Km. 130/2-3 in between LKM-CIR station with other allied works. 2 589527 Construction of Bank protection wall at 08-Jul-2021 08-Jul-2021 -DEN-1(CTG) KM. 41/1-2 DN line 150 Rft between 11:00 2020-21/23 STK-BYA Station with other allied works orang penting 19.  S** as per drawing. 589528 At SLHR-Through repairs to Qrt. No. E/9 08-Jul-2021 08-Jul-2021 DEN-1(CTG)/ A type-2 Units E/15 S type-2 units E/11 A 11:00 12:05 2020-21/24 type 2 Units Total 6 Units including roof treatment and other allied works. 589531 On CN branch Construction of ballast 08-Jul-2021 08-Jul-2021 DEN-1(CTG)/ protection wall with Precast R.C.C Slab 11:00 12:08 2020-21/25 and Post at Km. 20/5-21/1-2 Left side 2200-0 rft including repairs to cess & bank with other allied works as per drawing. 5 589541 On Main line Construction of Ballast Wall 08-Jul-2021 08-Jul-2021 DEN-1(CTG)/ at K.M. 90/3-91/8 DN Line 2600 rit 11:00 12:10 2020-21/26 Between FNI-SRS Station K.M. 117/1-3 DN line 400 rft between NKT-NOT station Total 3000 Rft with Others allied works as per drawing No CE/E-6275 589543 On main line Construction of Bank 08-Jul-2021 08-Jul-2021 DEN-1(CTG)/ protection wall Rail Pilling With Drum 11:00 12:12 2020-21/27 Sheet Bandelling at KM. 65/8-66/0 Up line 200 Rit and Km. 65/8-9 Dn line 170 Rft between MSTN-CNKA Station with other allied works as per drawing. 7 589546 Construction of Water Hydrant at NKA 08-Jul-2021 08-Jul-2021 DEN-1(CTG)/ Station 700 Rft with Water Connection and 11:00 12:16 2020-21/28 other allied works per approved drawing. 8 589658 At CDR Construction of Water Hydrant at 08-Jul-2021 08-Jul-2021 DEN-1(CTG)/ CDR Station 1290 Rft with Water 11:00 12:15 2020-21/29 Connection and other allied works as per approved drawing. 9 589669 On Main line Construction of Ballast Wall 08-Jul-2021 08-Jul-2021 DEN at 50/2-7 up line 1800 rft between BYA- 11:00 12:17 1(CTG)/2020-21/30 BTKAK.M. 65/3-7 DN line 1000 ft Between MSTN-CNKA Station Total 2800 Rft with Others allied works as per drawing No CE/E-6275 10 589673 on LKM-NKA branch line Manufacturing 08-Jul-2021 08-Jul-2021 DEN- fitting & fixing Caution Board 1For Speed 11:00 12:20 1(CTG)/2020-21/31 Breaker of Authorized LC gate Caution Board 2 Authorized LC gate Whistle Board for Authorized gate at Km.130/3-4130/9- 131/0131/3-4131/8-9134/7-8154/0-1154/1- 2164/2-3 Total 08 Nos of Level Crossing gate & Whistle Board for major Bridge at K.m. 166/5-6 Between CMH-MZD Station. 11 589677 Re-construction of boundary wall at 08-Jul-2021 08-Jul-2021 DEN- Chandpur officers rest house with other 11:00 12:22 1(CTG)/2020-21/32 | ancillary works as per approved drawing 12 589746 On CN Branch Construction of Bank 08-Jul-2021 08-Jul-2021 DEN- protection wall at Km. 23/9-10 Right side 11:00 12:25 1(CTG)/2020-21/33 275-0 including repairs to cess & Bank othet allied works in between KZI-NZA station as per drawing. This is an online Tender, where only e-Tenders will be accepted in the National e-GP portal and no offline/hard copies will be accepted. as OG S(21)(189) Md. Abdul Hanif Divisional Engineer-1 Bangladesh Railway (Chittagong) পিসি-৮৮৯/২১ ')
spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter


nlp1 = spacy.load("/content/model-best") #load the best model
doc = nlp1('Government of the People s (73 Republic of Bangladesh Office of the Executive Engineer, DPHE শেখ হাসিনার মূলনীতি গ্রাম শহরের উন্নতি Bagerhat District, Bagerhat Memo No. 46.03.0100.061.07.032,15-1057/50 Dated: 27/06/2021 e-GP Tender Notice No. 242020-21) e-Tender is invited in the National e-GP System Portal (http://www.eprocure.gov.bd) for the Procurement of the following works. Details are given below: The interested personş/firm may visit the website www.eprocure.gov.bd to get the details of the tender. 1 1 Ministry/Division Local Government Division. 2 | Implementing agency Department of Public Health Engineering. 3 Project/program name Revenue, DPHE Dhaka. 4 e-Tender IDs 590227, 590226, 590225, 590224, 590223, 590222. 5 Description of works 1. Rain Water Harvesting (RWH) at different upazila in Bagerhat district under revenue budget FY-2020-2021, Land & Houseless Family through Mujib Barsho. This is an online tender, where only e-Tender will be accepted in the National e-GP Portal and no offline/hard copy will be accepted. To submit e-Tender, registration in the National e-GP System Portal is required. Further information and guidelines are available in the National e-GP System Portal and e- GP help desk (helpdesk@eprocure.gov.bd). Size: 4 x3 Executive Engineer GD-1276 DPHE, Bagerhat ')
spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter


nlp1 = spacy.load("/content/model-best") #load the best model
doc = nlp1("Government of the Peoples Republic of Bangladesh Office of the Chief Engineer (WEST) Bangladesh Railway, Rajshahi. 24 No. 54.01.8100.155.02.060.21 Date : 26.06.2021 Invitation for e-Tenders SL No Date & 1. 2. 3. at 12.00 4. 5. Tender Name of Works Tender Tender Tender ID No. Publication Document Document Last Selleg Closing Time Downloading Date & Date & Time Time 573115 Construction of a New RCC BOX Culvert 27.06.2021 14.07.2021 15.07.2021 Bridge No 53/J/A (1 X 5.0 Meter, DG at 12.00 at 17.00 Hrs. at 12,30 Standard) at KM. 484/8-485/0 in between Hrs. Hrs. station Balabari-Ramnabazar as per approved plan No.CE/W/8706. 573128 Construction of a RCC BOX Bridge 27.06.2021 14.07.2021 at 15.07.2021 (2x3.0 Meter, DG Standard) in place of (2x5 ft) at 12,00 17.00 Hrs. at 12.15 Deck Girder Bridge No. 80/F at KM. 416/8- Hr s \u041drg. 417/0 in between station Bamondanga Chowdhurani as per approved plan. 583072 Construction of Platform Shed (Size: 150 ft 27.06.2021 14.07.2021 at 15.07.2021 34 A) including dismatling of Existing 17.00 Hrs. at 12.00 Platform Shed at Platform No-1 with other Hrs. Hrs. ancillary works at Abdulpur station as per approved plan. 590529 Construction of new platform shed (size: 300 ft 27.06.2021 14.07.2021 15.07 2021 X 14 ft) at Istand platform in between line no. at 12.00 at 17.00 Hrs. at 12.00 3 and with other ancillary works at Bir Hrs. Hrs. Muktijoddha Sirajul Islam station, Panchagarh 2.9 per approved plan. 583066 Extension of Platfom Shed (Size: 100 ft x 22 27.06.2021 14.07.2021 15.07.2021 ft) at Panchbibi end with other ancillary works at 12.00 at 17.00 Hrs. at 12.45 at Joypurhat station as per approved plan. Hrs. His. 583069 Extension of Platform Shed (Size: 150 ft x 22 27.06.2021 14.07.2021 at 15.07.2021 A) at Hili end and (Size: 25 ft x 17 ft + 125 ft x at 12.00 117.00 Hrs. at 12.15 22 ft) at Fulbari end with other ancillary works Hrs. Hrs. at Birampur station as per approved plan. 590356 Extension of Platform Shed (Size: 100 ft x 22 27.06.2021 14.07.2021 at 15.07.2021 A) at Nilphamari end including at 12.00 17.00 Hrs. at 12.30 Strengthening and improvement of existing Hrs. Hrs. Platform Shed (Size: 200 A x 22 ft) at Platform No, 1 and Construction of a new Platfom Shed (Size: 300 ft x 22 ft) at Platform No. 2 with other ancillary works at Domar as per approved plan. 583070 Extension of Platform Shed (Size: 2 x 150 AX 27.06.2021 14.07.2021 at 15.07.2021 20 ft) at both end including Strengthening and 17.00 Hrs. at 12.45 Improvement of existing Platform Shed (Size: Hrs. Hrs. 22$ ft x 31 ft + 200 ft x 20 ft), with other ancillary works at Saidpur station as per approved plan. 590367 Extension of Platform Shed (Size: 150 ft x 22 27.06.2021 14.07.2021 st 15.07.2021 A) at Domar end including Strengthening and at 12.00 17:00 Hrs. at 12.30 Improvement of existing Platfom Shed (Size: Hrs. Hrs. 350, ft x 22 ft + 100 ft x 20 ft) with other ancillary works at Nilphamari station as per approved plan. This is an online tender where only e-tender will be accepted on e-GP portal and no offline and hard copy will be accepted. To submit e-tender please register on e-GP system (www.eprocuro.gov.bd). 6. 7. 8. at 12.00 9. N Chief Engineer/West Bangladesh Railway Rajshahi.")
spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter



(Office of the Superintendent, Memo po:G H.D/2021/1278, Dinajpur)
